In [ ]:
## This file implements neural networks before and after lasso selection for p0006kpresabs_qual with four replicates.
## We compute the mean and standarad deviation of training and test accuracies.
## We also compute the mean and standard deviation of AUC ROC values for each model.

In [1]:
from numpy.random import seed
import numpy as np
seed(100)
import tensorflow
tensorflow.random.set_seed(123)

In [2]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/p0006kpresabs_qual.csv')
df.shape

(253, 20)

In [3]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [4]:
df['pheno']

0      1
1      0
2      1
3      1
4      1
      ..
248    1
249    0
250    0
251    0
252    0
Name: pheno, Length: 253, dtype: int64

In [5]:
df.head()

,id,TTGTTATAGTC,TTAATTTAATAGA,TTAACATAATAAT,TGCAATCTCTTTAT,TATTATGTTAATG,TACATACCGAT,GTGTATCATAAT,GCTGTTGAAATGGC,GCAAACATGCG,GAGTCCTGTT,GAGTCCTGTTT,GACAAACATGTATTAGCGTTATGTCGCGAACATCATAACCAGCAACATGCGATTGGCGTTAAGTCGTTTGATGATAAATATCACTTGCATGACTCGTGG,CTTTTTCACCTGT,CTTGTGAATTTAG,CTTATAACAATTACTATATTTGGCATTATATGTAGTATTATTTTCACGAG,CGCCATTATGTT,CAGAAAAGCGT,ACAATTACTATATTT,pheno
0,107,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,0,1,1,1
1,109,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,0,1,1,0
2,115,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1
3,120335,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,0,1,1,1
4,120337,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,0,1,1,1


In [6]:
df['pheno'].value_counts()

0    129
1     85
2     39
Name: pheno, dtype: int64

In [7]:
df_clean = df.drop(columns=['id'])

In [8]:
df_clean.shape

(253, 19)

In [9]:
df_clean.head()

,TTGTTATAGTC,TTAATTTAATAGA,TTAACATAATAAT,TGCAATCTCTTTAT,TATTATGTTAATG,TACATACCGAT,GTGTATCATAAT,GCTGTTGAAATGGC,GCAAACATGCG,GAGTCCTGTT,GAGTCCTGTTT,GACAAACATGTATTAGCGTTATGTCGCGAACATCATAACCAGCAACATGCGATTGGCGTTAAGTCGTTTGATGATAAATATCACTTGCATGACTCGTGG,CTTTTTCACCTGT,CTTGTGAATTTAG,CTTATAACAATTACTATATTTGGCATTATATGTAGTATTATTTTCACGAG,CGCCATTATGTT,CAGAAAAGCGT,ACAATTACTATATTT,pheno
0,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,0,1,1,1
1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,0,1,1,0
2,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1
3,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,0,1,1,1
4,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,0,1,1,1


In [10]:
X = df.loc[:, df.columns != 'pheno']
y = df['pheno']
print(X.shape, y.shape)

(253, 19) (253,)


In [11]:
# over-sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_over, y_over = overS.fit_resample(X, y)
print(sorted(Counter(y_over).items()))

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.bagging module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.base module is  deprecated in versi

[(0, 129), (1, 129), (2, 129)]


/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is depreca

In [12]:
############# Fully-Connected Neural Network ################

In [13]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1

In [14]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_over)

In [15]:
dat = pd.DataFrame(X_test_over[:,0])
dat['test'] = y_test_over

In [16]:
dat

,0,test
0,NRS383,1
1,NRS254,1
2,NRS218,1
3,NRS215,0
4,BCH-SA-14,2
...,...,...
112,NRS227,1
113,NRS272,1
114,CFBRSa24,0
115,CFBRSa74,0


In [17]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [18]:
#### neural network on over-sampling data
model1_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [19]:
model1_over.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [20]:
model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 270 samples, validate on 117 samples
Epoch 1/1000
270/270 [==============================] - 0s 1ms/step - loss: 1.1545 - accuracy: 0.2519 - val_loss: 1.1221 - val_accuracy: 0.2735
Epoch 2/1000
270/270 [==============================] - 0s 107us/step - loss: 1.1250 - accuracy: 0.2593 - val_loss: 1.1014 - val_accuracy: 0.2991
Epoch 3/1000
270/270 [==============================] - 0s 130us/step - loss: 1.1092 - accuracy: 0.3111 - val_loss: 1.0872 - val_accuracy: 0.3761
Epoch 4/1000
270/270 [==============================] - 0s 139us/step - loss: 1.0985 - accuracy: 0.3296 - val_loss: 1.0775 - val_accuracy: 0.3675
Epoch 5/1000
270/270 [==============================] - 0s 122us/step - loss: 1.0883 - accuracy: 0.3889 - val_loss: 1.0634 - val_accuracy: 0.4444
Epoch 6/1000
270/270 [==============================] - 0s 102us/step - loss: 1.0786 - accuracy: 0.4296 - val_loss: 1.0493 - val_accuracy: 0.4701
Epoch 7/1000
270/270 [==============================] - 0s 118us/step - loss: 1.

Epoch 56/1000
270/270 [==============================] - 0s 65us/step - loss: 0.9533 - accuracy: 0.5037 - val_loss: 0.9514 - val_accuracy: 0.5128
Epoch 57/1000
270/270 [==============================] - 0s 66us/step - loss: 0.9517 - accuracy: 0.5111 - val_loss: 0.9519 - val_accuracy: 0.5299
Epoch 58/1000
270/270 [==============================] - 0s 73us/step - loss: 0.9531 - accuracy: 0.5296 - val_loss: 0.9554 - val_accuracy: 0.5299
Epoch 59/1000
270/270 [==============================] - 0s 107us/step - loss: 0.9531 - accuracy: 0.4963 - val_loss: 0.9540 - val_accuracy: 0.5128
Epoch 60/1000
270/270 [==============================] - 0s 109us/step - loss: 0.9512 - accuracy: 0.5259 - val_loss: 0.9556 - val_accuracy: 0.5299
Epoch 61/1000
270/270 [==============================] - 0s 126us/step - loss: 0.9507 - accuracy: 0.5222 - val_loss: 0.9486 - val_accuracy: 0.5299
Epoch 62/1000
270/270 [==============================] - 0s 125us/step - loss: 0.9492 - accuracy: 0.4963 - val_loss: 0.95

270/270 [==============================] - 0s 86us/step - loss: 0.9128 - accuracy: 0.5370 - val_loss: 0.9487 - val_accuracy: 0.5299
Epoch 167/1000
270/270 [==============================] - 0s 93us/step - loss: 0.9053 - accuracy: 0.5481 - val_loss: 0.9468 - val_accuracy: 0.5299
Epoch 168/1000
270/270 [==============================] - 0s 112us/step - loss: 0.9068 - accuracy: 0.5444 - val_loss: 0.9468 - val_accuracy: 0.5299
Epoch 169/1000
270/270 [==============================] - 0s 142us/step - loss: 0.9067 - accuracy: 0.5222 - val_loss: 0.9483 - val_accuracy: 0.5299
Epoch 170/1000
270/270 [==============================] - 0s 122us/step - loss: 0.9065 - accuracy: 0.5148 - val_loss: 0.9498 - val_accuracy: 0.5299
Epoch 171/1000
270/270 [==============================] - 0s 139us/step - loss: 0.9051 - accuracy: 0.5481 - val_loss: 0.9477 - val_accuracy: 0.5299
Epoch 172/1000
270/270 [==============================] - 0s 117us/step - loss: 0.9038 - accuracy: 0.5481 - val_loss: 0.9457 - va

Epoch 277/1000
270/270 [==============================] - 0s 86us/step - loss: 0.8870 - accuracy: 0.5481 - val_loss: 0.9461 - val_accuracy: 0.5385
Epoch 278/1000
270/270 [==============================] - 0s 91us/step - loss: 0.8830 - accuracy: 0.5519 - val_loss: 0.9525 - val_accuracy: 0.5385
Epoch 279/1000
270/270 [==============================] - 0s 99us/step - loss: 0.8788 - accuracy: 0.5519 - val_loss: 0.9481 - val_accuracy: 0.5128
Epoch 280/1000
270/270 [==============================] - 0s 84us/step - loss: 0.8832 - accuracy: 0.5407 - val_loss: 0.9457 - val_accuracy: 0.5214
Epoch 281/1000
270/270 [==============================] - 0s 90us/step - loss: 0.8795 - accuracy: 0.5407 - val_loss: 0.9463 - val_accuracy: 0.5385
Epoch 282/1000
270/270 [==============================] - 0s 119us/step - loss: 0.8811 - accuracy: 0.5519 - val_loss: 0.9514 - val_accuracy: 0.5385
Epoch 283/1000
270/270 [==============================] - 0s 133us/step - loss: 0.8802 - accuracy: 0.5481 - val_loss:

Epoch 388/1000
270/270 [==============================] - 0s 114us/step - loss: 0.8640 - accuracy: 0.5481 - val_loss: 0.9467 - val_accuracy: 0.5470
Epoch 389/1000
270/270 [==============================] - 0s 177us/step - loss: 0.8632 - accuracy: 0.5481 - val_loss: 0.9433 - val_accuracy: 0.5470
Epoch 390/1000
270/270 [==============================] - 0s 146us/step - loss: 0.8632 - accuracy: 0.5593 - val_loss: 0.9457 - val_accuracy: 0.5470
Epoch 391/1000
270/270 [==============================] - 0s 170us/step - loss: 0.8639 - accuracy: 0.5593 - val_loss: 0.9492 - val_accuracy: 0.5470
Epoch 392/1000
270/270 [==============================] - 0s 153us/step - loss: 0.8628 - accuracy: 0.5704 - val_loss: 0.9485 - val_accuracy: 0.5299
Epoch 393/1000
270/270 [==============================] - 0s 113us/step - loss: 0.8659 - accuracy: 0.5481 - val_loss: 0.9492 - val_accuracy: 0.5299
Epoch 394/1000
270/270 [==============================] - 0s 102us/step - loss: 0.8634 - accuracy: 0.5407 - val_

...

270/270 [==============================] - 0s 78us/step - loss: 0.8418 - accuracy: 0.5481 - val_loss: 0.9591 - val_accuracy: 0.5470
Epoch 722/1000
270/270 [==============================] - 0s 84us/step - loss: 0.8435 - accuracy: 0.5593 - val_loss: 0.9677 - val_accuracy: 0.5470
Epoch 723/1000
270/270 [==============================] - 0s 80us/step - loss: 0.8444 - accuracy: 0.5519 - val_loss: 0.9588 - val_accuracy: 0.5299
Epoch 724/1000
270/270 [==============================] - 0s 82us/step - loss: 0.8424 - accuracy: 0.5407 - val_loss: 0.9609 - val_accuracy: 0.5470
Epoch 725/1000
270/270 [==============================] - 0s 77us/step - loss: 0.8418 - accuracy: 0.5556 - val_loss: 0.9590 - val_accuracy: 0.5470
Epoch 726/1000
270/270 [==============================] - 0s 79us/step - loss: 0.8408 - accuracy: 0.5593 - val_loss: 0.9620 - val_accuracy: 0.5470
Epoch 727/1000
270/270 [==============================] - 0s 87us/step - loss: 0.8431 - accuracy: 0.5593 - val_loss: 0.9596 - val_acc

Epoch 777/1000
270/270 [==============================] - 0s 61us/step - loss: 0.8400 - accuracy: 0.5556 - val_loss: 0.9657 - val_accuracy: 0.5470
Epoch 778/1000
270/270 [==============================] - 0s 58us/step - loss: 0.8408 - accuracy: 0.5333 - val_loss: 0.9641 - val_accuracy: 0.5299
Epoch 779/1000
270/270 [==============================] - 0s 60us/step - loss: 0.8398 - accuracy: 0.5222 - val_loss: 0.9640 - val_accuracy: 0.5470
Epoch 780/1000
270/270 [==============================] - 0s 60us/step - loss: 0.8394 - accuracy: 0.5593 - val_loss: 0.9619 - val_accuracy: 0.5470
Epoch 781/1000
270/270 [==============================] - 0s 59us/step - loss: 0.8389 - accuracy: 0.5630 - val_loss: 0.9663 - val_accuracy: 0.5470
Epoch 782/1000
270/270 [==============================] - 0s 59us/step - loss: 0.8399 - accuracy: 0.5630 - val_loss: 0.9634 - val_accuracy: 0.5470
Epoch 783/1000
270/270 [==============================] - 0s 62us/step - loss: 0.8448 - accuracy: 0.5593 - val_loss: 0

Epoch 888/1000
270/270 [==============================] - 0s 64us/step - loss: 0.8377 - accuracy: 0.5481 - val_loss: 0.9643 - val_accuracy: 0.5299
Epoch 889/1000
270/270 [==============================] - 0s 63us/step - loss: 0.8394 - accuracy: 0.5593 - val_loss: 0.9711 - val_accuracy: 0.5470
Epoch 890/1000
270/270 [==============================] - 0s 63us/step - loss: 0.8384 - accuracy: 0.5630 - val_loss: 0.9665 - val_accuracy: 0.5299
Epoch 891/1000
270/270 [==============================] - 0s 60us/step - loss: 0.8407 - accuracy: 0.5519 - val_loss: 0.9664 - val_accuracy: 0.5299
Epoch 892/1000
270/270 [==============================] - 0s 60us/step - loss: 0.8391 - accuracy: 0.5481 - val_loss: 0.9717 - val_accuracy: 0.5470
Epoch 893/1000
270/270 [==============================] - 0s 58us/step - loss: 0.8394 - accuracy: 0.5630 - val_loss: 0.9635 - val_accuracy: 0.5470
Epoch 894/1000
270/270 [==============================] - 0s 56us/step - loss: 0.8384 - accuracy: 0.5556 - val_loss: 0

Epoch 944/1000
270/270 [==============================] - 0s 74us/step - loss: 0.8366 - accuracy: 0.5370 - val_loss: 0.9710 - val_accuracy: 0.5470
Epoch 945/1000
270/270 [==============================] - 0s 75us/step - loss: 0.8413 - accuracy: 0.5630 - val_loss: 0.9694 - val_accuracy: 0.5470
Epoch 946/1000
270/270 [==============================] - 0s 74us/step - loss: 0.8353 - accuracy: 0.5630 - val_loss: 0.9613 - val_accuracy: 0.5299
Epoch 947/1000
270/270 [==============================] - 0s 73us/step - loss: 0.8387 - accuracy: 0.5519 - val_loss: 0.9631 - val_accuracy: 0.5214
Epoch 948/1000
270/270 [==============================] - 0s 75us/step - loss: 0.8414 - accuracy: 0.5370 - val_loss: 0.9668 - val_accuracy: 0.5470
Epoch 949/1000
270/270 [==============================] - 0s 78us/step - loss: 0.8354 - accuracy: 0.5630 - val_loss: 0.9622 - val_accuracy: 0.5299
Epoch 950/1000
270/270 [==============================] - 0s 81us/step - loss: 0.8370 - accuracy: 0.5519 - val_loss: 0

Epoch 1000/1000
270/270 [==============================] - 0s 73us/step - loss: 0.8364 - accuracy: 0.5630 - val_loss: 0.9742 - val_accuracy: 0.5470


In [21]:
acc_test_over = model1_over.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over*100))

117/117 [==============================] - 0s 89us/step
over-sampling test accuracy: 53.85%


In [21]:
pred = model1_over.predict_classes(X_test_over)
pred

array([0, 1, 1, 0, 0, 0, 0, 0, 2, 2, 0, 2, 1, 0, 0, 0, 2, 1, 0, 2, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 2, 2, 0, 1, 1, 0, 0, 2, 0, 2,
       0, 1, 1, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 1, 0,
       0, 0, 1, 1, 1, 0, 2, 0, 1, 1, 0, 0, 2, 1, 0, 2, 0, 0, 2, 1, 0, 2,
       0, 0, 1, 2, 0, 0, 1])

In [22]:
dat['pred'] = pred
dat

,0,test,pred
0,NRS383,1,0
1,NRS254,1,1
2,NRS218,1,1
3,NRS215,0,0
4,BCH-SA-14,2,0
...,...,...,...
112,NRS227,1,1
113,NRS272,1,2
114,CFBRSa24,0,0
115,CFBRSa74,0,0


In [23]:
proba1 = model1_over.predict_proba(X_test_over)
dat_proba1 = pd.DataFrame(proba1)

In [24]:
dat_proba1

,0,1,2
0,0.510052,0.256485,0.233462
1,0.272272,0.453846,0.273881
2,0.272272,0.453846,0.273881
3,0.994780,0.002158,0.003063
4,0.444397,0.374179,0.181424
...,...,...,...
112,0.272272,0.453846,0.273881
113,0.000551,0.005500,0.993949
114,0.444397,0.374179,0.181424
115,0.444397,0.374179,0.181424


In [25]:
dat_proba1.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba1.csv", index = False,
         header=None)

In [26]:
dat.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/1p006.csv", index = False,
         header=None)

In [25]:
hist1_over = model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 270 samples, validate on 117 samples
Epoch 1/1000
270/270 [==============================] - 0s 112us/step - loss: 0.8447 - accuracy: 0.5111 - val_loss: 0.9665 - val_accuracy: 0.5299
Epoch 2/1000
270/270 [==============================] - 0s 112us/step - loss: 0.8335 - accuracy: 0.5481 - val_loss: 0.9670 - val_accuracy: 0.5470
Epoch 3/1000
270/270 [==============================] - 0s 124us/step - loss: 0.8465 - accuracy: 0.5630 - val_loss: 0.9734 - val_accuracy: 0.5470
Epoch 4/1000
270/270 [==============================] - 0s 123us/step - loss: 0.8336 - accuracy: 0.5667 - val_loss: 0.9692 - val_accuracy: 0.5299
Epoch 5/1000
270/270 [==============================] - 0s 98us/step - loss: 0.8382 - accuracy: 0.5519 - val_loss: 0.9705 - val_accuracy: 0.5470
Epoch 6/1000
270/270 [==============================] - 0s 106us/step - loss: 0.8343 - accuracy: 0.5630 - val_loss: 0.9737 - val_accuracy: 0.5470
Epoch 7/1000
270/270 [==============================] - 0s 120us/step - loss: 0

Epoch 56/1000
270/270 [==============================] - 0s 237us/step - loss: 0.8344 - accuracy: 0.5333 - val_loss: 0.9634 - val_accuracy: 0.5214
Epoch 57/1000
270/270 [==============================] - 0s 161us/step - loss: 0.8342 - accuracy: 0.5407 - val_loss: 0.9647 - val_accuracy: 0.5470
Epoch 58/1000
270/270 [==============================] - 0s 138us/step - loss: 0.8344 - accuracy: 0.5630 - val_loss: 0.9686 - val_accuracy: 0.5470
Epoch 59/1000
270/270 [==============================] - 0s 141us/step - loss: 0.8322 - accuracy: 0.5593 - val_loss: 0.9704 - val_accuracy: 0.5299
Epoch 60/1000
270/270 [==============================] - 0s 160us/step - loss: 0.8361 - accuracy: 0.5519 - val_loss: 0.9706 - val_accuracy: 0.5299
Epoch 61/1000
270/270 [==============================] - 0s 112us/step - loss: 0.8338 - accuracy: 0.5630 - val_loss: 0.9746 - val_accuracy: 0.5470
Epoch 62/1000
270/270 [==============================] - 0s 123us/step - loss: 0.8385 - accuracy: 0.5630 - val_loss: 0

Epoch 112/1000
270/270 [==============================] - 0s 90us/step - loss: 0.8365 - accuracy: 0.5630 - val_loss: 0.9657 - val_accuracy: 0.5470
Epoch 113/1000
270/270 [==============================] - 0s 87us/step - loss: 0.8335 - accuracy: 0.5444 - val_loss: 0.9648 - val_accuracy: 0.5385
Epoch 114/1000
270/270 [==============================] - 0s 88us/step - loss: 0.8336 - accuracy: 0.5444 - val_loss: 0.9650 - val_accuracy: 0.5556
Epoch 115/1000
270/270 [==============================] - 0s 101us/step - loss: 0.8345 - accuracy: 0.5630 - val_loss: 0.9721 - val_accuracy: 0.5470
Epoch 116/1000
270/270 [==============================] - 0s 91us/step - loss: 0.8348 - accuracy: 0.5593 - val_loss: 0.9684 - val_accuracy: 0.5470
Epoch 117/1000
270/270 [==============================] - 0s 80us/step - loss: 0.8337 - accuracy: 0.5630 - val_loss: 0.9724 - val_accuracy: 0.5470
Epoch 118/1000
270/270 [==============================] - 0s 91us/step - loss: 0.8350 - accuracy: 0.5630 - val_loss: 

Epoch 168/1000
270/270 [==============================] - 0s 80us/step - loss: 0.8372 - accuracy: 0.5370 - val_loss: 0.9710 - val_accuracy: 0.5299
Epoch 169/1000
270/270 [==============================] - 0s 79us/step - loss: 0.8337 - accuracy: 0.5407 - val_loss: 0.9713 - val_accuracy: 0.5470
Epoch 170/1000
270/270 [==============================] - 0s 76us/step - loss: 0.8323 - accuracy: 0.5593 - val_loss: 0.9708 - val_accuracy: 0.5385
Epoch 171/1000
270/270 [==============================] - 0s 84us/step - loss: 0.8311 - accuracy: 0.5593 - val_loss: 0.9726 - val_accuracy: 0.5299
Epoch 172/1000
270/270 [==============================] - 0s 80us/step - loss: 0.8331 - accuracy: 0.5519 - val_loss: 0.9743 - val_accuracy: 0.5470
Epoch 173/1000
270/270 [==============================] - 0s 77us/step - loss: 0.8335 - accuracy: 0.5630 - val_loss: 0.9783 - val_accuracy: 0.5470
Epoch 174/1000
270/270 [==============================] - 0s 85us/step - loss: 0.8315 - accuracy: 0.5630 - val_loss: 0

270/270 [==============================] - 0s 109us/step - loss: 0.8321 - accuracy: 0.5667 - val_loss: 0.9721 - val_accuracy: 0.5470
Epoch 279/1000
270/270 [==============================] - 0s 80us/step - loss: 0.8333 - accuracy: 0.5630 - val_loss: 0.9768 - val_accuracy: 0.5470
Epoch 280/1000
270/270 [==============================] - 0s 87us/step - loss: 0.8323 - accuracy: 0.5630 - val_loss: 0.9756 - val_accuracy: 0.5470
Epoch 281/1000
270/270 [==============================] - 0s 81us/step - loss: 0.8332 - accuracy: 0.5222 - val_loss: 0.9753 - val_accuracy: 0.5385
Epoch 282/1000
270/270 [==============================] - 0s 118us/step - loss: 0.8317 - accuracy: 0.5630 - val_loss: 0.9787 - val_accuracy: 0.5385
Epoch 283/1000
270/270 [==============================] - 0s 101us/step - loss: 0.8316 - accuracy: 0.5519 - val_loss: 0.9791 - val_accuracy: 0.5214
Epoch 284/1000
270/270 [==============================] - 0s 105us/step - loss: 0.8327 - accuracy: 0.5333 - val_loss: 0.9750 - val

...

270/270 [==============================] - 0s 103us/step - loss: 0.8303 - accuracy: 0.5444 - val_loss: 0.9801 - val_accuracy: 0.5385
Epoch 609/1000
270/270 [==============================] - 0s 116us/step - loss: 0.8283 - accuracy: 0.5630 - val_loss: 0.9810 - val_accuracy: 0.5385
Epoch 610/1000
270/270 [==============================] - 0s 191us/step - loss: 0.8293 - accuracy: 0.5630 - val_loss: 0.9825 - val_accuracy: 0.5385
Epoch 611/1000
270/270 [==============================] - 0s 117us/step - loss: 0.8288 - accuracy: 0.5630 - val_loss: 0.9800 - val_accuracy: 0.5470
Epoch 612/1000
270/270 [==============================] - 0s 149us/step - loss: 0.8282 - accuracy: 0.5519 - val_loss: 0.9799 - val_accuracy: 0.5470
Epoch 613/1000
270/270 [==============================] - 0s 115us/step - loss: 0.8300 - accuracy: 0.5630 - val_loss: 0.9825 - val_accuracy: 0.5470
Epoch 614/1000
270/270 [==============================] - 0s 180us/step - loss: 0.8283 - accuracy: 0.5556 - val_loss: 0.9828 - 

270/270 [==============================] - 0s 104us/step - loss: 0.8326 - accuracy: 0.5444 - val_loss: 0.9838 - val_accuracy: 0.5470
Epoch 719/1000
270/270 [==============================] - 0s 97us/step - loss: 0.8294 - accuracy: 0.5519 - val_loss: 0.9831 - val_accuracy: 0.5470
Epoch 720/1000
270/270 [==============================] - 0s 105us/step - loss: 0.8312 - accuracy: 0.5630 - val_loss: 0.9829 - val_accuracy: 0.5470
Epoch 721/1000
270/270 [==============================] - 0s 105us/step - loss: 0.8320 - accuracy: 0.5444 - val_loss: 0.9799 - val_accuracy: 0.5385
Epoch 722/1000
270/270 [==============================] - 0s 114us/step - loss: 0.8284 - accuracy: 0.5444 - val_loss: 0.9815 - val_accuracy: 0.5385
Epoch 723/1000
270/270 [==============================] - 0s 110us/step - loss: 0.8282 - accuracy: 0.5630 - val_loss: 0.9823 - val_accuracy: 0.5470
Epoch 724/1000
270/270 [==============================] - 0s 140us/step - loss: 0.8339 - accuracy: 0.5630 - val_loss: 0.9781 - v

270/270 [==============================] - 0s 93us/step - loss: 0.8287 - accuracy: 0.5444 - val_loss: 0.9813 - val_accuracy: 0.5385
Epoch 829/1000
270/270 [==============================] - 0s 93us/step - loss: 0.8271 - accuracy: 0.5630 - val_loss: 0.9821 - val_accuracy: 0.5470
Epoch 830/1000
270/270 [==============================] - 0s 125us/step - loss: 0.8281 - accuracy: 0.5593 - val_loss: 0.9808 - val_accuracy: 0.5385
Epoch 831/1000
270/270 [==============================] - 0s 113us/step - loss: 0.8293 - accuracy: 0.5593 - val_loss: 0.9845 - val_accuracy: 0.5470
Epoch 832/1000
270/270 [==============================] - 0s 153us/step - loss: 0.8288 - accuracy: 0.5481 - val_loss: 0.9852 - val_accuracy: 0.5299
Epoch 833/1000
270/270 [==============================] - 0s 145us/step - loss: 0.8301 - accuracy: 0.5481 - val_loss: 0.9820 - val_accuracy: 0.5470
Epoch 834/1000
270/270 [==============================] - 0s 117us/step - loss: 0.8276 - accuracy: 0.5630 - val_loss: 0.9876 - va

Epoch 884/1000
270/270 [==============================] - 0s 95us/step - loss: 0.8271 - accuracy: 0.5630 - val_loss: 0.9857 - val_accuracy: 0.5299
Epoch 885/1000
270/270 [==============================] - 0s 80us/step - loss: 0.8295 - accuracy: 0.5519 - val_loss: 0.9855 - val_accuracy: 0.5299
Epoch 886/1000
270/270 [==============================] - 0s 98us/step - loss: 0.8280 - accuracy: 0.5667 - val_loss: 0.9860 - val_accuracy: 0.5470
Epoch 887/1000
270/270 [==============================] - 0s 97us/step - loss: 0.8294 - accuracy: 0.5630 - val_loss: 0.9840 - val_accuracy: 0.5470
Epoch 888/1000
270/270 [==============================] - 0s 111us/step - loss: 0.8301 - accuracy: 0.5630 - val_loss: 0.9876 - val_accuracy: 0.5470
Epoch 889/1000
270/270 [==============================] - 0s 125us/step - loss: 0.8269 - accuracy: 0.5630 - val_loss: 0.9856 - val_accuracy: 0.5299
Epoch 890/1000
270/270 [==============================] - 0s 118us/step - loss: 0.8282 - accuracy: 0.5519 - val_loss

270/270 [==============================] - 0s 231us/step - loss: 0.8295 - accuracy: 0.5630 - val_loss: 0.9930 - val_accuracy: 0.5470
Epoch 995/1000
270/270 [==============================] - 0s 183us/step - loss: 0.8293 - accuracy: 0.5630 - val_loss: 0.9830 - val_accuracy: 0.5470
Epoch 996/1000
270/270 [==============================] - 0s 100us/step - loss: 0.8279 - accuracy: 0.5444 - val_loss: 0.9838 - val_accuracy: 0.5556
Epoch 997/1000
270/270 [==============================] - 0s 104us/step - loss: 0.8284 - accuracy: 0.5630 - val_loss: 0.9819 - val_accuracy: 0.5556
Epoch 998/1000
270/270 [==============================] - 0s 129us/step - loss: 0.8283 - accuracy: 0.5667 - val_loss: 0.9826 - val_accuracy: 0.5470
Epoch 999/1000
270/270 [==============================] - 0s 129us/step - loss: 0.8309 - accuracy: 0.5370 - val_loss: 0.9829 - val_accuracy: 0.5299
Epoch 1000/1000
270/270 [==============================] - 0s 137us/step - loss: 0.8267 - accuracy: 0.5667 - val_loss: 0.9849 -

In [26]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over.history['accuracy'])*100))

over-sampling train accuracy: 55.28%


In [18]:
df_proba = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_2.xlsx",
                        sheet_name=0,
                        index_col=None)

In [19]:
df_proba

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabs_qual,CFBRSa26,0,0,0.758914,0.241086,4.638713e-07
1,p002ykpresabs_qual,NRS109,2,2,0.005361,0.016236,9.784034e-01
2,p002ykpresabs_qual,NRS112,0,0,0.726623,0.273376,1.520979e-06
3,p002ykpresabs_qual,NRS216,1,1,0.138322,0.861665,1.334123e-05
4,p002ykpresabs_qual,NRS021,0,0,0.882176,0.117824,1.414530e-10
...,...,...,...,...,...,...,...
4279,pyopresabsSTCC_qual,NRS148,2,2,0.000007,0.000099,9.998934e-01
4280,pyopresabsSTCC_qual,NRS255,2,2,0.000257,0.002048,9.976944e-01
4281,pyopresabsSTCC_qual,NRS205,2,2,0.000011,0.000045,9.999435e-01
4282,pyopresabsSTCC_qual,NRS255,2,2,0.000257,0.002048,9.976944e-01


In [20]:
y_prob = df_proba[df_proba['phage']=='p0006kpresabs_qual'].iloc[:,-3:]
y_prob = y_prob.to_numpy()
y_prob

array([[5.1005230e-01, 2.5648522e-01, 2.3346250e-01],
       [2.7227250e-01, 4.5384616e-01, 2.7388138e-01],
       [2.7227250e-01, 4.5384616e-01, 2.7388138e-01],
       [9.9477980e-01, 2.1576720e-03, 3.0625204e-03],
       [4.4439682e-01, 3.7417945e-01, 1.8142363e-01],
       [4.4439682e-01, 3.7417945e-01, 1.8142363e-01],
       [5.1005230e-01, 2.5648522e-01, 2.3346250e-01],
       [4.4439682e-01, 3.7417945e-01, 1.8142363e-01],
       [3.2683715e-04, 1.2991672e-08, 9.9967310e-01],
       [5.1640872e-02, 6.7280570e-04, 9.4768640e-01],
       [4.4439682e-01, 3.7417945e-01, 1.8142363e-01],
       [1.2761596e-02, 2.1610180e-01, 7.7113660e-01],
       [2.7227250e-01, 4.5384616e-01, 2.7388138e-01],
       [4.4439682e-01, 3.7417945e-01, 1.8142363e-01],
       [4.4439682e-01, 3.7417945e-01, 1.8142363e-01],
       [4.4439682e-01, 3.7417945e-01, 1.8142363e-01],
       [1.5892214e-03, 4.9088103e-01, 5.0752980e-01],
       [2.7227250e-01, 4.5384616e-01, 2.7388138e-01],
       [4.4439682e-01, 3.741

In [35]:
## Retrieved from https://github.com/scikit-learn/scikit-learn/issues/3298
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelBinarizer

def rocauc_ovo(truth, pred, average="macro", multi_class="ovo"):

    lb = LabelBinarizer()
    lb.fit(truth)

    truth = lb.transform(truth)   
    
    return roc_auc_score(truth, pred, average=average, multi_class=multi_class)

In [22]:
ovo1 = rocauc_ovo(y_test_over, y_prob, average="macro", multi_class="ovo")
ovo1

0.7098400175323253

In [36]:
def rocauc_ovr(truth, pred, average="macro", multi_class="ovr"):

    lb = LabelBinarizer()
    lb.fit(truth)

    truth = lb.transform(truth)   

    return roc_auc_score(truth, pred, average=average, multi_class=multi_class)

In [24]:
ovr1 = rocauc_ovr(y_test_over, y_prob, average="macro", multi_class="ovr")
ovr1

0.7098400175323253

In [25]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=234,
                                                    stratify=y_over)

In [26]:
dat2 = pd.DataFrame(X_test_over[:,0])
dat2['test'] = y_test_over

In [27]:
dat2

,0,test
0,BCH-SA-11,2
1,NRS161,1
2,BCH-SA-14,2
3,BCH-SA-11,2
4,CFBRSa49,1
...,...,...
112,NRS064,2
113,NRS266,2
114,NRS222,0
115,GA27,2


In [28]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [31]:
model1_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [32]:
model1_over2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [33]:
model1_over2.fit(X_train_over, y_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 270 samples, validate on 117 samples
Epoch 1/1000
270/270 [==============================] - 0s 392us/step - loss: 1.4445 - accuracy: 0.3370 - val_loss: 1.3060 - val_accuracy: 0.3248
Epoch 2/1000
270/270 [==============================] - 0s 98us/step - loss: 1.2325 - accuracy: 0.3111 - val_loss: 1.1691 - val_accuracy: 0.2821
Epoch 3/1000
270/270 [==============================] - 0s 86us/step - loss: 1.1538 - accuracy: 0.3296 - val_loss: 1.1367 - val_accuracy: 0.3248
Epoch 4/1000
270/270 [==============================] - 0s 88us/step - loss: 1.1366 - accuracy: 0.3074 - val_loss: 1.1217 - val_accuracy: 0.2906
Epoch 5/1000
270/270 [==============================] - 0s 120us/step - loss: 1.1248 - accuracy: 0.2667 - val_loss: 1.1093 - val_accuracy: 0.2821
Epoch 6/1000
270/270 [==============================] - 0s 83us/step - loss: 1.1139 - accuracy: 0.2778 - val_loss: 1.0952 - val_accuracy: 0.2906
Epoch 7/1000
270/270 [==============================] - 0s 106us/step - loss: 1.10

Epoch 112/1000
270/270 [==============================] - 0s 120us/step - loss: 0.9546 - accuracy: 0.5111 - val_loss: 0.9095 - val_accuracy: 0.5726
Epoch 113/1000
270/270 [==============================] - 0s 91us/step - loss: 0.9536 - accuracy: 0.5111 - val_loss: 0.9083 - val_accuracy: 0.5726
Epoch 114/1000
270/270 [==============================] - 0s 75us/step - loss: 0.9556 - accuracy: 0.4889 - val_loss: 0.9104 - val_accuracy: 0.5726
Epoch 115/1000
270/270 [==============================] - 0s 69us/step - loss: 0.9593 - accuracy: 0.5111 - val_loss: 0.9150 - val_accuracy: 0.5726
Epoch 116/1000
270/270 [==============================] - 0s 65us/step - loss: 0.9537 - accuracy: 0.5111 - val_loss: 0.9076 - val_accuracy: 0.5726
Epoch 117/1000
270/270 [==============================] - 0s 83us/step - loss: 0.9531 - accuracy: 0.5148 - val_loss: 0.9078 - val_accuracy: 0.5726
Epoch 118/1000
270/270 [==============================] - 0s 80us/step - loss: 0.9508 - accuracy: 0.5111 - val_loss: 

Epoch 223/1000
270/270 [==============================] - 0s 96us/step - loss: 0.9153 - accuracy: 0.5296 - val_loss: 0.9068 - val_accuracy: 0.5726
Epoch 224/1000
270/270 [==============================] - 0s 81us/step - loss: 0.9160 - accuracy: 0.5296 - val_loss: 0.9065 - val_accuracy: 0.5726
Epoch 225/1000
270/270 [==============================] - 0s 79us/step - loss: 0.9154 - accuracy: 0.5296 - val_loss: 0.9086 - val_accuracy: 0.5726
Epoch 226/1000
270/270 [==============================] - 0s 78us/step - loss: 0.9165 - accuracy: 0.5296 - val_loss: 0.9080 - val_accuracy: 0.5726
Epoch 227/1000
270/270 [==============================] - 0s 71us/step - loss: 0.9201 - accuracy: 0.5296 - val_loss: 0.9078 - val_accuracy: 0.5726
Epoch 228/1000
270/270 [==============================] - 0s 67us/step - loss: 0.9156 - accuracy: 0.5296 - val_loss: 0.9069 - val_accuracy: 0.5726
Epoch 229/1000
270/270 [==============================] - 0s 81us/step - loss: 0.9185 - accuracy: 0.5296 - val_loss: 0

Epoch 279/1000
270/270 [==============================] - 0s 85us/step - loss: 0.9047 - accuracy: 0.5296 - val_loss: 0.9118 - val_accuracy: 0.5641
Epoch 280/1000
270/270 [==============================] - 0s 64us/step - loss: 0.9034 - accuracy: 0.5259 - val_loss: 0.9154 - val_accuracy: 0.5726
Epoch 281/1000
270/270 [==============================] - 0s 65us/step - loss: 0.9040 - accuracy: 0.5296 - val_loss: 0.9130 - val_accuracy: 0.5726
Epoch 282/1000
270/270 [==============================] - 0s 63us/step - loss: 0.9063 - accuracy: 0.5296 - val_loss: 0.9141 - val_accuracy: 0.5641
Epoch 283/1000
270/270 [==============================] - 0s 83us/step - loss: 0.9088 - accuracy: 0.5296 - val_loss: 0.9132 - val_accuracy: 0.5641
Epoch 284/1000
270/270 [==============================] - 0s 66us/step - loss: 0.9057 - accuracy: 0.5296 - val_loss: 0.9173 - val_accuracy: 0.5726
Epoch 285/1000
270/270 [==============================] - 0s 76us/step - loss: 0.9054 - accuracy: 0.5296 - val_loss: 0

270/270 [==============================] - 0s 135us/step - loss: 0.8854 - accuracy: 0.5444 - val_loss: 0.9222 - val_accuracy: 0.5726
Epoch 390/1000
270/270 [==============================] - 0s 122us/step - loss: 0.8880 - accuracy: 0.5444 - val_loss: 0.9209 - val_accuracy: 0.5726
Epoch 391/1000
270/270 [==============================] - 0s 145us/step - loss: 0.8881 - accuracy: 0.5407 - val_loss: 0.9194 - val_accuracy: 0.5641
Epoch 392/1000
270/270 [==============================] - 0s 143us/step - loss: 0.8897 - accuracy: 0.5481 - val_loss: 0.9209 - val_accuracy: 0.5726
Epoch 393/1000
270/270 [==============================] - 0s 153us/step - loss: 0.8868 - accuracy: 0.5481 - val_loss: 0.9191 - val_accuracy: 0.5726
Epoch 394/1000
270/270 [==============================] - 0s 152us/step - loss: 0.8870 - accuracy: 0.5444 - val_loss: 0.9199 - val_accuracy: 0.5726
Epoch 395/1000
270/270 [==============================] - 0s 138us/step - loss: 0.8891 - accuracy: 0.5333 - val_loss: 0.9260 - 

...

Epoch 668/1000
270/270 [==============================] - 0s 80us/step - loss: 0.8624 - accuracy: 0.5519 - val_loss: 0.9385 - val_accuracy: 0.5726
Epoch 669/1000
270/270 [==============================] - 0s 82us/step - loss: 0.8628 - accuracy: 0.5519 - val_loss: 0.9428 - val_accuracy: 0.5726
Epoch 670/1000
270/270 [==============================] - 0s 84us/step - loss: 0.8615 - accuracy: 0.5519 - val_loss: 0.9391 - val_accuracy: 0.5726
Epoch 671/1000
270/270 [==============================] - 0s 105us/step - loss: 0.8617 - accuracy: 0.5519 - val_loss: 0.9415 - val_accuracy: 0.5726
Epoch 672/1000
270/270 [==============================] - 0s 122us/step - loss: 0.8648 - accuracy: 0.5519 - val_loss: 0.9384 - val_accuracy: 0.5726
Epoch 673/1000
270/270 [==============================] - 0s 142us/step - loss: 0.8629 - accuracy: 0.5519 - val_loss: 0.9423 - val_accuracy: 0.5726
Epoch 674/1000
270/270 [==============================] - 0s 114us/step - loss: 0.8639 - accuracy: 0.5519 - val_los

Epoch 779/1000
270/270 [==============================] - 0s 76us/step - loss: 0.8588 - accuracy: 0.5444 - val_loss: 0.9518 - val_accuracy: 0.5641
Epoch 780/1000
270/270 [==============================] - 0s 80us/step - loss: 0.8571 - accuracy: 0.5556 - val_loss: 0.9523 - val_accuracy: 0.5641
Epoch 781/1000
270/270 [==============================] - 0s 75us/step - loss: 0.8570 - accuracy: 0.5519 - val_loss: 0.9451 - val_accuracy: 0.5641
Epoch 782/1000
270/270 [==============================] - 0s 75us/step - loss: 0.8604 - accuracy: 0.5222 - val_loss: 0.9495 - val_accuracy: 0.5812
Epoch 783/1000
270/270 [==============================] - 0s 75us/step - loss: 0.8581 - accuracy: 0.5370 - val_loss: 0.9509 - val_accuracy: 0.5641
Epoch 784/1000
270/270 [==============================] - 0s 81us/step - loss: 0.8594 - accuracy: 0.5481 - val_loss: 0.9570 - val_accuracy: 0.5726
Epoch 785/1000
270/270 [==============================] - 0s 78us/step - loss: 0.8588 - accuracy: 0.5519 - val_loss: 0

Epoch 835/1000
270/270 [==============================] - 0s 72us/step - loss: 0.8556 - accuracy: 0.5444 - val_loss: 0.9520 - val_accuracy: 0.5726
Epoch 836/1000
270/270 [==============================] - 0s 79us/step - loss: 0.8560 - accuracy: 0.5481 - val_loss: 0.9506 - val_accuracy: 0.5641
Epoch 837/1000
270/270 [==============================] - 0s 73us/step - loss: 0.8545 - accuracy: 0.5519 - val_loss: 0.9478 - val_accuracy: 0.5726
Epoch 838/1000
270/270 [==============================] - 0s 73us/step - loss: 0.8561 - accuracy: 0.5519 - val_loss: 0.9487 - val_accuracy: 0.5726
Epoch 839/1000
270/270 [==============================] - 0s 78us/step - loss: 0.8550 - accuracy: 0.5519 - val_loss: 0.9468 - val_accuracy: 0.5726
Epoch 840/1000
270/270 [==============================] - 0s 77us/step - loss: 0.8555 - accuracy: 0.5185 - val_loss: 0.9483 - val_accuracy: 0.5897
Epoch 841/1000
270/270 [==============================] - 0s 77us/step - loss: 0.8558 - accuracy: 0.5444 - val_loss: 0

Epoch 891/1000
270/270 [==============================] - 0s 73us/step - loss: 0.8582 - accuracy: 0.5519 - val_loss: 0.9537 - val_accuracy: 0.5726
Epoch 892/1000
270/270 [==============================] - 0s 79us/step - loss: 0.8542 - accuracy: 0.5519 - val_loss: 0.9506 - val_accuracy: 0.5726
Epoch 893/1000
270/270 [==============================] - 0s 71us/step - loss: 0.8601 - accuracy: 0.5519 - val_loss: 0.9617 - val_accuracy: 0.5726
Epoch 894/1000
270/270 [==============================] - 0s 73us/step - loss: 0.8540 - accuracy: 0.5519 - val_loss: 0.9579 - val_accuracy: 0.5726
Epoch 895/1000
270/270 [==============================] - 0s 73us/step - loss: 0.8539 - accuracy: 0.5519 - val_loss: 0.9588 - val_accuracy: 0.5726
Epoch 896/1000
270/270 [==============================] - 0s 82us/step - loss: 0.8559 - accuracy: 0.5519 - val_loss: 0.9594 - val_accuracy: 0.5726
Epoch 897/1000
270/270 [==============================] - 0s 86us/step - loss: 0.8522 - accuracy: 0.5519 - val_loss: 0

Epoch 947/1000
270/270 [==============================] - 0s 84us/step - loss: 0.8568 - accuracy: 0.5519 - val_loss: 0.9554 - val_accuracy: 0.5726
Epoch 948/1000
270/270 [==============================] - 0s 84us/step - loss: 0.8544 - accuracy: 0.5111 - val_loss: 0.9538 - val_accuracy: 0.5641
Epoch 949/1000
270/270 [==============================] - 0s 72us/step - loss: 0.8555 - accuracy: 0.5519 - val_loss: 0.9564 - val_accuracy: 0.5641
Epoch 950/1000
270/270 [==============================] - 0s 80us/step - loss: 0.8535 - accuracy: 0.5519 - val_loss: 0.9517 - val_accuracy: 0.5641
Epoch 951/1000
270/270 [==============================] - 0s 89us/step - loss: 0.8541 - accuracy: 0.5074 - val_loss: 0.9492 - val_accuracy: 0.5726
Epoch 952/1000
270/270 [==============================] - 0s 75us/step - loss: 0.8525 - accuracy: 0.5519 - val_loss: 0.9479 - val_accuracy: 0.5726
Epoch 953/1000
270/270 [==============================] - 0s 80us/step - loss: 0.8526 - accuracy: 0.5519 - val_loss: 0

In [34]:
acc_test_over2 = model1_over2.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over2*100))

117/117 [==============================] - 0s 100us/step
over-sampling test accuracy: 55.56%


In [34]:
pred2 = model1_over2.predict_classes(X_test_over)
pred2

array([0, 1, 0, 0, 0, 2, 0, 1, 1, 0, 2, 0, 1, 0, 0, 2, 2, 0, 1, 1, 1, 0,
       0, 0, 0, 1, 2, 0, 2, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 2, 0, 1, 0, 0, 0, 0, 0, 0, 2, 2, 2, 1, 1, 0,
       0, 0, 1, 0, 0, 0, 1, 2, 0, 0, 0, 2, 1, 0, 0, 2, 0, 2, 0, 0, 1, 0,
       0, 0, 2, 1, 1, 1, 0, 0, 2, 2, 2, 2, 0, 0, 1, 0, 0, 0, 0, 2, 1, 0,
       0, 1, 2, 2, 0, 0, 0])

In [35]:
dat2['pred'] = pred2
dat2

,0,test,pred
0,BCH-SA-11,2,0
1,NRS161,1,1
2,BCH-SA-14,2,0
3,BCH-SA-11,2,0
4,CFBRSa49,1,0
...,...,...,...
112,NRS064,2,2
113,NRS266,2,2
114,NRS222,0,0
115,GA27,2,0


In [36]:
proba2 = model1_over2.predict_proba(X_test_over)
dat_proba2 = pd.DataFrame(proba2)

In [37]:
dat_proba2

,0,1,2
0,0.388714,0.380327,0.230959
1,0.191576,0.465061,0.343363
2,0.388714,0.380327,0.230959
3,0.388714,0.380327,0.230959
4,0.388714,0.380327,0.230959
...,...,...,...
112,0.000037,0.011136,0.988827
113,0.219150,0.004308,0.776542
114,0.533972,0.280422,0.185605
115,0.388714,0.380327,0.230959


In [38]:
dat_proba2.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba2.csv", index = False,
         header=None)

In [39]:
dat2.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/2p006.csv", index = False,
         header=None)

In [38]:
hist1_over2 = model1_over2.fit(X_train_over, y_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 270 samples, validate on 117 samples
Epoch 1/1000
270/270 [==============================] - 0s 102us/step - loss: 0.8440 - accuracy: 0.5593 - val_loss: 1.0299 - val_accuracy: 0.5726
Epoch 2/1000
270/270 [==============================] - 0s 96us/step - loss: 0.8462 - accuracy: 0.5519 - val_loss: 1.0241 - val_accuracy: 0.5726
Epoch 3/1000
270/270 [==============================] - 0s 119us/step - loss: 0.8446 - accuracy: 0.5519 - val_loss: 1.0193 - val_accuracy: 0.5641
Epoch 4/1000
270/270 [==============================] - 0s 100us/step - loss: 0.8455 - accuracy: 0.5519 - val_loss: 1.0195 - val_accuracy: 0.5641
Epoch 5/1000
270/270 [==============================] - 0s 93us/step - loss: 0.8449 - accuracy: 0.5519 - val_loss: 1.0251 - val_accuracy: 0.5726
Epoch 6/1000
270/270 [==============================] - 0s 81us/step - loss: 0.8450 - accuracy: 0.5519 - val_loss: 1.0229 - val_accuracy: 0.5726
Epoch 7/1000
270/270 [==============================] - 0s 82us/step - loss: 0.84

Epoch 57/1000
270/270 [==============================] - 0s 81us/step - loss: 0.8452 - accuracy: 0.5519 - val_loss: 1.0268 - val_accuracy: 0.5726
Epoch 58/1000
270/270 [==============================] - 0s 74us/step - loss: 0.8492 - accuracy: 0.5519 - val_loss: 1.0216 - val_accuracy: 0.5726
Epoch 59/1000
270/270 [==============================] - 0s 87us/step - loss: 0.8449 - accuracy: 0.5481 - val_loss: 1.0223 - val_accuracy: 0.5726
Epoch 60/1000
270/270 [==============================] - 0s 186us/step - loss: 0.8455 - accuracy: 0.5519 - val_loss: 1.0247 - val_accuracy: 0.5726
Epoch 61/1000
270/270 [==============================] - 0s 176us/step - loss: 0.8449 - accuracy: 0.5519 - val_loss: 1.0232 - val_accuracy: 0.5726
Epoch 62/1000
270/270 [==============================] - 0s 183us/step - loss: 0.8430 - accuracy: 0.5519 - val_loss: 1.0182 - val_accuracy: 0.5641
Epoch 63/1000
270/270 [==============================] - 0s 113us/step - loss: 0.8456 - accuracy: 0.5519 - val_loss: 1.02

Epoch 113/1000
270/270 [==============================] - 0s 105us/step - loss: 0.8445 - accuracy: 0.5519 - val_loss: 1.0340 - val_accuracy: 0.5726
Epoch 114/1000
270/270 [==============================] - 0s 145us/step - loss: 0.8453 - accuracy: 0.5519 - val_loss: 1.0285 - val_accuracy: 0.5726
Epoch 115/1000
270/270 [==============================] - 0s 98us/step - loss: 0.8450 - accuracy: 0.5222 - val_loss: 1.0331 - val_accuracy: 0.5641
Epoch 116/1000
270/270 [==============================] - 0s 93us/step - loss: 0.8456 - accuracy: 0.5444 - val_loss: 1.0318 - val_accuracy: 0.5641
Epoch 117/1000
270/270 [==============================] - 0s 96us/step - loss: 0.8424 - accuracy: 0.5519 - val_loss: 1.0361 - val_accuracy: 0.5726
Epoch 118/1000
270/270 [==============================] - 0s 97us/step - loss: 0.8506 - accuracy: 0.5074 - val_loss: 1.0385 - val_accuracy: 0.5897
Epoch 119/1000
270/270 [==============================] - 0s 92us/step - loss: 0.8525 - accuracy: 0.5222 - val_loss:

270/270 [==============================] - 0s 122us/step - loss: 0.8485 - accuracy: 0.5481 - val_loss: 1.0371 - val_accuracy: 0.5641
Epoch 224/1000
270/270 [==============================] - 0s 165us/step - loss: 0.8423 - accuracy: 0.5481 - val_loss: 1.0448 - val_accuracy: 0.5726
Epoch 225/1000
270/270 [==============================] - 0s 107us/step - loss: 0.8440 - accuracy: 0.5519 - val_loss: 1.0432 - val_accuracy: 0.5641
Epoch 226/1000
270/270 [==============================] - 0s 96us/step - loss: 0.8475 - accuracy: 0.5111 - val_loss: 1.0418 - val_accuracy: 0.5812
Epoch 227/1000
270/270 [==============================] - 0s 125us/step - loss: 0.8453 - accuracy: 0.5259 - val_loss: 1.0402 - val_accuracy: 0.5556
Epoch 228/1000
270/270 [==============================] - 0s 164us/step - loss: 0.8421 - accuracy: 0.5519 - val_loss: 1.0444 - val_accuracy: 0.5641
Epoch 229/1000
270/270 [==============================] - 0s 128us/step - loss: 0.8458 - accuracy: 0.5519 - val_loss: 1.0467 - v

270/270 [==============================] - 0s 96us/step - loss: 0.8418 - accuracy: 0.5519 - val_loss: 1.0444 - val_accuracy: 0.5726
Epoch 334/1000
270/270 [==============================] - 0s 145us/step - loss: 0.8472 - accuracy: 0.5519 - val_loss: 1.0471 - val_accuracy: 0.5726
Epoch 335/1000
270/270 [==============================] - 0s 185us/step - loss: 0.8414 - accuracy: 0.5519 - val_loss: 1.0472 - val_accuracy: 0.5726
Epoch 336/1000
270/270 [==============================] - 0s 209us/step - loss: 0.8424 - accuracy: 0.5519 - val_loss: 1.0471 - val_accuracy: 0.5726
Epoch 337/1000
270/270 [==============================] - ETA: 0s - loss: 0.9238 - accuracy: 0.37 - 0s 109us/step - loss: 0.8437 - accuracy: 0.5519 - val_loss: 1.0515 - val_accuracy: 0.5726
Epoch 338/1000
270/270 [==============================] - 0s 103us/step - loss: 0.8406 - accuracy: 0.5519 - val_loss: 1.0452 - val_accuracy: 0.5726
Epoch 339/1000
270/270 [==============================] - 0s 96us/step - loss: 0.8486 

...

270/270 [==============================] - 0s 102us/step - loss: 0.8437 - accuracy: 0.5519 - val_loss: 1.0666 - val_accuracy: 0.5726
Epoch 554/1000
270/270 [==============================] - 0s 86us/step - loss: 0.8400 - accuracy: 0.5519 - val_loss: 1.0609 - val_accuracy: 0.5726
Epoch 555/1000
270/270 [==============================] - 0s 81us/step - loss: 0.8417 - accuracy: 0.5519 - val_loss: 1.0622 - val_accuracy: 0.5556
Epoch 556/1000
270/270 [==============================] - 0s 109us/step - loss: 0.8403 - accuracy: 0.5519 - val_loss: 1.0690 - val_accuracy: 0.5726
Epoch 557/1000
270/270 [==============================] - 0s 112us/step - loss: 0.8412 - accuracy: 0.5519 - val_loss: 1.0701 - val_accuracy: 0.5641
Epoch 558/1000
270/270 [==============================] - 0s 148us/step - loss: 0.8407 - accuracy: 0.5519 - val_loss: 1.0665 - val_accuracy: 0.5641
Epoch 559/1000
270/270 [==============================] - 0s 114us/step - loss: 0.8466 - accuracy: 0.5444 - val_loss: 1.0660 - va

270/270 [==============================] - 0s 147us/step - loss: 0.8420 - accuracy: 0.5556 - val_loss: 1.0843 - val_accuracy: 0.5726
Epoch 664/1000
270/270 [==============================] - 0s 88us/step - loss: 0.8448 - accuracy: 0.5481 - val_loss: 1.0787 - val_accuracy: 0.5556
Epoch 665/1000
270/270 [==============================] - 0s 110us/step - loss: 0.8403 - accuracy: 0.5481 - val_loss: 1.0829 - val_accuracy: 0.5726
Epoch 666/1000
270/270 [==============================] - 0s 94us/step - loss: 0.8414 - accuracy: 0.5519 - val_loss: 1.0848 - val_accuracy: 0.5726
Epoch 667/1000
270/270 [==============================] - 0s 90us/step - loss: 0.8436 - accuracy: 0.5519 - val_loss: 1.0830 - val_accuracy: 0.5726
Epoch 668/1000
270/270 [==============================] - 0s 98us/step - loss: 0.8386 - accuracy: 0.5556 - val_loss: 1.0759 - val_accuracy: 0.5641
Epoch 669/1000
270/270 [==============================] - 0s 74us/step - loss: 0.8406 - accuracy: 0.5519 - val_loss: 1.0774 - val_a

Epoch 719/1000
270/270 [==============================] - 0s 108us/step - loss: 0.8408 - accuracy: 0.5481 - val_loss: 1.0840 - val_accuracy: 0.5556
Epoch 720/1000
270/270 [==============================] - 0s 82us/step - loss: 0.8407 - accuracy: 0.5519 - val_loss: 1.0833 - val_accuracy: 0.5556
Epoch 721/1000
270/270 [==============================] - 0s 95us/step - loss: 0.8398 - accuracy: 0.5519 - val_loss: 1.0823 - val_accuracy: 0.5556
Epoch 722/1000
270/270 [==============================] - 0s 75us/step - loss: 0.8411 - accuracy: 0.5556 - val_loss: 1.0867 - val_accuracy: 0.5726
Epoch 723/1000
270/270 [==============================] - 0s 97us/step - loss: 0.8423 - accuracy: 0.5519 - val_loss: 1.0820 - val_accuracy: 0.5726
Epoch 724/1000
270/270 [==============================] - 0s 72us/step - loss: 0.8398 - accuracy: 0.5444 - val_loss: 1.0823 - val_accuracy: 0.5556
Epoch 725/1000
270/270 [==============================] - 0s 98us/step - loss: 0.8411 - accuracy: 0.5444 - val_loss: 

270/270 [==============================] - 0s 97us/step - loss: 0.8381 - accuracy: 0.5519 - val_loss: 1.0948 - val_accuracy: 0.5556
Epoch 830/1000
270/270 [==============================] - 0s 143us/step - loss: 0.8401 - accuracy: 0.5519 - val_loss: 1.0955 - val_accuracy: 0.5556
Epoch 831/1000
270/270 [==============================] - 0s 147us/step - loss: 0.8425 - accuracy: 0.5519 - val_loss: 1.0943 - val_accuracy: 0.5641
Epoch 832/1000
270/270 [==============================] - 0s 155us/step - loss: 0.8418 - accuracy: 0.5519 - val_loss: 1.0965 - val_accuracy: 0.5726
Epoch 833/1000
270/270 [==============================] - 0s 133us/step - loss: 0.8444 - accuracy: 0.5185 - val_loss: 1.0905 - val_accuracy: 0.5556
Epoch 834/1000
270/270 [==============================] - 0s 431us/step - loss: 0.8405 - accuracy: 0.5519 - val_loss: 1.0936 - val_accuracy: 0.5641
Epoch 835/1000
270/270 [==============================] - 0s 660us/step - loss: 0.8427 - accuracy: 0.5556 - val_loss: 1.1000 - v

270/270 [==============================] - 0s 109us/step - loss: 0.8410 - accuracy: 0.5556 - val_loss: 1.0978 - val_accuracy: 0.5641
Epoch 940/1000
270/270 [==============================] - 0s 154us/step - loss: 0.8397 - accuracy: 0.5519 - val_loss: 1.1061 - val_accuracy: 0.5641
Epoch 941/1000
270/270 [==============================] - 0s 106us/step - loss: 0.8403 - accuracy: 0.5519 - val_loss: 1.1017 - val_accuracy: 0.5641
Epoch 942/1000
270/270 [==============================] - 0s 101us/step - loss: 0.8388 - accuracy: 0.5519 - val_loss: 1.0995 - val_accuracy: 0.5641
Epoch 943/1000
270/270 [==============================] - 0s 108us/step - loss: 0.8390 - accuracy: 0.5519 - val_loss: 1.0997 - val_accuracy: 0.5641
Epoch 944/1000
270/270 [==============================] - 0s 118us/step - loss: 0.8406 - accuracy: 0.5519 - val_loss: 1.0974 - val_accuracy: 0.5641
Epoch 945/1000
270/270 [==============================] - 0s 626us/step - loss: 0.8392 - accuracy: 0.5519 - val_loss: 1.1015 - 

In [39]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over2.history['accuracy'])*100))

over-sampling train accuracy: 54.66%


In [29]:
df_proba2 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_2.xlsx",
                        sheet_name=1,
                        index_col=None)

In [30]:
df_proba2

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabs_qual,NRS148,2,2,0.000056,1.748042e-03,9.981960e-01
1,p002ykpresabs_qual,BCH-SA-03,1,0,0.712007,2.879924e-01,9.646217e-07
2,p002ykpresabs_qual,NRS218,1,1,0.006222,9.937732e-01,4.482882e-06
3,p002ykpresabs_qual,NRS036,0,0,0.882617,1.173831e-01,2.310933e-10
4,p002ykpresabs_qual,NRS386,1,0,0.571179,4.288184e-01,2.444667e-06
...,...,...,...,...,...,...,...
4279,pyopresabsSTCC_qual,NRS112,1,1,0.001860,9.979747e-01,1.653396e-04
4280,pyopresabsSTCC_qual,SR1065,0,0,0.982940,1.705227e-02,7.349168e-06
4281,pyopresabsSTCC_qual,NRS203,0,0,0.997093,1.962516e-03,9.441347e-04
4282,pyopresabsSTCC_qual,CFBREBSa129,0,0,1.000000,3.031141e-13,3.208205e-09


In [31]:
y_prob2 = df_proba2[df_proba2['phage']=='p0006kpresabs_qual'].iloc[:,-3:]
y_prob2 = y_prob2.to_numpy()
y_prob2

array([[3.88713870e-01, 3.80326800e-01, 2.30959280e-01],
       [1.91575570e-01, 4.65061280e-01, 3.43363200e-01],
       [3.88713870e-01, 3.80326800e-01, 2.30959280e-01],
       [3.88713870e-01, 3.80326800e-01, 2.30959280e-01],
       [3.88713870e-01, 3.80326800e-01, 2.30959280e-01],
       [2.60268400e-03, 5.43314400e-03, 9.91964200e-01],
       [3.88713870e-01, 3.80326800e-01, 2.30959280e-01],
       [1.91575570e-01, 4.65061280e-01, 3.43363200e-01],
       [1.91575570e-01, 4.65061280e-01, 3.43363200e-01],
       [3.88713870e-01, 3.80326800e-01, 2.30959280e-01],
       [8.44065370e-07, 8.08236250e-08, 9.99999050e-01],
       [3.88713870e-01, 3.80326800e-01, 2.30959280e-01],
       [1.91575570e-01, 4.65061280e-01, 3.43363200e-01],
       [5.33972440e-01, 2.80422400e-01, 1.85605100e-01],
       [3.88713870e-01, 3.80326800e-01, 2.30959280e-01],
       [7.32258330e-06, 1.04781775e-05, 9.99982240e-01],
       [1.44944440e-02, 1.55937800e-02, 9.69911800e-01],
       [3.88713870e-01, 3.80326

In [32]:
ovo2 = rocauc_ovo(y_test_over, y_prob2, average="macro", multi_class="ovo")
ovo2

0.7341661187815034

In [33]:
ovr2 = rocauc_ovr(y_test_over, y_prob2, average="macro", multi_class="ovr")
ovr2

0.7341661187815034

In [34]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=345,
                                                    stratify=y_over)

In [35]:
dat3 = pd.DataFrame(X_test_over[:,0])
dat3['test'] = y_test_over

In [36]:
dat3

,0,test
0,SR4187,0
1,NRS177,0
2,NRS109,2
3,CFBREBSa131,2
4,SR4152,1
...,...,...
112,NRS110,2
113,CFBRSa25,1
114,834N,0
115,CFBREBSa114,1


In [37]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [44]:
model1_over3 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [45]:
model1_over3.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [46]:
model1_over3.fit(X_train_over, y_train_over,
          batch_size=64, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 270 samples, validate on 117 samples
Epoch 1/1000
270/270 [==============================] - 0s 408us/step - loss: 1.2662 - accuracy: 0.3407 - val_loss: 1.2398 - val_accuracy: 0.3419
Epoch 2/1000
270/270 [==============================] - 0s 58us/step - loss: 1.1721 - accuracy: 0.3481 - val_loss: 1.1671 - val_accuracy: 0.3675
Epoch 3/1000
270/270 [==============================] - 0s 108us/step - loss: 1.1043 - accuracy: 0.3519 - val_loss: 1.1242 - val_accuracy: 0.4103
Epoch 4/1000
270/270 [==============================] - 0s 63us/step - loss: 1.0678 - accuracy: 0.3852 - val_loss: 1.1041 - val_accuracy: 0.4359
Epoch 5/1000
270/270 [==============================] - 0s 65us/step - loss: 1.0500 - accuracy: 0.4222 - val_loss: 1.0983 - val_accuracy: 0.3761
Epoch 6/1000
270/270 [==============================] - 0s 94us/step - loss: 1.0474 - accuracy: 0.4407 - val_loss: 1.0992 - val_accuracy: 0.3761
Epoch 7/1000
270/270 [==============================] - 0s 109us/step - loss: 1.04

Epoch 57/1000
270/270 [==============================] - 0s 89us/step - loss: 0.9469 - accuracy: 0.5185 - val_loss: 1.0221 - val_accuracy: 0.4359
Epoch 58/1000
270/270 [==============================] - 0s 69us/step - loss: 0.9467 - accuracy: 0.5296 - val_loss: 1.0242 - val_accuracy: 0.4274
Epoch 59/1000
270/270 [==============================] - 0s 67us/step - loss: 0.9475 - accuracy: 0.5296 - val_loss: 1.0246 - val_accuracy: 0.4274
Epoch 60/1000
270/270 [==============================] - 0s 60us/step - loss: 0.9441 - accuracy: 0.5296 - val_loss: 1.0198 - val_accuracy: 0.4359
Epoch 61/1000
270/270 [==============================] - 0s 79us/step - loss: 0.9475 - accuracy: 0.5148 - val_loss: 1.0205 - val_accuracy: 0.4957
Epoch 62/1000
270/270 [==============================] - 0s 105us/step - loss: 0.9471 - accuracy: 0.5407 - val_loss: 1.0201 - val_accuracy: 0.4359
Epoch 63/1000
270/270 [==============================] - 0s 266us/step - loss: 0.9473 - accuracy: 0.5333 - val_loss: 1.0203

Epoch 113/1000
270/270 [==============================] - 0s 75us/step - loss: 0.9245 - accuracy: 0.5296 - val_loss: 1.0135 - val_accuracy: 0.5043
Epoch 114/1000
270/270 [==============================] - 0s 67us/step - loss: 0.9228 - accuracy: 0.5296 - val_loss: 1.0119 - val_accuracy: 0.5043
Epoch 115/1000
270/270 [==============================] - 0s 50us/step - loss: 0.9212 - accuracy: 0.5370 - val_loss: 1.0117 - val_accuracy: 0.5043
Epoch 116/1000
270/270 [==============================] - 0s 45us/step - loss: 0.9210 - accuracy: 0.5370 - val_loss: 1.0101 - val_accuracy: 0.5043
Epoch 117/1000
270/270 [==============================] - 0s 45us/step - loss: 0.9199 - accuracy: 0.5407 - val_loss: 1.0097 - val_accuracy: 0.4188
Epoch 118/1000
270/270 [==============================] - 0s 66us/step - loss: 0.9204 - accuracy: 0.5407 - val_loss: 1.0099 - val_accuracy: 0.4188
Epoch 119/1000
270/270 [==============================] - 0s 55us/step - loss: 0.9192 - accuracy: 0.5407 - val_loss: 1

Epoch 224/1000
270/270 [==============================] - 0s 52us/step - loss: 0.8948 - accuracy: 0.5556 - val_loss: 1.0006 - val_accuracy: 0.4615
Epoch 225/1000
270/270 [==============================] - 0s 53us/step - loss: 0.8951 - accuracy: 0.5444 - val_loss: 1.0015 - val_accuracy: 0.5214
Epoch 226/1000
270/270 [==============================] - 0s 63us/step - loss: 0.8935 - accuracy: 0.5444 - val_loss: 1.0014 - val_accuracy: 0.5299
Epoch 227/1000
270/270 [==============================] - 0s 47us/step - loss: 0.8937 - accuracy: 0.5444 - val_loss: 1.0012 - val_accuracy: 0.5299
Epoch 228/1000
270/270 [==============================] - 0s 55us/step - loss: 0.8938 - accuracy: 0.5481 - val_loss: 1.0023 - val_accuracy: 0.5214
Epoch 229/1000
270/270 [==============================] - 0s 48us/step - loss: 0.8913 - accuracy: 0.5444 - val_loss: 1.0074 - val_accuracy: 0.4615
Epoch 230/1000
270/270 [==============================] - 0s 49us/step - loss: 0.8926 - accuracy: 0.5519 - val_loss: 1

Epoch 280/1000
270/270 [==============================] - 0s 57us/step - loss: 0.8840 - accuracy: 0.5593 - val_loss: 1.0066 - val_accuracy: 0.4615
Epoch 281/1000
270/270 [==============================] - 0s 100us/step - loss: 0.8889 - accuracy: 0.5444 - val_loss: 1.0078 - val_accuracy: 0.4615
Epoch 282/1000
270/270 [==============================] - 0s 49us/step - loss: 0.8863 - accuracy: 0.5593 - val_loss: 1.0087 - val_accuracy: 0.4615
Epoch 283/1000
270/270 [==============================] - 0s 46us/step - loss: 0.8851 - accuracy: 0.5593 - val_loss: 1.0084 - val_accuracy: 0.4615
Epoch 284/1000
270/270 [==============================] - 0s 68us/step - loss: 0.8815 - accuracy: 0.5593 - val_loss: 1.0092 - val_accuracy: 0.4615
Epoch 285/1000
270/270 [==============================] - 0s 93us/step - loss: 0.8832 - accuracy: 0.5333 - val_loss: 1.0041 - val_accuracy: 0.5299
Epoch 286/1000
270/270 [==============================] - 0s 51us/step - loss: 0.8814 - accuracy: 0.5111 - val_loss: 

...

Epoch 615/1000
270/270 [==============================] - 0s 59us/step - loss: 0.8436 - accuracy: 0.5593 - val_loss: 1.0480 - val_accuracy: 0.4615
Epoch 616/1000
270/270 [==============================] - 0s 58us/step - loss: 0.8437 - accuracy: 0.5593 - val_loss: 1.0458 - val_accuracy: 0.5299
Epoch 617/1000
270/270 [==============================] - 0s 56us/step - loss: 0.8428 - accuracy: 0.5519 - val_loss: 1.0468 - val_accuracy: 0.5299
Epoch 618/1000
270/270 [==============================] - 0s 59us/step - loss: 0.8438 - accuracy: 0.5519 - val_loss: 1.0541 - val_accuracy: 0.5214
Epoch 619/1000
270/270 [==============================] - 0s 57us/step - loss: 0.8453 - accuracy: 0.5519 - val_loss: 1.0529 - val_accuracy: 0.4615
Epoch 620/1000
270/270 [==============================] - 0s 53us/step - loss: 0.8447 - accuracy: 0.5593 - val_loss: 1.0526 - val_accuracy: 0.4615
Epoch 621/1000
270/270 [==============================] - ETA: 0s - loss: 0.9172 - accuracy: 0.48 - 0s 50us/step - los

Epoch 726/1000
270/270 [==============================] - 0s 84us/step - loss: 0.8384 - accuracy: 0.5556 - val_loss: 1.0647 - val_accuracy: 0.5299
Epoch 727/1000
270/270 [==============================] - 0s 86us/step - loss: 0.8417 - accuracy: 0.5556 - val_loss: 1.0630 - val_accuracy: 0.5299
Epoch 728/1000
270/270 [==============================] - 0s 75us/step - loss: 0.8372 - accuracy: 0.5667 - val_loss: 1.0691 - val_accuracy: 0.4615
Epoch 729/1000
270/270 [==============================] - 0s 77us/step - loss: 0.8385 - accuracy: 0.5593 - val_loss: 1.0787 - val_accuracy: 0.4615
Epoch 730/1000
270/270 [==============================] - 0s 83us/step - loss: 0.8382 - accuracy: 0.5593 - val_loss: 1.0733 - val_accuracy: 0.4615
Epoch 731/1000
270/270 [==============================] - 0s 76us/step - loss: 0.8370 - accuracy: 0.5704 - val_loss: 1.0740 - val_accuracy: 0.5214
Epoch 732/1000
270/270 [==============================] - 0s 63us/step - loss: 0.8426 - accuracy: 0.5556 - val_loss: 1

Epoch 782/1000
270/270 [==============================] - 0s 55us/step - loss: 0.8367 - accuracy: 0.5519 - val_loss: 1.0942 - val_accuracy: 0.4615
Epoch 783/1000
270/270 [==============================] - 0s 55us/step - loss: 0.8399 - accuracy: 0.5630 - val_loss: 1.0919 - val_accuracy: 0.4615
Epoch 784/1000
270/270 [==============================] - 0s 55us/step - loss: 0.8353 - accuracy: 0.5111 - val_loss: 1.0815 - val_accuracy: 0.4615
Epoch 785/1000
270/270 [==============================] - 0s 66us/step - loss: 0.8342 - accuracy: 0.5630 - val_loss: 1.0818 - val_accuracy: 0.4701
Epoch 786/1000
270/270 [==============================] - 0s 59us/step - loss: 0.8369 - accuracy: 0.5667 - val_loss: 1.0806 - val_accuracy: 0.4701
Epoch 787/1000
270/270 [==============================] - 0s 68us/step - loss: 0.8348 - accuracy: 0.5630 - val_loss: 1.0825 - val_accuracy: 0.4615
Epoch 788/1000
270/270 [==============================] - 0s 54us/step - loss: 0.8344 - accuracy: 0.5630 - val_loss: 1

270/270 [==============================] - 0s 62us/step - loss: 0.8349 - accuracy: 0.5556 - val_loss: 1.0993 - val_accuracy: 0.5214
Epoch 893/1000
270/270 [==============================] - 0s 62us/step - loss: 0.8327 - accuracy: 0.5556 - val_loss: 1.1010 - val_accuracy: 0.5214
Epoch 894/1000
270/270 [==============================] - 0s 71us/step - loss: 0.8309 - accuracy: 0.5704 - val_loss: 1.0967 - val_accuracy: 0.4615
Epoch 895/1000
270/270 [==============================] - 0s 70us/step - loss: 0.8315 - accuracy: 0.5593 - val_loss: 1.0960 - val_accuracy: 0.4615
Epoch 896/1000
270/270 [==============================] - 0s 69us/step - loss: 0.8361 - accuracy: 0.5630 - val_loss: 1.0984 - val_accuracy: 0.4615
Epoch 897/1000
270/270 [==============================] - 0s 67us/step - loss: 0.8316 - accuracy: 0.5630 - val_loss: 1.1039 - val_accuracy: 0.4615
Epoch 898/1000
270/270 [==============================] - 0s 59us/step - loss: 0.8320 - accuracy: 0.5630 - val_loss: 1.1133 - val_acc

Epoch 948/1000
270/270 [==============================] - 0s 57us/step - loss: 0.8302 - accuracy: 0.5630 - val_loss: 1.1002 - val_accuracy: 0.4615
Epoch 949/1000
270/270 [==============================] - 0s 61us/step - loss: 0.8292 - accuracy: 0.5630 - val_loss: 1.0997 - val_accuracy: 0.5214
Epoch 950/1000
270/270 [==============================] - 0s 72us/step - loss: 0.8318 - accuracy: 0.5556 - val_loss: 1.0984 - val_accuracy: 0.5128
Epoch 951/1000
270/270 [==============================] - 0s 62us/step - loss: 0.8308 - accuracy: 0.5556 - val_loss: 1.0987 - val_accuracy: 0.5214
Epoch 952/1000
270/270 [==============================] - 0s 55us/step - loss: 0.8339 - accuracy: 0.5444 - val_loss: 1.1041 - val_accuracy: 0.4615
Epoch 953/1000
270/270 [==============================] - 0s 72us/step - loss: 0.8311 - accuracy: 0.5519 - val_loss: 1.0961 - val_accuracy: 0.5214
Epoch 954/1000
270/270 [==============================] - 0s 64us/step - loss: 0.8304 - accuracy: 0.5519 - val_loss: 1

In [59]:
acc_test_over3 = model1_over3.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over3*100))

117/117 [==============================] - 0s 51us/step
over-sampling test accuracy: 52.14%


In [47]:
pred3 = model1_over3.predict_classes(X_test_over)
pred3

array([1, 0, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 0, 2, 1, 1, 1, 1, 1,
       2, 1, 1, 1, 1, 2, 1, 1, 0, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 2, 2,
       2, 2, 0, 1, 1, 1, 2, 2, 2, 2, 1, 1, 2, 1, 0, 1, 1, 1, 1, 1, 2, 1,
       1, 2, 0, 1, 1, 1, 1, 2, 2, 1, 1, 1, 2, 1, 1, 2, 1, 2, 1, 2, 2, 1,
       1, 1, 2, 0, 1, 1, 1, 1, 2, 1, 1, 1, 1, 0, 1, 2, 2, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1])

In [48]:
dat3['pred'] = pred3
dat3

,0,test,pred
0,SR4187,0,1
1,NRS177,0,0
2,NRS109,2,2
3,CFBREBSa131,2,1
4,SR4152,1,1
...,...,...,...
112,NRS110,2,1
113,CFBRSa25,1,1
114,834N,0,1
115,CFBREBSa114,1,1


In [49]:
proba3 = model1_over3.predict_proba(X_test_over)
dat_proba3 = pd.DataFrame(proba3)

In [50]:
dat_proba3

,0,1,2
0,0.363948,0.427751,0.208301
1,0.511673,0.310775,0.177551
2,0.045383,0.189247,0.765370
3,0.363948,0.427751,0.208301
4,0.363948,0.427751,0.208301
...,...,...,...
112,0.363948,0.427751,0.208301
113,0.363948,0.427751,0.208301
114,0.363948,0.427751,0.208301
115,0.363948,0.427751,0.208301


In [51]:
dat_proba3.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba3.csv", index = False,
         header=None)

In [52]:
dat3.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/3p006.csv", index = False,
         header=None)

In [63]:
hist1_over3 = model1_over3.fit(X_train_over, y_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 270 samples, validate on 117 samples
Epoch 1/1000
270/270 [==============================] - 0s 98us/step - loss: 0.8446 - accuracy: 0.5481 - val_loss: 1.0492 - val_accuracy: 0.4615
Epoch 2/1000
270/270 [==============================] - 0s 116us/step - loss: 0.8436 - accuracy: 0.5185 - val_loss: 1.0550 - val_accuracy: 0.5214
Epoch 3/1000
270/270 [==============================] - 0s 129us/step - loss: 0.8438 - accuracy: 0.5741 - val_loss: 1.0581 - val_accuracy: 0.4615
Epoch 4/1000
270/270 [==============================] - 0s 104us/step - loss: 0.8420 - accuracy: 0.5667 - val_loss: 1.0514 - val_accuracy: 0.5214
Epoch 5/1000
270/270 [==============================] - 0s 113us/step - loss: 0.8428 - accuracy: 0.5556 - val_loss: 1.0533 - val_accuracy: 0.5214
Epoch 6/1000
270/270 [==============================] - 0s 116us/step - loss: 0.8457 - accuracy: 0.5667 - val_loss: 1.0607 - val_accuracy: 0.4615
Epoch 7/1000
270/270 [==============================] - 0s 98us/step - loss: 0.

Epoch 57/1000
270/270 [==============================] - 0s 76us/step - loss: 0.8413 - accuracy: 0.5556 - val_loss: 1.0579 - val_accuracy: 0.5214
Epoch 58/1000
270/270 [==============================] - 0s 72us/step - loss: 0.8392 - accuracy: 0.5741 - val_loss: 1.0648 - val_accuracy: 0.4615
Epoch 59/1000
270/270 [==============================] - 0s 95us/step - loss: 0.8414 - accuracy: 0.5630 - val_loss: 1.0648 - val_accuracy: 0.4615
Epoch 60/1000
270/270 [==============================] - 0s 109us/step - loss: 0.8397 - accuracy: 0.5630 - val_loss: 1.0626 - val_accuracy: 0.4615
Epoch 61/1000
270/270 [==============================] - 0s 126us/step - loss: 0.8396 - accuracy: 0.5630 - val_loss: 1.0599 - val_accuracy: 0.4615
Epoch 62/1000
270/270 [==============================] - 0s 157us/step - loss: 0.8398 - accuracy: 0.5630 - val_loss: 1.0542 - val_accuracy: 0.5214
Epoch 63/1000
270/270 [==============================] - 0s 168us/step - loss: 0.8398 - accuracy: 0.5444 - val_loss: 1.05

Epoch 113/1000
270/270 [==============================] - 0s 93us/step - loss: 0.8422 - accuracy: 0.5259 - val_loss: 1.0646 - val_accuracy: 0.5214
Epoch 114/1000
270/270 [==============================] - 0s 94us/step - loss: 0.8379 - accuracy: 0.5704 - val_loss: 1.0563 - val_accuracy: 0.4615
Epoch 115/1000
270/270 [==============================] - 0s 87us/step - loss: 0.8404 - accuracy: 0.5630 - val_loss: 1.0578 - val_accuracy: 0.4615
Epoch 116/1000
270/270 [==============================] - 0s 93us/step - loss: 0.8369 - accuracy: 0.5593 - val_loss: 1.0551 - val_accuracy: 0.5214
Epoch 117/1000
270/270 [==============================] - 0s 111us/step - loss: 0.8387 - accuracy: 0.5556 - val_loss: 1.0571 - val_accuracy: 0.5214
Epoch 118/1000
270/270 [==============================] - 0s 84us/step - loss: 0.8372 - accuracy: 0.5667 - val_loss: 1.0609 - val_accuracy: 0.4615
Epoch 119/1000
270/270 [==============================] - 0s 86us/step - loss: 0.8364 - accuracy: 0.5630 - val_loss: 

270/270 [==============================] - 0s 166us/step - loss: 0.8335 - accuracy: 0.5296 - val_loss: 1.0759 - val_accuracy: 0.4615
Epoch 224/1000
270/270 [==============================] - 0s 74us/step - loss: 0.8339 - accuracy: 0.5630 - val_loss: 1.0767 - val_accuracy: 0.4615
Epoch 225/1000
270/270 [==============================] - 0s 79us/step - loss: 0.8364 - accuracy: 0.5370 - val_loss: 1.0758 - val_accuracy: 0.5214
Epoch 226/1000
270/270 [==============================] - 0s 76us/step - loss: 0.8394 - accuracy: 0.5259 - val_loss: 1.0711 - val_accuracy: 0.4615
Epoch 227/1000
270/270 [==============================] - 0s 150us/step - loss: 0.8338 - accuracy: 0.5630 - val_loss: 1.0707 - val_accuracy: 0.4615
Epoch 228/1000
270/270 [==============================] - 0s 105us/step - loss: 0.8361 - accuracy: 0.5333 - val_loss: 1.0712 - val_accuracy: 0.5214
Epoch 229/1000
270/270 [==============================] - 0s 105us/step - loss: 0.8373 - accuracy: 0.5593 - val_loss: 1.0778 - val

270/270 [==============================] - 0s 149us/step - loss: 0.8313 - accuracy: 0.5556 - val_loss: 1.0875 - val_accuracy: 0.5214
Epoch 334/1000
270/270 [==============================] - 0s 301us/step - loss: 0.8341 - accuracy: 0.5444 - val_loss: 1.0919 - val_accuracy: 0.4615
Epoch 335/1000
270/270 [==============================] - 0s 205us/step - loss: 0.8320 - accuracy: 0.5407 - val_loss: 1.0875 - val_accuracy: 0.5214
Epoch 336/1000
270/270 [==============================] - 0s 142us/step - loss: 0.8316 - accuracy: 0.5556 - val_loss: 1.0839 - val_accuracy: 0.5214
Epoch 337/1000
270/270 [==============================] - 0s 126us/step - loss: 0.8297 - accuracy: 0.5667 - val_loss: 1.0824 - val_accuracy: 0.4615
Epoch 338/1000
270/270 [==============================] - 0s 106us/step - loss: 0.8313 - accuracy: 0.5630 - val_loss: 1.0892 - val_accuracy: 0.4615
Epoch 339/1000
270/270 [==============================] - 0s 106us/step - loss: 0.8305 - accuracy: 0.5630 - val_loss: 1.0863 - 

...

270/270 [==============================] - 0s 122us/step - loss: 0.8275 - accuracy: 0.5630 - val_loss: 1.1123 - val_accuracy: 0.4615
Epoch 554/1000
270/270 [==============================] - 0s 128us/step - loss: 0.8248 - accuracy: 0.5630 - val_loss: 1.1151 - val_accuracy: 0.4615
Epoch 555/1000
270/270 [==============================] - 0s 147us/step - loss: 0.8265 - accuracy: 0.5259 - val_loss: 1.1119 - val_accuracy: 0.5128
Epoch 556/1000
270/270 [==============================] - 0s 124us/step - loss: 0.8262 - accuracy: 0.5407 - val_loss: 1.1143 - val_accuracy: 0.4615
Epoch 557/1000
270/270 [==============================] - 0s 123us/step - loss: 0.8249 - accuracy: 0.5630 - val_loss: 1.1192 - val_accuracy: 0.4615
Epoch 558/1000
270/270 [==============================] - 0s 126us/step - loss: 0.8240 - accuracy: 0.5556 - val_loss: 1.1250 - val_accuracy: 0.5214
Epoch 559/1000
270/270 [==============================] - 0s 133us/step - loss: 0.8286 - accuracy: 0.5519 - val_loss: 1.1174 - 

270/270 [==============================] - 0s 132us/step - loss: 0.8328 - accuracy: 0.5296 - val_loss: 1.1316 - val_accuracy: 0.5128
Epoch 664/1000
270/270 [==============================] - 0s 143us/step - loss: 0.8257 - accuracy: 0.5593 - val_loss: 1.1416 - val_accuracy: 0.4615
Epoch 665/1000
270/270 [==============================] - 0s 160us/step - loss: 0.8286 - accuracy: 0.5704 - val_loss: 1.1317 - val_accuracy: 0.5128
Epoch 666/1000
270/270 [==============================] - 0s 135us/step - loss: 0.8245 - accuracy: 0.5296 - val_loss: 1.1249 - val_accuracy: 0.4530
Epoch 667/1000
270/270 [==============================] - 0s 118us/step - loss: 0.8214 - accuracy: 0.5704 - val_loss: 1.1296 - val_accuracy: 0.4615
Epoch 668/1000
270/270 [==============================] - 0s 144us/step - loss: 0.8285 - accuracy: 0.5259 - val_loss: 1.1301 - val_accuracy: 0.4615
Epoch 669/1000
270/270 [==============================] - 0s 114us/step - loss: 0.8267 - accuracy: 0.5667 - val_loss: 1.1346 - 

270/270 [==============================] - 0s 122us/step - loss: 0.8221 - accuracy: 0.5630 - val_loss: 1.1435 - val_accuracy: 0.4530
Epoch 774/1000
270/270 [==============================] - 0s 109us/step - loss: 0.8231 - accuracy: 0.5667 - val_loss: 1.1417 - val_accuracy: 0.4615
Epoch 775/1000
270/270 [==============================] - 0s 85us/step - loss: 0.8214 - accuracy: 0.5667 - val_loss: 1.1429 - val_accuracy: 0.5128
Epoch 776/1000
270/270 [==============================] - 0s 90us/step - loss: 0.8216 - accuracy: 0.5407 - val_loss: 1.1444 - val_accuracy: 0.4530
Epoch 777/1000
270/270 [==============================] - 0s 101us/step - loss: 0.8226 - accuracy: 0.5630 - val_loss: 1.1378 - val_accuracy: 0.4530
Epoch 778/1000
270/270 [==============================] - 0s 83us/step - loss: 0.8262 - accuracy: 0.5667 - val_loss: 1.1407 - val_accuracy: 0.4615
Epoch 779/1000
270/270 [==============================] - 0s 87us/step - loss: 0.8222 - accuracy: 0.5407 - val_loss: 1.1342 - val_

Epoch 829/1000
270/270 [==============================] - 0s 99us/step - loss: 0.8225 - accuracy: 0.5667 - val_loss: 1.1457 - val_accuracy: 0.4530
Epoch 830/1000
270/270 [==============================] - 0s 89us/step - loss: 0.8219 - accuracy: 0.5630 - val_loss: 1.1506 - val_accuracy: 0.4615
Epoch 831/1000
270/270 [==============================] - 0s 89us/step - loss: 0.8222 - accuracy: 0.5667 - val_loss: 1.1500 - val_accuracy: 0.4615
Epoch 832/1000
270/270 [==============================] - 0s 86us/step - loss: 0.8217 - accuracy: 0.5630 - val_loss: 1.1492 - val_accuracy: 0.4530
Epoch 833/1000
270/270 [==============================] - 0s 81us/step - loss: 0.8214 - accuracy: 0.5704 - val_loss: 1.1534 - val_accuracy: 0.4615
Epoch 834/1000
270/270 [==============================] - ETA: 0s - loss: 0.7981 - accuracy: 0.59 - 0s 78us/step - loss: 0.8226 - accuracy: 0.5630 - val_loss: 1.1479 - val_accuracy: 0.4530
Epoch 835/1000
270/270 [==============================] - 0s 89us/step - los

270/270 [==============================] - 0s 111us/step - loss: 0.8205 - accuracy: 0.5370 - val_loss: 1.1591 - val_accuracy: 0.5214
Epoch 940/1000
270/270 [==============================] - 0s 116us/step - loss: 0.8210 - accuracy: 0.5593 - val_loss: 1.1588 - val_accuracy: 0.4615
Epoch 941/1000
270/270 [==============================] - 0s 92us/step - loss: 0.8218 - accuracy: 0.5667 - val_loss: 1.1608 - val_accuracy: 0.4615
Epoch 942/1000
270/270 [==============================] - 0s 94us/step - loss: 0.8209 - accuracy: 0.5481 - val_loss: 1.1606 - val_accuracy: 0.4530
Epoch 943/1000
270/270 [==============================] - 0s 77us/step - loss: 0.8204 - accuracy: 0.5667 - val_loss: 1.1630 - val_accuracy: 0.4530
Epoch 944/1000
270/270 [==============================] - 0s 101us/step - loss: 0.8190 - accuracy: 0.5667 - val_loss: 1.1628 - val_accuracy: 0.5128
Epoch 945/1000
270/270 [==============================] - 0s 94us/step - loss: 0.8317 - accuracy: 0.5593 - val_loss: 1.1604 - val_

In [64]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over3.history['accuracy'])*100))

over-sampling train accuracy: 55.63%


In [38]:
df_proba3 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_2.xlsx",
                        sheet_name=2,
                        index_col=None)

In [39]:
df_proba3

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabs_qual,NRS109,2,2,0.004477,0.013518,9.820048e-01
1,p002ykpresabs_qual,NRS109,2,2,0.004477,0.013518,9.820048e-01
2,p002ykpresabs_qual,NRS222,0,0,0.851725,0.148269,5.980786e-06
3,p002ykpresabs_qual,NRS109,2,2,0.004477,0.013518,9.820048e-01
4,p002ykpresabs_qual,GA50245,0,0,0.812055,0.187945,1.161034e-07
...,...,...,...,...,...,...,...
4279,pyopresabsSTCC_qual,NRS255,2,2,0.000633,0.000928,9.984396e-01
4280,pyopresabsSTCC_qual,NRS255,2,2,0.000633,0.000928,9.984396e-01
4281,pyopresabsSTCC_qual,NRS266,1,1,0.025932,0.974061,7.323514e-06
4282,pyopresabsSTCC_qual,NRS001,1,1,0.000597,0.999403,3.675362e-10


In [40]:
y_prob3 = df_proba3[df_proba3['phage']=='p0006kpresabs_qual'].iloc[:,-3:]
y_prob3 = y_prob3.to_numpy()
y_prob3

array([[3.6394837e-01, 4.2775100e-01, 2.0830064e-01],
       [5.1167310e-01, 3.1077550e-01, 1.7755142e-01],
       [4.5383340e-02, 1.8924704e-01, 7.6536953e-01],
       [3.6394837e-01, 4.2775100e-01, 2.0830064e-01],
       [3.6394837e-01, 4.2775100e-01, 2.0830064e-01],
       [2.0531367e-01, 4.8763870e-01, 3.0704760e-01],
       [3.6394837e-01, 4.2775100e-01, 2.0830064e-01],
       [3.6394837e-01, 4.2775100e-01, 2.0830064e-01],
       [3.6394837e-01, 4.2775100e-01, 2.0830064e-01],
       [3.6394837e-01, 4.2775100e-01, 2.0830064e-01],
       [3.6394837e-01, 4.2775100e-01, 2.0830064e-01],
       [2.0531367e-01, 4.8763870e-01, 3.0704760e-01],
       [7.8565580e-02, 5.2625774e-03, 9.1617185e-01],
       [3.6394837e-01, 4.2775100e-01, 2.0830064e-01],
       [2.0531367e-01, 4.8763870e-01, 3.0704760e-01],
       [9.7123253e-01, 2.2602128e-02, 6.1653630e-03],
       [4.4029397e-03, 1.4818345e-04, 9.9544890e-01],
       [3.6394837e-01, 4.2775100e-01, 2.0830064e-01],
       [3.6394837e-01, 4.277

In [41]:
ovo3 = rocauc_ovo(y_test_over, y_prob3, average="macro", multi_class="ovo")
ovo3

0.6669953977646285

In [42]:
ovr3 = rocauc_ovr(y_test_over, y_prob3, average="macro", multi_class="ovr")
ovr3

0.6669953977646285

In [43]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=456,
                                                    stratify=y_over)

In [44]:
dat4 = pd.DataFrame(X_test_over[:,0])
dat4['test'] = y_test_over

In [45]:
dat4

,0,test
0,CFBRSa25,1
1,CFBRSa07,0
2,NRS247,0
3,NY439,2
4,CFBREBSa110,1
...,...,...
112,SR1129,0
113,NRS172,0
114,NRS205,2
115,NY439,2


In [46]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [57]:
model1_over4 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    
    Dense(3, activation='softmax'),
])

In [58]:
model1_over4.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [59]:
model1_over4.fit(X_train_over, y_train_over,
          batch_size=64, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 270 samples, validate on 117 samples
Epoch 1/1000
270/270 [==============================] - 0s 660us/step - loss: 1.3464 - accuracy: 0.3296 - val_loss: 1.2446 - val_accuracy: 0.3333
Epoch 2/1000
270/270 [==============================] - 0s 168us/step - loss: 1.2268 - accuracy: 0.3852 - val_loss: 1.1529 - val_accuracy: 0.3761
Epoch 3/1000
270/270 [==============================] - 0s 124us/step - loss: 1.1447 - accuracy: 0.4333 - val_loss: 1.1073 - val_accuracy: 0.3590
Epoch 4/1000
270/270 [==============================] - 0s 77us/step - loss: 1.1070 - accuracy: 0.3444 - val_loss: 1.0897 - val_accuracy: 0.3504
Epoch 5/1000
270/270 [==============================] - 0s 62us/step - loss: 1.0908 - accuracy: 0.3185 - val_loss: 1.0887 - val_accuracy: 0.3504
Epoch 6/1000
270/270 [==============================] - 0s 67us/step - loss: 1.0852 - accuracy: 0.4185 - val_loss: 1.0895 - val_accuracy: 0.3675
Epoch 7/1000
270/270 [==============================] - 0s 87us/step - loss: 1.08

Epoch 57/1000
270/270 [==============================] - 0s 51us/step - loss: 0.9483 - accuracy: 0.5259 - val_loss: 1.0335 - val_accuracy: 0.4530
Epoch 58/1000
270/270 [==============================] - 0s 49us/step - loss: 0.9492 - accuracy: 0.5407 - val_loss: 1.0362 - val_accuracy: 0.4530
Epoch 59/1000
270/270 [==============================] - 0s 100us/step - loss: 0.9478 - accuracy: 0.5407 - val_loss: 1.0353 - val_accuracy: 0.4530
Epoch 60/1000
270/270 [==============================] - 0s 87us/step - loss: 0.9452 - accuracy: 0.5407 - val_loss: 1.0338 - val_accuracy: 0.4615
Epoch 61/1000
270/270 [==============================] - 0s 49us/step - loss: 0.9445 - accuracy: 0.5259 - val_loss: 1.0352 - val_accuracy: 0.4701
Epoch 62/1000
270/270 [==============================] - 0s 57us/step - loss: 0.9475 - accuracy: 0.5259 - val_loss: 1.0360 - val_accuracy: 0.4701
Epoch 63/1000
270/270 [==============================] - 0s 67us/step - loss: 0.9451 - accuracy: 0.5259 - val_loss: 1.0342 

Epoch 113/1000
270/270 [==============================] - 0s 66us/step - loss: 0.9246 - accuracy: 0.5556 - val_loss: 1.0435 - val_accuracy: 0.4530
Epoch 114/1000
270/270 [==============================] - 0s 126us/step - loss: 0.9215 - accuracy: 0.5556 - val_loss: 1.0446 - val_accuracy: 0.4444
Epoch 115/1000
270/270 [==============================] - 0s 117us/step - loss: 0.9206 - accuracy: 0.5556 - val_loss: 1.0446 - val_accuracy: 0.4444
Epoch 116/1000
270/270 [==============================] - 0s 68us/step - loss: 0.9195 - accuracy: 0.5519 - val_loss: 1.0454 - val_accuracy: 0.4444
Epoch 117/1000
270/270 [==============================] - 0s 59us/step - loss: 0.9204 - accuracy: 0.5519 - val_loss: 1.0460 - val_accuracy: 0.4530
Epoch 118/1000
270/270 [==============================] - 0s 72us/step - loss: 0.9213 - accuracy: 0.5519 - val_loss: 1.0466 - val_accuracy: 0.4530
Epoch 119/1000
270/270 [==============================] - 0s 267us/step - loss: 0.9208 - accuracy: 0.5519 - val_loss

Epoch 224/1000
270/270 [==============================] - 0s 41us/step - loss: 0.8911 - accuracy: 0.5667 - val_loss: 1.0477 - val_accuracy: 0.4530
Epoch 225/1000
270/270 [==============================] - 0s 47us/step - loss: 0.8894 - accuracy: 0.5704 - val_loss: 1.0480 - val_accuracy: 0.4530
Epoch 226/1000
270/270 [==============================] - 0s 43us/step - loss: 0.8892 - accuracy: 0.5667 - val_loss: 1.0480 - val_accuracy: 0.4530
Epoch 227/1000
270/270 [==============================] - 0s 47us/step - loss: 0.8891 - accuracy: 0.5667 - val_loss: 1.0487 - val_accuracy: 0.4530
Epoch 228/1000
270/270 [==============================] - 0s 52us/step - loss: 0.8888 - accuracy: 0.5667 - val_loss: 1.0496 - val_accuracy: 0.4530
Epoch 229/1000
270/270 [==============================] - 0s 50us/step - loss: 0.8887 - accuracy: 0.5667 - val_loss: 1.0524 - val_accuracy: 0.4530
Epoch 230/1000
270/270 [==============================] - 0s 51us/step - loss: 0.8912 - accuracy: 0.5667 - val_loss: 1

Epoch 280/1000
270/270 [==============================] - 0s 41us/step - loss: 0.8772 - accuracy: 0.5704 - val_loss: 1.0555 - val_accuracy: 0.4444
Epoch 281/1000
270/270 [==============================] - 0s 48us/step - loss: 0.8786 - accuracy: 0.5741 - val_loss: 1.0575 - val_accuracy: 0.4444
Epoch 282/1000
270/270 [==============================] - 0s 59us/step - loss: 0.8794 - accuracy: 0.5741 - val_loss: 1.0550 - val_accuracy: 0.4444
Epoch 283/1000
270/270 [==============================] - 0s 83us/step - loss: 0.8784 - accuracy: 0.5741 - val_loss: 1.0524 - val_accuracy: 0.4444
Epoch 284/1000
270/270 [==============================] - 0s 47us/step - loss: 0.8756 - accuracy: 0.5704 - val_loss: 1.0531 - val_accuracy: 0.4530
Epoch 285/1000
270/270 [==============================] - 0s 58us/step - loss: 0.8775 - accuracy: 0.5519 - val_loss: 1.0555 - val_accuracy: 0.4530
Epoch 286/1000
270/270 [==============================] - 0s 58us/step - loss: 0.8797 - accuracy: 0.5519 - val_loss: 1

...

Epoch 616/1000
270/270 [==============================] - 0s 50us/step - loss: 0.8343 - accuracy: 0.5852 - val_loss: 1.0777 - val_accuracy: 0.4530
Epoch 617/1000
270/270 [==============================] - 0s 58us/step - loss: 0.8333 - accuracy: 0.5852 - val_loss: 1.0776 - val_accuracy: 0.4530
Epoch 618/1000
270/270 [==============================] - 0s 71us/step - loss: 0.8330 - accuracy: 0.5852 - val_loss: 1.0767 - val_accuracy: 0.4530
Epoch 619/1000
270/270 [==============================] - 0s 76us/step - loss: 0.8319 - accuracy: 0.5852 - val_loss: 1.0775 - val_accuracy: 0.4530
Epoch 620/1000
270/270 [==============================] - 0s 62us/step - loss: 0.8303 - accuracy: 0.5852 - val_loss: 1.0803 - val_accuracy: 0.4530
Epoch 621/1000
270/270 [==============================] - 0s 60us/step - loss: 0.8324 - accuracy: 0.5741 - val_loss: 1.0845 - val_accuracy: 0.4530
Epoch 622/1000
270/270 [==============================] - 0s 57us/step - loss: 0.8346 - accuracy: 0.5667 - val_loss: 1

Epoch 672/1000
270/270 [==============================] - 0s 64us/step - loss: 0.8340 - accuracy: 0.5852 - val_loss: 1.0839 - val_accuracy: 0.4530
Epoch 673/1000
270/270 [==============================] - 0s 49us/step - loss: 0.8292 - accuracy: 0.5852 - val_loss: 1.0837 - val_accuracy: 0.4530
Epoch 674/1000
270/270 [==============================] - 0s 72us/step - loss: 0.8301 - accuracy: 0.5852 - val_loss: 1.0861 - val_accuracy: 0.4530
Epoch 675/1000
270/270 [==============================] - 0s 110us/step - loss: 0.8309 - accuracy: 0.5667 - val_loss: 1.0873 - val_accuracy: 0.4530
Epoch 676/1000
270/270 [==============================] - 0s 54us/step - loss: 0.8311 - accuracy: 0.5852 - val_loss: 1.0836 - val_accuracy: 0.4530
Epoch 677/1000
270/270 [==============================] - 0s 61us/step - loss: 0.8286 - accuracy: 0.5852 - val_loss: 1.0835 - val_accuracy: 0.4530
Epoch 678/1000
270/270 [==============================] - 0s 68us/step - loss: 0.8271 - accuracy: 0.5852 - val_loss: 

Epoch 728/1000
270/270 [==============================] - 0s 70us/step - loss: 0.8234 - accuracy: 0.5852 - val_loss: 1.0809 - val_accuracy: 0.4530
Epoch 729/1000
270/270 [==============================] - 0s 75us/step - loss: 0.8236 - accuracy: 0.5852 - val_loss: 1.0828 - val_accuracy: 0.4530
Epoch 730/1000
270/270 [==============================] - 0s 77us/step - loss: 0.8249 - accuracy: 0.5852 - val_loss: 1.0845 - val_accuracy: 0.4530
Epoch 731/1000
270/270 [==============================] - 0s 83us/step - loss: 0.8249 - accuracy: 0.5852 - val_loss: 1.0841 - val_accuracy: 0.4530
Epoch 732/1000
270/270 [==============================] - 0s 78us/step - loss: 0.8237 - accuracy: 0.5852 - val_loss: 1.0840 - val_accuracy: 0.4530
Epoch 733/1000
270/270 [==============================] - 0s 70us/step - loss: 0.8229 - accuracy: 0.5852 - val_loss: 1.0851 - val_accuracy: 0.4530
Epoch 734/1000
270/270 [==============================] - 0s 59us/step - loss: 0.8219 - accuracy: 0.5889 - val_loss: 1

Epoch 784/1000
270/270 [==============================] - 0s 74us/step - loss: 0.8199 - accuracy: 0.5889 - val_loss: 1.0914 - val_accuracy: 0.4444
Epoch 785/1000
270/270 [==============================] - 0s 64us/step - loss: 0.8206 - accuracy: 0.5926 - val_loss: 1.0931 - val_accuracy: 0.4444
Epoch 786/1000
270/270 [==============================] - 0s 124us/step - loss: 0.8199 - accuracy: 0.5926 - val_loss: 1.0933 - val_accuracy: 0.4444
Epoch 787/1000
270/270 [==============================] - 0s 165us/step - loss: 0.8205 - accuracy: 0.5704 - val_loss: 1.0912 - val_accuracy: 0.4359
Epoch 788/1000
270/270 [==============================] - 0s 89us/step - loss: 0.8204 - accuracy: 0.5889 - val_loss: 1.0893 - val_accuracy: 0.4615
Epoch 789/1000
270/270 [==============================] - 0s 78us/step - loss: 0.8213 - accuracy: 0.5630 - val_loss: 1.0906 - val_accuracy: 0.4444
Epoch 790/1000
270/270 [==============================] - 0s 69us/step - loss: 0.8200 - accuracy: 0.5741 - val_loss:

Epoch 895/1000
270/270 [==============================] - 0s 73us/step - loss: 0.8145 - accuracy: 0.5704 - val_loss: 1.1000 - val_accuracy: 0.4444
Epoch 896/1000
270/270 [==============================] - 0s 59us/step - loss: 0.8143 - accuracy: 0.5926 - val_loss: 1.0995 - val_accuracy: 0.4444
Epoch 897/1000
270/270 [==============================] - 0s 74us/step - loss: 0.8143 - accuracy: 0.5926 - val_loss: 1.1001 - val_accuracy: 0.4444
Epoch 898/1000
270/270 [==============================] - 0s 69us/step - loss: 0.8137 - accuracy: 0.5926 - val_loss: 1.0993 - val_accuracy: 0.4444
Epoch 899/1000
270/270 [==============================] - 0s 82us/step - loss: 0.8150 - accuracy: 0.5926 - val_loss: 1.0971 - val_accuracy: 0.4444
Epoch 900/1000
270/270 [==============================] - 0s 89us/step - loss: 0.8132 - accuracy: 0.5926 - val_loss: 1.0969 - val_accuracy: 0.4444
Epoch 901/1000
270/270 [==============================] - 0s 97us/step - loss: 0.8154 - accuracy: 0.5741 - val_loss: 1

In [100]:
acc_test_over4 = model1_over4.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over4*100))

117/117 [==============================] - 0s 79us/step
over-sampling test accuracy: 45.30%


In [60]:
pred4 = model1_over4.predict_classes(X_test_over)
pred4

array([1, 1, 0, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 2, 1, 2, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 2, 1, 1, 1, 1, 1, 1, 0, 2, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 2, 1,
       1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 2, 1,
       2, 2, 2, 1, 1, 0, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 0, 1,
       0, 1, 1, 1, 2, 2, 1])

In [61]:
dat4['pred'] = pred4
dat4

,0,test,pred
0,CFBRSa25,1,1
1,CFBRSa07,0,1
2,NRS247,0,0
3,NY439,2,2
4,CFBREBSa110,1,1
...,...,...,...
112,SR1129,0,1
113,NRS172,0,1
114,NRS205,2,2
115,NY439,2,2


In [62]:
proba4 = model1_over4.predict_proba(X_test_over)
dat_proba4 = pd.DataFrame(proba4)

In [63]:
dat_proba4

,0,1,2
0,0.366434,4.095403e-01,0.224026
1,0.068183,8.598678e-01,0.071949
2,0.622689,2.851807e-01,0.092130
3,0.000686,7.877519e-04,0.998526
4,0.366434,4.095403e-01,0.224026
...,...,...,...
112,0.366434,4.095403e-01,0.224026
113,0.188536,5.454503e-01,0.266014
114,0.001019,9.435421e-07,0.998980
115,0.000686,7.877519e-04,0.998526


In [64]:
dat_proba4.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba4.csv", index = False,
         header=None)

In [65]:
dat4.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/4p006.csv", index = False,
         header=None)

In [104]:
hist1_over4 = model1_over4.fit(X_train_over, y_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 270 samples, validate on 117 samples
Epoch 1/1000
270/270 [==============================] - 0s 100us/step - loss: 0.8280 - accuracy: 0.5852 - val_loss: 1.1341 - val_accuracy: 0.4530
Epoch 2/1000
270/270 [==============================] - 0s 92us/step - loss: 0.8207 - accuracy: 0.5889 - val_loss: 1.1390 - val_accuracy: 0.4359
Epoch 3/1000
270/270 [==============================] - 0s 98us/step - loss: 0.8276 - accuracy: 0.5926 - val_loss: 1.1380 - val_accuracy: 0.4359
Epoch 4/1000
270/270 [==============================] - 0s 121us/step - loss: 0.8235 - accuracy: 0.5926 - val_loss: 1.1393 - val_accuracy: 0.4530
Epoch 5/1000
270/270 [==============================] - 0s 101us/step - loss: 0.8303 - accuracy: 0.5667 - val_loss: 1.1347 - val_accuracy: 0.4530
Epoch 6/1000
270/270 [==============================] - 0s 82us/step - loss: 0.8239 - accuracy: 0.5667 - val_loss: 1.1350 - val_accuracy: 0.4530
Epoch 7/1000
270/270 [==============================] - 0s 131us/step - loss: 0.8

Epoch 112/1000
270/270 [==============================] - 0s 126us/step - loss: 0.8188 - accuracy: 0.5926 - val_loss: 1.1453 - val_accuracy: 0.4359
Epoch 113/1000
270/270 [==============================] - 0s 107us/step - loss: 0.8188 - accuracy: 0.5852 - val_loss: 1.1494 - val_accuracy: 0.4359
Epoch 114/1000
270/270 [==============================] - 0s 98us/step - loss: 0.8169 - accuracy: 0.5852 - val_loss: 1.1421 - val_accuracy: 0.4359
Epoch 115/1000
270/270 [==============================] - 0s 94us/step - loss: 0.8174 - accuracy: 0.5926 - val_loss: 1.1450 - val_accuracy: 0.4530
Epoch 116/1000
270/270 [==============================] - 0s 107us/step - loss: 0.8182 - accuracy: 0.5889 - val_loss: 1.1499 - val_accuracy: 0.4530
Epoch 117/1000
270/270 [==============================] - 0s 102us/step - loss: 0.8175 - accuracy: 0.5926 - val_loss: 1.1518 - val_accuracy: 0.4359
Epoch 118/1000
270/270 [==============================] - 0s 92us/step - loss: 0.8173 - accuracy: 0.5926 - val_los

270/270 [==============================] - 0s 134us/step - loss: 0.8114 - accuracy: 0.5741 - val_loss: 1.1592 - val_accuracy: 0.4359
Epoch 223/1000
270/270 [==============================] - 0s 85us/step - loss: 0.8115 - accuracy: 0.5926 - val_loss: 1.1577 - val_accuracy: 0.4274
Epoch 224/1000
270/270 [==============================] - 0s 113us/step - loss: 0.8118 - accuracy: 0.5963 - val_loss: 1.1557 - val_accuracy: 0.4274
Epoch 225/1000
270/270 [==============================] - 0s 86us/step - loss: 0.8119 - accuracy: 0.5963 - val_loss: 1.1588 - val_accuracy: 0.4359
Epoch 226/1000
270/270 [==============================] - 0s 85us/step - loss: 0.8132 - accuracy: 0.5926 - val_loss: 1.1596 - val_accuracy: 0.4359
Epoch 227/1000
270/270 [==============================] - 0s 148us/step - loss: 0.8135 - accuracy: 0.5926 - val_loss: 1.1539 - val_accuracy: 0.4359
Epoch 228/1000
270/270 [==============================] - 0s 195us/step - loss: 0.8125 - accuracy: 0.5926 - val_loss: 1.1516 - val

Epoch 333/1000
270/270 [==============================] - 0s 82us/step - loss: 0.8111 - accuracy: 0.5926 - val_loss: 1.1626 - val_accuracy: 0.4359
Epoch 334/1000
270/270 [==============================] - 0s 82us/step - loss: 0.8105 - accuracy: 0.5630 - val_loss: 1.1619 - val_accuracy: 0.4359
Epoch 335/1000
270/270 [==============================] - 0s 89us/step - loss: 0.8105 - accuracy: 0.5926 - val_loss: 1.1621 - val_accuracy: 0.4274
Epoch 336/1000
270/270 [==============================] - 0s 94us/step - loss: 0.8104 - accuracy: 0.5852 - val_loss: 1.1663 - val_accuracy: 0.4359
Epoch 337/1000
270/270 [==============================] - 0s 182us/step - loss: 0.8095 - accuracy: 0.5593 - val_loss: 1.1646 - val_accuracy: 0.4359
Epoch 338/1000
270/270 [==============================] - 0s 164us/step - loss: 0.8082 - accuracy: 0.5926 - val_loss: 1.1661 - val_accuracy: 0.4274
Epoch 339/1000
270/270 [==============================] - 0s 116us/step - loss: 0.8110 - accuracy: 0.5926 - val_loss

270/270 [==============================] - 0s 128us/step - loss: 0.8075 - accuracy: 0.5630 - val_loss: 1.1713 - val_accuracy: 0.4274
Epoch 444/1000
270/270 [==============================] - 0s 108us/step - loss: 0.8068 - accuracy: 0.5926 - val_loss: 1.1754 - val_accuracy: 0.4274
Epoch 445/1000
270/270 [==============================] - 0s 155us/step - loss: 0.8100 - accuracy: 0.5593 - val_loss: 1.1808 - val_accuracy: 0.4359
Epoch 446/1000
270/270 [==============================] - 0s 143us/step - loss: 0.8065 - accuracy: 0.5593 - val_loss: 1.1773 - val_accuracy: 0.4359
Epoch 447/1000
270/270 [==============================] - 0s 137us/step - loss: 0.8110 - accuracy: 0.5926 - val_loss: 1.1747 - val_accuracy: 0.4274
Epoch 448/1000
270/270 [==============================] - 0s 145us/step - loss: 0.8051 - accuracy: 0.5926 - val_loss: 1.1747 - val_accuracy: 0.4359
Epoch 449/1000
270/270 [==============================] - 0s 115us/step - loss: 0.8113 - accuracy: 0.5667 - val_loss: 1.1797 - 

270/270 [==============================] - 0s 146us/step - loss: 0.8072 - accuracy: 0.5926 - val_loss: 1.1802 - val_accuracy: 0.4274
Epoch 554/1000
270/270 [==============================] - 0s 141us/step - loss: 0.8071 - accuracy: 0.5556 - val_loss: 1.1814 - val_accuracy: 0.4359
Epoch 555/1000
270/270 [==============================] - 0s 127us/step - loss: 0.8058 - accuracy: 0.5778 - val_loss: 1.1805 - val_accuracy: 0.4359
Epoch 556/1000
270/270 [==============================] - 0s 132us/step - loss: 0.8073 - accuracy: 0.5926 - val_loss: 1.1864 - val_accuracy: 0.4359
Epoch 557/1000
270/270 [==============================] - 0s 201us/step - loss: 0.8065 - accuracy: 0.5926 - val_loss: 1.1883 - val_accuracy: 0.4274
Epoch 558/1000
270/270 [==============================] - 0s 202us/step - loss: 0.8040 - accuracy: 0.5926 - val_loss: 1.1843 - val_accuracy: 0.4274
Epoch 559/1000
270/270 [==============================] - 0s 508us/step - loss: 0.8068 - accuracy: 0.5481 - val_loss: 1.1797 - 

270/270 [==============================] - 0s 102us/step - loss: 0.8025 - accuracy: 0.5926 - val_loss: 1.1986 - val_accuracy: 0.4274
Epoch 664/1000
270/270 [==============================] - 0s 121us/step - loss: 0.8023 - accuracy: 0.5926 - val_loss: 1.1997 - val_accuracy: 0.4359
Epoch 665/1000
270/270 [==============================] - 0s 101us/step - loss: 0.8025 - accuracy: 0.5926 - val_loss: 1.1992 - val_accuracy: 0.4274
Epoch 666/1000
270/270 [==============================] - 0s 101us/step - loss: 0.8066 - accuracy: 0.5926 - val_loss: 1.1922 - val_accuracy: 0.4274
Epoch 667/1000
270/270 [==============================] - 0s 128us/step - loss: 0.8010 - accuracy: 0.5852 - val_loss: 1.1943 - val_accuracy: 0.4274
Epoch 668/1000
270/270 [==============================] - 0s 96us/step - loss: 0.8037 - accuracy: 0.5741 - val_loss: 1.1950 - val_accuracy: 0.4274
Epoch 669/1000
270/270 [==============================] - 0s 117us/step - loss: 0.8037 - accuracy: 0.5926 - val_loss: 1.1889 - v

270/270 [==============================] - 0s 181us/step - loss: 0.8030 - accuracy: 0.5926 - val_loss: 1.2059 - val_accuracy: 0.4274
Epoch 774/1000
270/270 [==============================] - 0s 89us/step - loss: 0.8060 - accuracy: 0.5926 - val_loss: 1.1985 - val_accuracy: 0.4274
Epoch 775/1000
270/270 [==============================] - 0s 83us/step - loss: 0.8080 - accuracy: 0.5407 - val_loss: 1.2130 - val_accuracy: 0.4274
Epoch 776/1000
270/270 [==============================] - 0s 83us/step - loss: 0.8056 - accuracy: 0.5778 - val_loss: 1.2002 - val_accuracy: 0.4274
Epoch 777/1000
270/270 [==============================] - 0s 80us/step - loss: 0.8045 - accuracy: 0.5926 - val_loss: 1.1971 - val_accuracy: 0.4274
Epoch 778/1000
270/270 [==============================] - 0s 85us/step - loss: 0.8028 - accuracy: 0.5926 - val_loss: 1.1988 - val_accuracy: 0.4359
Epoch 779/1000
270/270 [==============================] - 0s 81us/step - loss: 0.8016 - accuracy: 0.5926 - val_loss: 1.2031 - val_ac

Epoch 829/1000
270/270 [==============================] - 0s 282us/step - loss: 0.8024 - accuracy: 0.5926 - val_loss: 1.2115 - val_accuracy: 0.4274
Epoch 830/1000
270/270 [==============================] - 0s 131us/step - loss: 0.8008 - accuracy: 0.5926 - val_loss: 1.2068 - val_accuracy: 0.4274
Epoch 831/1000
270/270 [==============================] - 0s 235us/step - loss: 0.8028 - accuracy: 0.5963 - val_loss: 1.2049 - val_accuracy: 0.4359
Epoch 832/1000
270/270 [==============================] - 0s 120us/step - loss: 0.8024 - accuracy: 0.5926 - val_loss: 1.2042 - val_accuracy: 0.4274
Epoch 833/1000
270/270 [==============================] - 0s 107us/step - loss: 0.8015 - accuracy: 0.5926 - val_loss: 1.2030 - val_accuracy: 0.4274
Epoch 834/1000
270/270 [==============================] - 0s 102us/step - loss: 0.8052 - accuracy: 0.5889 - val_loss: 1.2030 - val_accuracy: 0.4274
Epoch 835/1000
270/270 [==============================] - 0s 113us/step - loss: 0.7998 - accuracy: 0.5815 - val_

270/270 [==============================] - 0s 144us/step - loss: 0.8011 - accuracy: 0.5926 - val_loss: 1.2180 - val_accuracy: 0.4274
Epoch 940/1000
270/270 [==============================] - 0s 199us/step - loss: 0.8000 - accuracy: 0.5741 - val_loss: 1.2150 - val_accuracy: 0.4274
Epoch 941/1000
270/270 [==============================] - 0s 144us/step - loss: 0.8031 - accuracy: 0.5778 - val_loss: 1.2059 - val_accuracy: 0.4274
Epoch 942/1000
270/270 [==============================] - 0s 114us/step - loss: 0.8000 - accuracy: 0.5926 - val_loss: 1.2048 - val_accuracy: 0.4274
Epoch 943/1000
270/270 [==============================] - 0s 119us/step - loss: 0.8017 - accuracy: 0.5926 - val_loss: 1.2117 - val_accuracy: 0.4274
Epoch 944/1000
270/270 [==============================] - 0s 112us/step - loss: 0.7998 - accuracy: 0.5926 - val_loss: 1.2109 - val_accuracy: 0.4274
Epoch 945/1000
270/270 [==============================] - 0s 224us/step - loss: 0.8021 - accuracy: 0.5630 - val_loss: 1.2171 - 

In [105]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over4.history['accuracy'])*100))

over-sampling train accuracy: 58.42%


In [47]:
df_proba4 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_2.xlsx",
                        sheet_name=3,
                        index_col=None)

In [48]:
df_proba4

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabs_qual,NRS110,1,1,0.000003,0.999997,5.870196e-13
1,p002ykpresabs_qual,NRS216,1,1,0.039254,0.960745,9.078969e-07
2,p002ykpresabs_qual,NRS386,1,1,0.326752,0.673248,1.061032e-07
3,p002ykpresabs_qual,CFBRSa25,0,0,0.611084,0.388916,7.664974e-07
4,p002ykpresabs_qual,BCH-SA-03,1,0,0.611084,0.388916,7.664974e-07
...,...,...,...,...,...,...,...
4279,pyopresabsSTCC_qual,NRS236,1,1,0.000052,0.999768,1.803156e-04
4280,pyopresabsSTCC_qual,NRS029,0,1,0.322350,0.677496,1.533154e-04
4281,pyopresabsSTCC_qual,NRS148,2,2,0.000006,0.000026,9.999682e-01
4282,pyopresabsSTCC_qual,CFBRSa28,0,0,0.999288,0.000176,5.361527e-04


In [49]:
y_prob4 = df_proba4[df_proba4['phage']=='p0006kpresabs_qual'].iloc[:,-3:]
y_prob4 = y_prob4.to_numpy()
y_prob4

array([[3.6643374e-01, 4.0954033e-01, 2.2402598e-01],
       [6.8182690e-02, 8.5986780e-01, 7.1949480e-02],
       [6.2268910e-01, 2.8518075e-01, 9.2130050e-02],
       [6.8591950e-04, 7.8775187e-04, 9.9852633e-01],
       [3.6643374e-01, 4.0954033e-01, 2.2402598e-01],
       [3.6643374e-01, 4.0954033e-01, 2.2402598e-01],
       [3.6643374e-01, 4.0954033e-01, 2.2402598e-01],
       [3.6643374e-01, 4.0954033e-01, 2.2402598e-01],
       [3.6643374e-01, 4.0954033e-01, 2.2402598e-01],
       [3.6643374e-01, 4.0954033e-01, 2.2402598e-01],
       [7.9174160e-03, 2.4257533e-02, 9.6782506e-01],
       [1.8853603e-01, 5.4545030e-01, 2.6601360e-01],
       [3.6643374e-01, 4.0954033e-01, 2.2402598e-01],
       [3.6643374e-01, 4.0954033e-01, 2.2402598e-01],
       [3.6643374e-01, 4.0954033e-01, 2.2402598e-01],
       [2.9292464e-02, 1.8446264e-01, 7.8624487e-01],
       [2.5603467e-01, 6.2875860e-02, 6.8108946e-01],
       [3.6643374e-01, 4.0954033e-01, 2.2402598e-01],
       [3.6643374e-01, 4.095

In [50]:
ovo4 = rocauc_ovo(y_test_over, y_prob4, average="macro", multi_class="ovo")
ovo4

0.6375191759807145

In [51]:
ovr4 = rocauc_ovr(y_test_over, y_prob4, average="macro", multi_class="ovr")
ovr4

0.6375191759807145

In [52]:
ovos = [ovo1, ovo2, ovo3, ovo4]
np.mean(ovos)

0.6871301775147929

In [53]:
np.std(ovos)

0.037399122427101535

In [54]:
ovrs = [ovr1, ovr2, ovr3, ovr4]
np.mean(ovrs)

0.6871301775147929

In [55]:
np.std(ovrs)

0.037399122427101535

In [106]:
accs = [acc_test_over, acc_test_over2, acc_test_over3, acc_test_over4]

In [107]:
mean = np.mean(accs)
print('over-sampling test accuracy mean: %.2f%%' % (mean*100))

over-sampling test accuracy mean: 51.71%


In [108]:
std = np.std(accs)
print('over-sampling test accuracy standard deviation:', std)

over-sampling test accuracy standard deviation: 0.03893348678044613


In [109]:
accs_train = [np.mean(hist1_over.history['accuracy']), np.mean(hist1_over2.history['accuracy']), np.mean(hist1_over3.history['accuracy']),
             np.mean(hist1_over4.history['accuracy'])]

In [110]:
mean_train = np.mean(accs_train)
print('over-sampling train accuracy mean: %.2f%%' % (mean_train*100))

over-sampling train accuracy mean: 56.00%


In [111]:
std_train = np.std(accs_train)
print('over-sampling train accuracy standard deviation:', std_train)

over-sampling train accuracy standard deviation: 0.014425634


In [ ]:
############ Feature selection using lasso ##########

In [14]:
## Retrieved from https://towardsdatascience.com/feature-selection-using-regularisation-a3678b71e499
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
import numpy as np

In [15]:
selection = SelectFromModel(LogisticRegression(C=1, penalty='l1', solver='liblinear'))
selection.fit(X_over[:,1:], y_over)

SelectFromModel(estimator=LogisticRegression(C=1, class_weight=None, dual=False,
                                             fit_intercept=True,
                                             intercept_scaling=1, l1_ratio=None,
                                             max_iter=100, multi_class='auto',
                                             n_jobs=None, penalty='l1',
                                             random_state=None,
                                             solver='liblinear', tol=0.0001,
                                             verbose=0, warm_start=False),
                max_features=None, norm_order=1, prefit=False, threshold=None)

In [16]:
names = np.array(df_clean.columns).tolist()
names.remove('pheno')

In [17]:
X_train_features_over = np.vstack((names, X_over[:,1:]))
X_train_features_over = pd.DataFrame(X_train_features_over)

In [18]:
sel_features = X_train_features_over.columns[(selection.get_support())]
print('total features: {}'.format((X_train_features_over.shape[1])))
print('selected features: {}'.format(len(sel_features)))

total features: 18
selected features: 15


In [19]:
cols = sel_features.values
cols.reshape((1, -1))

array([[ 1,  2,  3,  4,  5,  6,  8,  9, 10, 11, 12, 13, 14, 15, 17]])

In [20]:
names_arr = np.array(names)
names_arr[cols]

array(['TTAATTTAATAGA', 'TTAACATAATAAT', 'TGCAATCTCTTTAT',
       'TATTATGTTAATG', 'TACATACCGAT', 'GTGTATCATAAT', 'GCAAACATGCG',
       'GAGTCCTGTT', 'GAGTCCTGTTT',
       'GACAAACATGTATTAGCGTTATGTCGCGAACATCATAACCAGCAACATGCGATTGGCGTTAAGTCGTTTGATGATAAATATCACTTGCATGACTCGTGG',
       'CTTTTTCACCTGT', 'CTTGTGAATTTAG',
       'CTTATAACAATTACTATATTTGGCATTATATGTAGTATTATTTTCACGAG',
       'CGCCATTATGTT', 'ACAATTACTATATTT'], dtype='<U99')

In [21]:
###### keep selected variables as a new dataframe
df_sel = df_clean.loc[:,names_arr[cols]].copy()
df_sel['pheno'] = df_clean['pheno']

In [22]:
df_sel['strain'] = X.iloc[:,0]

In [23]:
df_sel

,TTAATTTAATAGA,TTAACATAATAAT,TGCAATCTCTTTAT,TATTATGTTAATG,TACATACCGAT,GTGTATCATAAT,GCAAACATGCG,GAGTCCTGTT,GAGTCCTGTTT,GACAAACATGTATTAGCGTTATGTCGCGAACATCATAACCAGCAACATGCGATTGGCGTTAAGTCGTTTGATGATAAATATCACTTGCATGACTCGTGG,CTTTTTCACCTGT,CTTGTGAATTTAG,CTTATAACAATTACTATATTTGGCATTATATGTAGTATTATTTTCACGAG,CGCCATTATGTT,ACAATTACTATATTT,pheno,strain
0,1,1,1,1,1,1,1,1,1,0,1,1,1,0,1,1,107
1,1,1,1,1,1,1,1,1,1,0,1,1,1,0,1,0,109
2,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,115
3,1,1,1,1,1,1,1,1,1,0,1,1,1,0,1,1,120335
4,1,1,1,1,1,1,1,1,1,0,1,1,1,0,1,1,120337
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,1,1,1,1,1,1,1,1,1,0,1,1,1,0,1,1,SR4152
249,1,1,1,1,1,1,1,1,1,0,1,1,1,0,1,0,SR4153
250,1,1,1,1,1,1,1,1,1,0,1,1,1,0,1,0,SR4155
251,1,1,1,1,1,1,1,1,1,0,1,1,1,0,1,0,SR4156


In [24]:
X_sel = df_sel.loc[:, df_sel.columns != 'pheno']
y_sel = df_sel['pheno']
print(X_sel.shape, y_sel.shape, df_sel.shape)

(253, 16) (253,) (253, 17)


In [25]:
df_sel['pheno'].value_counts()

0    129
1     85
2     39
Name: pheno, dtype: int64

In [26]:
# over-sampling
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_sel_over, y_sel_over = overS.fit_resample(X_sel, y_sel)
print(sorted(Counter(y_sel_over).items()))

[(0, 129), (1, 129), (2, 129)]


/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [27]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_sel_train_over, X_sel_test_over, y_sel_train_over, y_sel_test_over = train_test_split(X_sel_over, y_sel_over,
                                                    test_size = 0.3,
                                                    random_state=567,
                                                    stratify=y_sel_over)

In [28]:
dat5 = pd.DataFrame(X_sel_test_over[:,-1])
dat5['test'] = y_sel_test_over

In [29]:
dat5

,0,test
0,NRS245,1
1,NY439,2
2,CA544,1
3,CA541,2
4,EUH15,1
...,...,...
112,NRS112,0
113,CFBRSa51,2
114,NRS383,1
115,NRS247,0


In [30]:
X_sel_train_over = X_sel_train_over[:,:-1]
X_sel_test_over = X_sel_test_over[:,:-1]

In [83]:
#### neural network on over-sampling data
model2_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [84]:
model2_over.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [85]:
model2_over.fit(X_sel_train_over, y_sel_train_over,
          batch_size=64, epochs=1000,
          validation_data=(X_sel_test_over, y_sel_test_over))

Train on 270 samples, validate on 117 samples
Epoch 1/1000
270/270 [==============================] - 0s 486us/step - loss: 1.1115 - accuracy: 0.3222 - val_loss: 1.0999 - val_accuracy: 0.3504
Epoch 2/1000
270/270 [==============================] - 0s 73us/step - loss: 1.1029 - accuracy: 0.3444 - val_loss: 1.0936 - val_accuracy: 0.3932
Epoch 3/1000
270/270 [==============================] - 0s 88us/step - loss: 1.0954 - accuracy: 0.3741 - val_loss: 1.0868 - val_accuracy: 0.3932
Epoch 4/1000
270/270 [==============================] - 0s 97us/step - loss: 1.0880 - accuracy: 0.4037 - val_loss: 1.0795 - val_accuracy: 0.4530
Epoch 5/1000
270/270 [==============================] - 0s 93us/step - loss: 1.0800 - accuracy: 0.4185 - val_loss: 1.0731 - val_accuracy: 0.4615
Epoch 6/1000
270/270 [==============================] - 0s 92us/step - loss: 1.0733 - accuracy: 0.4296 - val_loss: 1.0669 - val_accuracy: 0.4615
Epoch 7/1000
270/270 [==============================] - 0s 108us/step - loss: 1.066

Epoch 57/1000
270/270 [==============================] - 0s 79us/step - loss: 0.9610 - accuracy: 0.5222 - val_loss: 0.9826 - val_accuracy: 0.5214
Epoch 58/1000
270/270 [==============================] - 0s 76us/step - loss: 0.9606 - accuracy: 0.5222 - val_loss: 0.9828 - val_accuracy: 0.5214
Epoch 59/1000
270/270 [==============================] - 0s 107us/step - loss: 0.9599 - accuracy: 0.5222 - val_loss: 0.9840 - val_accuracy: 0.5214
Epoch 60/1000
270/270 [==============================] - 0s 85us/step - loss: 0.9596 - accuracy: 0.5222 - val_loss: 0.9841 - val_accuracy: 0.5043
Epoch 61/1000
270/270 [==============================] - 0s 59us/step - loss: 0.9597 - accuracy: 0.4741 - val_loss: 0.9824 - val_accuracy: 0.5214
Epoch 62/1000
270/270 [==============================] - 0s 58us/step - loss: 0.9585 - accuracy: 0.5222 - val_loss: 0.9820 - val_accuracy: 0.5214
Epoch 63/1000
270/270 [==============================] - 0s 56us/step - loss: 0.9585 - accuracy: 0.5185 - val_loss: 0.9828 

Epoch 113/1000
270/270 [==============================] - 0s 95us/step - loss: 0.9421 - accuracy: 0.5370 - val_loss: 0.9759 - val_accuracy: 0.5214
Epoch 114/1000
270/270 [==============================] - 0s 59us/step - loss: 0.9430 - accuracy: 0.5222 - val_loss: 0.9773 - val_accuracy: 0.5214
Epoch 115/1000
270/270 [==============================] - 0s 81us/step - loss: 0.9423 - accuracy: 0.5222 - val_loss: 0.9796 - val_accuracy: 0.5214
Epoch 116/1000
270/270 [==============================] - 0s 66us/step - loss: 0.9432 - accuracy: 0.5185 - val_loss: 0.9811 - val_accuracy: 0.5214
Epoch 117/1000
270/270 [==============================] - 0s 83us/step - loss: 0.9431 - accuracy: 0.5185 - val_loss: 0.9787 - val_accuracy: 0.5214
Epoch 118/1000
270/270 [==============================] - 0s 112us/step - loss: 0.9411 - accuracy: 0.5037 - val_loss: 0.9778 - val_accuracy: 0.5043
Epoch 119/1000
270/270 [==============================] - 0s 90us/step - loss: 0.9423 - accuracy: 0.5074 - val_loss: 

Epoch 169/1000
270/270 [==============================] - 0s 106us/step - loss: 0.9317 - accuracy: 0.5259 - val_loss: 0.9751 - val_accuracy: 0.5214
Epoch 170/1000
270/270 [==============================] - 0s 72us/step - loss: 0.9311 - accuracy: 0.5259 - val_loss: 0.9739 - val_accuracy: 0.5214
Epoch 171/1000
270/270 [==============================] - 0s 105us/step - loss: 0.9319 - accuracy: 0.5296 - val_loss: 0.9738 - val_accuracy: 0.5214
Epoch 172/1000
270/270 [==============================] - 0s 106us/step - loss: 0.9312 - accuracy: 0.5296 - val_loss: 0.9745 - val_accuracy: 0.5214
Epoch 173/1000
270/270 [==============================] - 0s 67us/step - loss: 0.9313 - accuracy: 0.5296 - val_loss: 0.9741 - val_accuracy: 0.5214
Epoch 174/1000
270/270 [==============================] - 0s 60us/step - loss: 0.9341 - accuracy: 0.5296 - val_loss: 0.9725 - val_accuracy: 0.5214
Epoch 175/1000
270/270 [==============================] - 0s 56us/step - loss: 0.9316 - accuracy: 0.5296 - val_loss

Epoch 225/1000
270/270 [==============================] - 0s 48us/step - loss: 0.9258 - accuracy: 0.5185 - val_loss: 0.9776 - val_accuracy: 0.5043
Epoch 226/1000
270/270 [==============================] - 0s 42us/step - loss: 0.9269 - accuracy: 0.5185 - val_loss: 0.9786 - val_accuracy: 0.5214
Epoch 227/1000
270/270 [==============================] - 0s 41us/step - loss: 0.9256 - accuracy: 0.5296 - val_loss: 0.9746 - val_accuracy: 0.5214
Epoch 228/1000
270/270 [==============================] - 0s 41us/step - loss: 0.9243 - accuracy: 0.5296 - val_loss: 0.9720 - val_accuracy: 0.5214
Epoch 229/1000
270/270 [==============================] - 0s 54us/step - loss: 0.9231 - accuracy: 0.5333 - val_loss: 0.9713 - val_accuracy: 0.5214
Epoch 230/1000
270/270 [==============================] - 0s 44us/step - loss: 0.9243 - accuracy: 0.5333 - val_loss: 0.9714 - val_accuracy: 0.5214
Epoch 231/1000
270/270 [==============================] - 0s 47us/step - loss: 0.9214 - accuracy: 0.5333 - val_loss: 0

...

Epoch 728/1000
270/270 [==============================] - 0s 53us/step - loss: 0.8700 - accuracy: 0.5519 - val_loss: 0.9589 - val_accuracy: 0.5299
Epoch 729/1000
270/270 [==============================] - 0s 61us/step - loss: 0.8691 - accuracy: 0.5519 - val_loss: 0.9585 - val_accuracy: 0.5299
Epoch 730/1000
270/270 [==============================] - 0s 80us/step - loss: 0.8702 - accuracy: 0.5519 - val_loss: 0.9568 - val_accuracy: 0.5299
Epoch 731/1000
270/270 [==============================] - 0s 56us/step - loss: 0.8698 - accuracy: 0.5370 - val_loss: 0.9569 - val_accuracy: 0.5128
Epoch 732/1000
270/270 [==============================] - 0s 58us/step - loss: 0.8697 - accuracy: 0.5407 - val_loss: 0.9576 - val_accuracy: 0.5128
Epoch 733/1000
270/270 [==============================] - 0s 66us/step - loss: 0.8702 - accuracy: 0.5407 - val_loss: 0.9593 - val_accuracy: 0.5128
Epoch 734/1000
270/270 [==============================] - 0s 57us/step - loss: 0.8693 - accuracy: 0.5407 - val_loss: 0

Epoch 784/1000
270/270 [==============================] - 0s 38us/step - loss: 0.8669 - accuracy: 0.5111 - val_loss: 0.9596 - val_accuracy: 0.5128
Epoch 785/1000
270/270 [==============================] - 0s 38us/step - loss: 0.8655 - accuracy: 0.5444 - val_loss: 0.9615 - val_accuracy: 0.5128
Epoch 786/1000
270/270 [==============================] - 0s 43us/step - loss: 0.8670 - accuracy: 0.5296 - val_loss: 0.9665 - val_accuracy: 0.5299
Epoch 787/1000
270/270 [==============================] - 0s 42us/step - loss: 0.8676 - accuracy: 0.5519 - val_loss: 0.9682 - val_accuracy: 0.5299
Epoch 788/1000
270/270 [==============================] - 0s 39us/step - loss: 0.8678 - accuracy: 0.5519 - val_loss: 0.9661 - val_accuracy: 0.5299
Epoch 789/1000
270/270 [==============================] - 0s 37us/step - loss: 0.8670 - accuracy: 0.5519 - val_loss: 0.9600 - val_accuracy: 0.5299
Epoch 790/1000
270/270 [==============================] - 0s 38us/step - loss: 0.8658 - accuracy: 0.5333 - val_loss: 0

Epoch 840/1000
270/270 [==============================] - 0s 79us/step - loss: 0.8634 - accuracy: 0.5556 - val_loss: 0.9577 - val_accuracy: 0.5299
Epoch 841/1000
270/270 [==============================] - 0s 76us/step - loss: 0.8645 - accuracy: 0.5556 - val_loss: 0.9574 - val_accuracy: 0.5299
Epoch 842/1000
270/270 [==============================] - 0s 83us/step - loss: 0.8640 - accuracy: 0.5556 - val_loss: 0.9593 - val_accuracy: 0.5299
Epoch 843/1000
270/270 [==============================] - 0s 80us/step - loss: 0.8638 - accuracy: 0.5556 - val_loss: 0.9605 - val_accuracy: 0.5299
Epoch 844/1000
270/270 [==============================] - 0s 67us/step - loss: 0.8633 - accuracy: 0.5556 - val_loss: 0.9604 - val_accuracy: 0.5128
Epoch 845/1000
270/270 [==============================] - 0s 73us/step - loss: 0.8636 - accuracy: 0.5444 - val_loss: 0.9598 - val_accuracy: 0.5128
Epoch 846/1000
270/270 [==============================] - 0s 93us/step - loss: 0.8635 - accuracy: 0.5444 - val_loss: 0

Epoch 896/1000
270/270 [==============================] - 0s 39us/step - loss: 0.8612 - accuracy: 0.5296 - val_loss: 0.9681 - val_accuracy: 0.5128
Epoch 897/1000
270/270 [==============================] - 0s 45us/step - loss: 0.8622 - accuracy: 0.5444 - val_loss: 0.9685 - val_accuracy: 0.5128
Epoch 898/1000
270/270 [==============================] - 0s 36us/step - loss: 0.8636 - accuracy: 0.5111 - val_loss: 0.9691 - val_accuracy: 0.5128
Epoch 899/1000
270/270 [==============================] - 0s 41us/step - loss: 0.8640 - accuracy: 0.5444 - val_loss: 0.9646 - val_accuracy: 0.5128
Epoch 900/1000
270/270 [==============================] - 0s 38us/step - loss: 0.8653 - accuracy: 0.5444 - val_loss: 0.9668 - val_accuracy: 0.5128
Epoch 901/1000
270/270 [==============================] - 0s 93us/step - loss: 0.8644 - accuracy: 0.5444 - val_loss: 0.9641 - val_accuracy: 0.5128
Epoch 902/1000
270/270 [==============================] - 0s 43us/step - loss: 0.8625 - accuracy: 0.5370 - val_loss: 0

Epoch 952/1000
270/270 [==============================] - 0s 64us/step - loss: 0.8604 - accuracy: 0.5556 - val_loss: 0.9655 - val_accuracy: 0.5299
Epoch 953/1000
270/270 [==============================] - 0s 74us/step - loss: 0.8601 - accuracy: 0.5556 - val_loss: 0.9650 - val_accuracy: 0.5299
Epoch 954/1000
270/270 [==============================] - 0s 67us/step - loss: 0.8591 - accuracy: 0.5556 - val_loss: 0.9612 - val_accuracy: 0.5299
Epoch 955/1000
270/270 [==============================] - 0s 62us/step - loss: 0.8624 - accuracy: 0.5556 - val_loss: 0.9579 - val_accuracy: 0.5214
Epoch 956/1000
270/270 [==============================] - 0s 54us/step - loss: 0.8625 - accuracy: 0.5556 - val_loss: 0.9579 - val_accuracy: 0.5214
Epoch 957/1000
270/270 [==============================] - 0s 60us/step - loss: 0.8611 - accuracy: 0.5556 - val_loss: 0.9610 - val_accuracy: 0.5214
Epoch 958/1000
270/270 [==============================] - 0s 55us/step - loss: 0.8591 - accuracy: 0.5556 - val_loss: 0

In [182]:
acc_test2_over = model2_over.evaluate(X_sel_test_over, y_sel_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test2_over*100))

117/117 [==============================] - 0s 66us/step
over-sampling test accuracy: 52.14%


In [86]:
pred5 = model2_over.predict_classes(X_sel_test_over)
pred5

array([2, 2, 0, 0, 0, 0, 0, 1, 0, 1, 0, 2, 2, 1, 2, 1, 2, 0, 0, 1, 2, 1,
       0, 2, 0, 0, 0, 0, 1, 0, 2, 0, 2, 0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 2,
       0, 0, 2, 0, 1, 0, 1, 2, 0, 0, 0, 2, 0, 0, 0, 1, 0, 2, 0, 0, 2, 0,
       1, 1, 0, 1, 2, 1, 0, 1, 2, 0, 1, 0, 0, 0, 2, 0, 0, 2, 0, 2, 1, 0,
       1, 0, 0, 2, 0, 0, 0, 2, 2, 1, 1, 0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       2, 0, 0, 0, 0, 0, 0])

In [87]:
dat5['pred'] = pred5
dat5

,0,test,pred
0,NRS245,1,2
1,NY439,2,2
2,CA544,1,0
3,CA541,2,0
4,EUH15,1,0
...,...,...,...
112,NRS112,0,0
113,CFBRSa51,2,0
114,NRS383,1,0
115,NRS247,0,0


In [88]:
proba5 = model2_over.predict_proba(X_sel_test_over)
dat_proba5 = pd.DataFrame(proba5)

In [89]:
dat_proba5

,0,1,2
0,0.013458,0.216479,0.770063
1,0.026742,0.000929,0.972329
2,0.414748,0.362633,0.222618
3,0.414748,0.362633,0.222618
4,0.414748,0.362633,0.222618
...,...,...,...
112,0.672051,0.276171,0.051778
113,0.414748,0.362633,0.222618
114,0.672051,0.276171,0.051778
115,0.672051,0.276171,0.051778


In [90]:
dat_proba5.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba5.csv", index = False,
         header=None)

In [91]:
dat5.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/5p006.csv", index = False,
         header=None)

In [191]:
hist2_over = model2_over.fit(X_sel_train_over, y_sel_train_over,
          batch_size=64, epochs=1000,
          validation_data=(X_sel_test_over, y_sel_test_over))

Train on 270 samples, validate on 117 samples
Epoch 1/1000
270/270 [==============================] - 0s 77us/step - loss: 0.8524 - accuracy: 0.5556 - val_loss: 1.0148 - val_accuracy: 0.5214
Epoch 2/1000
270/270 [==============================] - 0s 57us/step - loss: 0.8541 - accuracy: 0.5556 - val_loss: 1.0150 - val_accuracy: 0.5214
Epoch 3/1000
270/270 [==============================] - 0s 55us/step - loss: 0.8496 - accuracy: 0.5556 - val_loss: 1.0110 - val_accuracy: 0.5043
Epoch 4/1000
270/270 [==============================] - 0s 71us/step - loss: 0.8484 - accuracy: 0.5444 - val_loss: 1.0111 - val_accuracy: 0.5043
Epoch 5/1000
270/270 [==============================] - 0s 83us/step - loss: 0.8516 - accuracy: 0.5444 - val_loss: 1.0080 - val_accuracy: 0.5043
Epoch 6/1000
270/270 [==============================] - 0s 68us/step - loss: 0.8496 - accuracy: 0.5444 - val_loss: 1.0063 - val_accuracy: 0.5043
Epoch 7/1000
270/270 [==============================] - 0s 94us/step - loss: 0.8475 

Epoch 57/1000
270/270 [==============================] - 0s 50us/step - loss: 0.8492 - accuracy: 0.5556 - val_loss: 1.0052 - val_accuracy: 0.5214
Epoch 58/1000
270/270 [==============================] - 0s 68us/step - loss: 0.8480 - accuracy: 0.5556 - val_loss: 1.0083 - val_accuracy: 0.5214
Epoch 59/1000
270/270 [==============================] - 0s 67us/step - loss: 0.8498 - accuracy: 0.5556 - val_loss: 1.0094 - val_accuracy: 0.5214
Epoch 60/1000
270/270 [==============================] - 0s 56us/step - loss: 0.8492 - accuracy: 0.5556 - val_loss: 1.0115 - val_accuracy: 0.5214
Epoch 61/1000
270/270 [==============================] - 0s 58us/step - loss: 0.8491 - accuracy: 0.5556 - val_loss: 1.0126 - val_accuracy: 0.5214
Epoch 62/1000
270/270 [==============================] - 0s 63us/step - loss: 0.8484 - accuracy: 0.5556 - val_loss: 1.0138 - val_accuracy: 0.5214
Epoch 63/1000
270/270 [==============================] - 0s 63us/step - loss: 0.8485 - accuracy: 0.5556 - val_loss: 1.0132 -

Epoch 113/1000
270/270 [==============================] - 0s 66us/step - loss: 0.8485 - accuracy: 0.5519 - val_loss: 1.0139 - val_accuracy: 0.5214
Epoch 114/1000
270/270 [==============================] - 0s 83us/step - loss: 0.8496 - accuracy: 0.5593 - val_loss: 1.0147 - val_accuracy: 0.5299
Epoch 115/1000
270/270 [==============================] - 0s 103us/step - loss: 0.8493 - accuracy: 0.5556 - val_loss: 1.0122 - val_accuracy: 0.5299
Epoch 116/1000
270/270 [==============================] - 0s 80us/step - loss: 0.8484 - accuracy: 0.5593 - val_loss: 1.0080 - val_accuracy: 0.5128
Epoch 117/1000
270/270 [==============================] - 0s 109us/step - loss: 0.8482 - accuracy: 0.5481 - val_loss: 1.0081 - val_accuracy: 0.4957
Epoch 118/1000
270/270 [==============================] - 0s 93us/step - loss: 0.8479 - accuracy: 0.5741 - val_loss: 1.0076 - val_accuracy: 0.5128
Epoch 119/1000
270/270 [==============================] - 0s 82us/step - loss: 0.8484 - accuracy: 0.5556 - val_loss:

Epoch 224/1000
270/270 [==============================] - 0s 55us/step - loss: 0.8475 - accuracy: 0.5556 - val_loss: 1.0113 - val_accuracy: 0.5214
Epoch 225/1000
270/270 [==============================] - 0s 44us/step - loss: 0.8472 - accuracy: 0.5630 - val_loss: 1.0104 - val_accuracy: 0.5043
Epoch 226/1000
270/270 [==============================] - 0s 57us/step - loss: 0.8494 - accuracy: 0.5444 - val_loss: 1.0113 - val_accuracy: 0.5043
Epoch 227/1000
270/270 [==============================] - 0s 61us/step - loss: 0.8490 - accuracy: 0.5444 - val_loss: 1.0074 - val_accuracy: 0.5128
Epoch 228/1000
270/270 [==============================] - 0s 57us/step - loss: 0.8517 - accuracy: 0.5556 - val_loss: 1.0096 - val_accuracy: 0.5214
Epoch 229/1000
270/270 [==============================] - 0s 54us/step - loss: 0.8524 - accuracy: 0.5556 - val_loss: 1.0105 - val_accuracy: 0.5214
Epoch 230/1000
270/270 [==============================] - 0s 53us/step - loss: 0.8488 - accuracy: 0.5556 - val_loss: 1

Epoch 280/1000
270/270 [==============================] - 0s 58us/step - loss: 0.8485 - accuracy: 0.5444 - val_loss: 1.0215 - val_accuracy: 0.5043
Epoch 281/1000
270/270 [==============================] - 0s 59us/step - loss: 0.8504 - accuracy: 0.5296 - val_loss: 1.0214 - val_accuracy: 0.5214
Epoch 282/1000
270/270 [==============================] - 0s 73us/step - loss: 0.8478 - accuracy: 0.5556 - val_loss: 1.0182 - val_accuracy: 0.5214
Epoch 283/1000
270/270 [==============================] - 0s 385us/step - loss: 0.8475 - accuracy: 0.5556 - val_loss: 1.0183 - val_accuracy: 0.5214
Epoch 284/1000
270/270 [==============================] - 0s 89us/step - loss: 0.8479 - accuracy: 0.5185 - val_loss: 1.0192 - val_accuracy: 0.5043
Epoch 285/1000
270/270 [==============================] - 0s 73us/step - loss: 0.8479 - accuracy: 0.5444 - val_loss: 1.0197 - val_accuracy: 0.5043
Epoch 286/1000
270/270 [==============================] - 0s 62us/step - loss: 0.8476 - accuracy: 0.5519 - val_loss: 

...

Epoch 669/1000
270/270 [==============================] - 0s 58us/step - loss: 0.8476 - accuracy: 0.5444 - val_loss: 1.0285 - val_accuracy: 0.5043
Epoch 670/1000
270/270 [==============================] - 0s 49us/step - loss: 0.8454 - accuracy: 0.5444 - val_loss: 1.0252 - val_accuracy: 0.5214
Epoch 671/1000
270/270 [==============================] - 0s 55us/step - loss: 0.8461 - accuracy: 0.5556 - val_loss: 1.0286 - val_accuracy: 0.5214
Epoch 672/1000
270/270 [==============================] - 0s 59us/step - loss: 0.8486 - accuracy: 0.5556 - val_loss: 1.0320 - val_accuracy: 0.5214
Epoch 673/1000
270/270 [==============================] - 0s 65us/step - loss: 0.8470 - accuracy: 0.5556 - val_loss: 1.0324 - val_accuracy: 0.5214
Epoch 674/1000
270/270 [==============================] - 0s 66us/step - loss: 0.8462 - accuracy: 0.5519 - val_loss: 1.0314 - val_accuracy: 0.5043
Epoch 675/1000
270/270 [==============================] - 0s 67us/step - loss: 0.8476 - accuracy: 0.5444 - val_loss: 1

Epoch 725/1000
270/270 [==============================] - 0s 56us/step - loss: 0.8452 - accuracy: 0.5556 - val_loss: 1.0270 - val_accuracy: 0.5214
Epoch 726/1000
270/270 [==============================] - 0s 51us/step - loss: 0.8478 - accuracy: 0.5556 - val_loss: 1.0252 - val_accuracy: 0.5214
Epoch 727/1000
270/270 [==============================] - 0s 61us/step - loss: 0.8485 - accuracy: 0.5259 - val_loss: 1.0253 - val_accuracy: 0.5043
Epoch 728/1000
270/270 [==============================] - 0s 64us/step - loss: 0.8460 - accuracy: 0.5630 - val_loss: 1.0299 - val_accuracy: 0.5214
Epoch 729/1000
270/270 [==============================] - 0s 62us/step - loss: 0.8466 - accuracy: 0.5556 - val_loss: 1.0347 - val_accuracy: 0.5043
Epoch 730/1000
270/270 [==============================] - 0s 60us/step - loss: 0.8475 - accuracy: 0.5444 - val_loss: 1.0331 - val_accuracy: 0.5128
Epoch 731/1000
270/270 [==============================] - 0s 60us/step - loss: 0.8489 - accuracy: 0.5444 - val_loss: 1

Epoch 781/1000
270/270 [==============================] - 0s 69us/step - loss: 0.8459 - accuracy: 0.5444 - val_loss: 1.0284 - val_accuracy: 0.4957
Epoch 782/1000
270/270 [==============================] - 0s 53us/step - loss: 0.8490 - accuracy: 0.5444 - val_loss: 1.0337 - val_accuracy: 0.5043
Epoch 783/1000
270/270 [==============================] - 0s 70us/step - loss: 0.8488 - accuracy: 0.5444 - val_loss: 1.0290 - val_accuracy: 0.5043
Epoch 784/1000
270/270 [==============================] - 0s 73us/step - loss: 0.8451 - accuracy: 0.5407 - val_loss: 1.0299 - val_accuracy: 0.5214
Epoch 785/1000
270/270 [==============================] - 0s 60us/step - loss: 0.8465 - accuracy: 0.5556 - val_loss: 1.0357 - val_accuracy: 0.5214
Epoch 786/1000
270/270 [==============================] - 0s 57us/step - loss: 0.8494 - accuracy: 0.5556 - val_loss: 1.0352 - val_accuracy: 0.5214
Epoch 787/1000
270/270 [==============================] - 0s 70us/step - loss: 0.8483 - accuracy: 0.5556 - val_loss: 1

Epoch 892/1000
270/270 [==============================] - 0s 55us/step - loss: 0.8455 - accuracy: 0.5370 - val_loss: 1.0279 - val_accuracy: 0.5214
Epoch 893/1000
270/270 [==============================] - 0s 56us/step - loss: 0.8443 - accuracy: 0.5556 - val_loss: 1.0313 - val_accuracy: 0.5214
Epoch 894/1000
270/270 [==============================] - 0s 55us/step - loss: 0.8451 - accuracy: 0.5556 - val_loss: 1.0321 - val_accuracy: 0.5214
Epoch 895/1000
270/270 [==============================] - 0s 61us/step - loss: 0.8447 - accuracy: 0.5444 - val_loss: 1.0300 - val_accuracy: 0.5043
Epoch 896/1000
270/270 [==============================] - 0s 68us/step - loss: 0.8443 - accuracy: 0.5370 - val_loss: 1.0240 - val_accuracy: 0.5128
Epoch 897/1000
270/270 [==============================] - 0s 374us/step - loss: 0.8462 - accuracy: 0.5556 - val_loss: 1.0230 - val_accuracy: 0.5128
Epoch 898/1000
270/270 [==============================] - 0s 60us/step - loss: 0.8477 - accuracy: 0.5556 - val_loss: 

Epoch 948/1000
270/270 [==============================] - 0s 54us/step - loss: 0.8466 - accuracy: 0.5556 - val_loss: 1.0237 - val_accuracy: 0.5128
Epoch 949/1000
270/270 [==============================] - 0s 60us/step - loss: 0.8467 - accuracy: 0.5556 - val_loss: 1.0275 - val_accuracy: 0.5128
Epoch 950/1000
270/270 [==============================] - 0s 70us/step - loss: 0.8451 - accuracy: 0.5556 - val_loss: 1.0303 - val_accuracy: 0.5128
Epoch 951/1000
270/270 [==============================] - 0s 72us/step - loss: 0.8464 - accuracy: 0.5556 - val_loss: 1.0353 - val_accuracy: 0.5128
Epoch 952/1000
270/270 [==============================] - 0s 77us/step - loss: 0.8467 - accuracy: 0.5556 - val_loss: 1.0398 - val_accuracy: 0.5214
Epoch 953/1000
270/270 [==============================] - 0s 64us/step - loss: 0.8478 - accuracy: 0.5519 - val_loss: 1.0374 - val_accuracy: 0.5128
Epoch 954/1000
270/270 [==============================] - 0s 62us/step - loss: 0.8446 - accuracy: 0.5556 - val_loss: 1

In [192]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist2_over.history['accuracy'])*100))

over-sampling train accuracy: 54.99%


In [37]:
df_proba5 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_lasso_2.xlsx",
                        sheet_name=0,
                        index_col=None)

In [38]:
df_proba5

,phage,strain,phenotype,prediction,0,1,2
0,p0006kpresabs_qual,NRS245,1,2,1.345807e-02,2.164788e-01,7.700630e-01
1,p0006kpresabs_qual,NY439,2,2,2.674153e-02,9.294230e-04,9.723290e-01
2,p0006kpresabs_qual,CA544,1,0,4.147484e-01,3.626331e-01,2.226184e-01
3,p0006kpresabs_qual,CA541,2,0,4.147484e-01,3.626331e-01,2.226184e-01
4,p0006kpresabs_qual,EUH15,1,0,4.147484e-01,3.626331e-01,2.226184e-01
...,...,...,...,...,...,...,...
984,p0017Skpresabs_qual,CA541,1,1,3.723218e-01,6.276781e-01,1.945911e-08
985,p0017Skpresabs_qual,SR4152,1,0,7.372800e-01,2.627200e-01,4.197748e-08
986,p0017Skpresabs_qual,NRS110,2,2,4.194510e-08,7.508231e-09,1.000000e+00
987,p0017Skpresabs_qual,CFBRSa70,0,0,7.372800e-01,2.627200e-01,4.197748e-08


In [39]:
y_prob5 = df_proba5[df_proba5['phage']=='p0006kpresabs_qual'].iloc[:,-3:]
y_prob5 = y_prob5.to_numpy()
y_prob5

array([[1.3458068e-02, 2.1647884e-01, 7.7006304e-01],
       [2.6741534e-02, 9.2942297e-04, 9.7232900e-01],
       [4.1474843e-01, 3.6263310e-01, 2.2261843e-01],
       [4.1474843e-01, 3.6263310e-01, 2.2261843e-01],
       [4.1474843e-01, 3.6263310e-01, 2.2261843e-01],
       [4.1474843e-01, 3.6263310e-01, 2.2261843e-01],
       [7.8940870e-01, 1.7715381e-01, 3.3437528e-02],
       [2.3190995e-01, 4.0615067e-01, 3.6193937e-01],
       [4.1474843e-01, 3.6263310e-01, 2.2261843e-01],
       [2.3190995e-01, 4.0615067e-01, 3.6193937e-01],
       [4.1474843e-01, 3.6263310e-01, 2.2261843e-01],
       [2.6736180e-01, 1.2357602e-01, 6.0906225e-01],
       [7.7902250e-02, 1.6777480e-01, 7.5432295e-01],
       [2.3190995e-01, 4.0615067e-01, 3.6193937e-01],
       [4.0215075e-01, 9.9052295e-02, 4.9879685e-01],
       [2.3190995e-01, 4.0615067e-01, 3.6193937e-01],
       [3.9979957e-02, 5.7650580e-02, 9.0236950e-01],
       [4.1474843e-01, 3.6263310e-01, 2.2261843e-01],
       [4.1474843e-01, 3.626

In [40]:
ovo5 = rocauc_ovo(y_sel_test_over, y_prob5, average="macro", multi_class="ovo")
ovo5

0.6949375410913872

In [41]:
ovr5 = rocauc_ovr(y_sel_test_over, y_prob5, average="macro", multi_class="ovr")
ovr5

0.6949375410913872

In [42]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_sel_train_over, X_sel_test_over, y_sel_train_over, y_sel_test_over = train_test_split(X_sel_over, y_sel_over,
                                                    test_size = 0.3,
                                                    random_state=678,
                                                    stratify=y_sel_over)

In [43]:
dat6 = pd.DataFrame(X_sel_test_over[:,-1])
dat6['test'] = y_sel_test_over

In [44]:
dat6

,0,test
0,NRS249,2
1,NRS188,1
2,NRS232,2
3,NY439,2
4,GA27,2
...,...,...
112,SR3569,0
113,NRS204,0
114,NRS203,0
115,CFBRSa25,1


In [45]:
X_sel_train_over = X_sel_train_over[:,:-1]
X_sel_test_over = X_sel_test_over[:,:-1]

In [96]:
model2_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train_over.shape[1],)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
])

In [97]:
model2_over2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [98]:
model2_over2.fit(X_sel_train_over, y_sel_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test_over, y_sel_test_over))

Train on 270 samples, validate on 117 samples
Epoch 1/1000
270/270 [==============================] - 0s 682us/step - loss: 1.1482 - accuracy: 0.3370 - val_loss: 1.1163 - val_accuracy: 0.3590
Epoch 2/1000
270/270 [==============================] - 0s 115us/step - loss: 1.0679 - accuracy: 0.4667 - val_loss: 1.0888 - val_accuracy: 0.3761
Epoch 3/1000
270/270 [==============================] - 0s 160us/step - loss: 1.0398 - accuracy: 0.4778 - val_loss: 1.0831 - val_accuracy: 0.3932
Epoch 4/1000
270/270 [==============================] - 0s 145us/step - loss: 1.0272 - accuracy: 0.4593 - val_loss: 1.0818 - val_accuracy: 0.3419
Epoch 5/1000
270/270 [==============================] - 0s 140us/step - loss: 1.0165 - accuracy: 0.4222 - val_loss: 1.0775 - val_accuracy: 0.3761
Epoch 6/1000
270/270 [==============================] - 0s 135us/step - loss: 1.0033 - accuracy: 0.4889 - val_loss: 1.0740 - val_accuracy: 0.4274
Epoch 7/1000
270/270 [==============================] - 0s 135us/step - loss: 

Epoch 112/1000
270/270 [==============================] - 0s 91us/step - loss: 0.8857 - accuracy: 0.5519 - val_loss: 1.0630 - val_accuracy: 0.4444
Epoch 113/1000
270/270 [==============================] - 0s 85us/step - loss: 0.8879 - accuracy: 0.5185 - val_loss: 1.0635 - val_accuracy: 0.4444
Epoch 114/1000
270/270 [==============================] - 0s 82us/step - loss: 0.8868 - accuracy: 0.5296 - val_loss: 1.0641 - val_accuracy: 0.4786
Epoch 115/1000
270/270 [==============================] - 0s 84us/step - loss: 0.8843 - accuracy: 0.5519 - val_loss: 1.0660 - val_accuracy: 0.4786
Epoch 116/1000
270/270 [==============================] - 0s 87us/step - loss: 0.8846 - accuracy: 0.5519 - val_loss: 1.0633 - val_accuracy: 0.4701
Epoch 117/1000
270/270 [==============================] - 0s 85us/step - loss: 0.8862 - accuracy: 0.5481 - val_loss: 1.0639 - val_accuracy: 0.4701
Epoch 118/1000
270/270 [==============================] - 0s 95us/step - loss: 0.8862 - accuracy: 0.5148 - val_loss: 1

Epoch 223/1000
270/270 [==============================] - 0s 74us/step - loss: 0.8685 - accuracy: 0.5519 - val_loss: 1.0975 - val_accuracy: 0.4701
Epoch 224/1000
270/270 [==============================] - 0s 70us/step - loss: 0.8661 - accuracy: 0.5519 - val_loss: 1.1018 - val_accuracy: 0.4701
Epoch 225/1000
270/270 [==============================] - 0s 71us/step - loss: 0.8669 - accuracy: 0.5519 - val_loss: 1.1059 - val_accuracy: 0.4444
Epoch 226/1000
270/270 [==============================] - 0s 75us/step - loss: 0.8683 - accuracy: 0.5519 - val_loss: 1.1027 - val_accuracy: 0.4701
Epoch 227/1000
270/270 [==============================] - 0s 71us/step - loss: 0.8660 - accuracy: 0.5519 - val_loss: 1.1010 - val_accuracy: 0.4701
Epoch 228/1000
270/270 [==============================] - 0s 72us/step - loss: 0.8666 - accuracy: 0.5519 - val_loss: 1.0968 - val_accuracy: 0.4701
Epoch 229/1000
270/270 [==============================] - 0s 71us/step - loss: 0.8692 - accuracy: 0.5111 - val_loss: 1

Epoch 279/1000
270/270 [==============================] - 0s 75us/step - loss: 0.8619 - accuracy: 0.5519 - val_loss: 1.1093 - val_accuracy: 0.4701
Epoch 280/1000
270/270 [==============================] - 0s 72us/step - loss: 0.8613 - accuracy: 0.5370 - val_loss: 1.1107 - val_accuracy: 0.4701
Epoch 281/1000
270/270 [==============================] - 0s 73us/step - loss: 0.8603 - accuracy: 0.5519 - val_loss: 1.1141 - val_accuracy: 0.4701
Epoch 282/1000
270/270 [==============================] - 0s 70us/step - loss: 0.8614 - accuracy: 0.5519 - val_loss: 1.1210 - val_accuracy: 0.4701
Epoch 283/1000
270/270 [==============================] - 0s 73us/step - loss: 0.8628 - accuracy: 0.5519 - val_loss: 1.1203 - val_accuracy: 0.4701
Epoch 284/1000
270/270 [==============================] - 0s 74us/step - loss: 0.8648 - accuracy: 0.5111 - val_loss: 1.1200 - val_accuracy: 0.4359
Epoch 285/1000
270/270 [==============================] - 0s 71us/step - loss: 0.8601 - accuracy: 0.5444 - val_loss: 1

270/270 [==============================] - 0s 89us/step - loss: 0.8566 - accuracy: 0.5296 - val_loss: 1.1338 - val_accuracy: 0.4701
Epoch 390/1000
270/270 [==============================] - 0s 105us/step - loss: 0.8569 - accuracy: 0.5556 - val_loss: 1.1430 - val_accuracy: 0.4444
Epoch 391/1000
270/270 [==============================] - 0s 110us/step - loss: 0.8582 - accuracy: 0.5481 - val_loss: 1.1373 - val_accuracy: 0.4701
Epoch 392/1000
270/270 [==============================] - 0s 94us/step - loss: 0.8606 - accuracy: 0.5074 - val_loss: 1.1416 - val_accuracy: 0.4701
Epoch 393/1000
270/270 [==============================] - 0s 112us/step - loss: 0.8634 - accuracy: 0.5519 - val_loss: 1.1551 - val_accuracy: 0.4444
Epoch 394/1000
270/270 [==============================] - 0s 93us/step - loss: 0.8632 - accuracy: 0.5148 - val_loss: 1.1377 - val_accuracy: 0.4615
Epoch 395/1000
270/270 [==============================] - 0s 92us/step - loss: 0.8592 - accuracy: 0.5407 - val_loss: 1.1190 - val_

...

Epoch 557/1000
270/270 [==============================] - 0s 88us/step - loss: 0.8618 - accuracy: 0.5556 - val_loss: 1.1603 - val_accuracy: 0.4444
Epoch 558/1000
270/270 [==============================] - 0s 90us/step - loss: 0.8513 - accuracy: 0.5296 - val_loss: 1.1648 - val_accuracy: 0.4701
Epoch 559/1000
270/270 [==============================] - 0s 102us/step - loss: 0.8514 - accuracy: 0.5556 - val_loss: 1.1553 - val_accuracy: 0.4701
Epoch 560/1000
270/270 [==============================] - 0s 92us/step - loss: 0.8518 - accuracy: 0.5556 - val_loss: 1.1590 - val_accuracy: 0.4701
Epoch 561/1000
270/270 [==============================] - 0s 88us/step - loss: 0.8503 - accuracy: 0.5556 - val_loss: 1.1663 - val_accuracy: 0.4701
Epoch 562/1000
270/270 [==============================] - 0s 89us/step - loss: 0.8541 - accuracy: 0.5148 - val_loss: 1.1604 - val_accuracy: 0.4359
Epoch 563/1000
270/270 [==============================] - 0s 83us/step - loss: 0.8510 - accuracy: 0.5037 - val_loss: 

270/270 [==============================] - 0s 140us/step - loss: 0.8505 - accuracy: 0.5148 - val_loss: 1.1877 - val_accuracy: 0.4701
Epoch 668/1000
270/270 [==============================] - 0s 177us/step - loss: 0.8482 - accuracy: 0.5556 - val_loss: 1.1864 - val_accuracy: 0.4701
Epoch 669/1000
270/270 [==============================] - 0s 152us/step - loss: 0.8487 - accuracy: 0.5556 - val_loss: 1.1827 - val_accuracy: 0.4701
Epoch 670/1000
270/270 [==============================] - 0s 146us/step - loss: 0.8500 - accuracy: 0.5481 - val_loss: 1.1922 - val_accuracy: 0.4701
Epoch 671/1000
270/270 [==============================] - 0s 125us/step - loss: 0.8495 - accuracy: 0.5556 - val_loss: 1.1971 - val_accuracy: 0.4701
Epoch 672/1000
270/270 [==============================] - 0s 116us/step - loss: 0.8504 - accuracy: 0.5556 - val_loss: 1.1988 - val_accuracy: 0.4444
Epoch 673/1000
270/270 [==============================] - 0s 113us/step - loss: 0.8488 - accuracy: 0.5556 - val_loss: 1.1908 - 

270/270 [==============================] - 0s 140us/step - loss: 0.8552 - accuracy: 0.5296 - val_loss: 1.2148 - val_accuracy: 0.4444
Epoch 778/1000
270/270 [==============================] - 0s 108us/step - loss: 0.8495 - accuracy: 0.5519 - val_loss: 1.2156 - val_accuracy: 0.4701
Epoch 779/1000
270/270 [==============================] - 0s 85us/step - loss: 0.8483 - accuracy: 0.5556 - val_loss: 1.2227 - val_accuracy: 0.4701
Epoch 780/1000
270/270 [==============================] - 0s 75us/step - loss: 0.8478 - accuracy: 0.5593 - val_loss: 1.2321 - val_accuracy: 0.4444
Epoch 781/1000
270/270 [==============================] - 0s 102us/step - loss: 0.8486 - accuracy: 0.5444 - val_loss: 1.2394 - val_accuracy: 0.4701
Epoch 782/1000
270/270 [==============================] - 0s 99us/step - loss: 0.8477 - accuracy: 0.5519 - val_loss: 1.1954 - val_accuracy: 0.4701
Epoch 783/1000
270/270 [==============================] - 0s 90us/step - loss: 0.8488 - accuracy: 0.5556 - val_loss: 1.1867 - val_

Epoch 833/1000
270/270 [==============================] - 0s 99us/step - loss: 0.8447 - accuracy: 0.5593 - val_loss: 1.2352 - val_accuracy: 0.4701
Epoch 834/1000
270/270 [==============================] - 0s 99us/step - loss: 0.8498 - accuracy: 0.5556 - val_loss: 1.2381 - val_accuracy: 0.4786
Epoch 835/1000
270/270 [==============================] - 0s 117us/step - loss: 0.8484 - accuracy: 0.5370 - val_loss: 1.2425 - val_accuracy: 0.4701
Epoch 836/1000
270/270 [==============================] - 0s 126us/step - loss: 0.8523 - accuracy: 0.5444 - val_loss: 1.2316 - val_accuracy: 0.4359
Epoch 837/1000
270/270 [==============================] - 0s 146us/step - loss: 0.8500 - accuracy: 0.5556 - val_loss: 1.2236 - val_accuracy: 0.4701
Epoch 838/1000
270/270 [==============================] - 0s 89us/step - loss: 0.8549 - accuracy: 0.5148 - val_loss: 1.2376 - val_accuracy: 0.4701
Epoch 839/1000
270/270 [==============================] - 0s 100us/step - loss: 0.8645 - accuracy: 0.5259 - val_los

270/270 [==============================] - 0s 142us/step - loss: 0.8460 - accuracy: 0.5556 - val_loss: 1.2196 - val_accuracy: 0.4701
Epoch 944/1000
270/270 [==============================] - 0s 132us/step - loss: 0.8534 - accuracy: 0.5556 - val_loss: 1.2367 - val_accuracy: 0.4444
Epoch 945/1000
270/270 [==============================] - 0s 136us/step - loss: 0.8448 - accuracy: 0.5407 - val_loss: 1.2453 - val_accuracy: 0.4786
Epoch 946/1000
270/270 [==============================] - 0s 113us/step - loss: 0.8483 - accuracy: 0.5556 - val_loss: 1.2485 - val_accuracy: 0.4786
Epoch 947/1000
270/270 [==============================] - 0s 104us/step - loss: 0.8481 - accuracy: 0.5556 - val_loss: 1.2493 - val_accuracy: 0.4701
Epoch 948/1000
270/270 [==============================] - 0s 120us/step - loss: 0.8450 - accuracy: 0.5222 - val_loss: 1.2524 - val_accuracy: 0.4359
Epoch 949/1000
270/270 [==============================] - 0s 116us/step - loss: 0.8478 - accuracy: 0.5111 - val_loss: 1.2501 - 

In [215]:
acc_test2_over2 = model2_over2.evaluate(X_sel_test_over, y_sel_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test2_over2*100))

117/117 [==============================] - 0s 58us/step
over-sampling test accuracy: 47.01%


In [99]:
pred6 = model2_over2.predict_classes(X_sel_test_over)
pred6

array([1, 1, 2, 2, 1, 1, 1, 0, 1, 2, 2, 2, 1, 0, 1, 2, 1, 1, 2, 1, 1, 1,
       2, 1, 2, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 0, 0,
       1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 0, 1, 1, 1, 0, 1, 2, 1, 1,
       1, 1, 0, 1, 1, 2, 1, 1, 0, 1, 2, 1, 0, 1, 2, 2, 1, 1, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 1, 2,
       0, 0, 1, 0, 0, 1, 1])

In [100]:
dat6['pred'] = pred6
dat6

,0,test,pred
0,NRS249,2,1
1,NRS188,1,1
2,NRS232,2,2
3,NY439,2,2
4,GA27,2,1
...,...,...,...
112,SR3569,0,1
113,NRS204,0,0
114,NRS203,0,0
115,CFBRSa25,1,1


In [101]:
proba6 = model2_over2.predict_proba(X_sel_test_over)
dat_proba6 = pd.DataFrame(proba6)

In [102]:
dat_proba6

,0,1,2
0,0.188887,0.510804,0.300309
1,0.188887,0.510804,0.300309
2,0.422291,0.070299,0.507410
3,0.000356,0.000298,0.999347
4,0.394097,0.418422,0.187481
...,...,...,...
112,0.394097,0.418422,0.187481
113,0.731846,0.199811,0.068343
114,0.925632,0.044885,0.029483
115,0.394097,0.418422,0.187481


In [103]:
dat_proba6.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba6.csv", index = False,
         header=None)

In [104]:
dat6.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/6p006.csv", index = False,
         header=None)

In [219]:
hist2_over2 = model2_over2.fit(X_sel_train_over, y_sel_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test_over, y_sel_test_over))

Train on 270 samples, validate on 117 samples
Epoch 1/1000
270/270 [==============================] - 0s 116us/step - loss: 0.8446 - accuracy: 0.5556 - val_loss: 1.1620 - val_accuracy: 0.4444
Epoch 2/1000
270/270 [==============================] - 0s 122us/step - loss: 0.8488 - accuracy: 0.5556 - val_loss: 1.1627 - val_accuracy: 0.4444
Epoch 3/1000
270/270 [==============================] - 0s 139us/step - loss: 0.8491 - accuracy: 0.5630 - val_loss: 1.1629 - val_accuracy: 0.4701
Epoch 4/1000
270/270 [==============================] - 0s 131us/step - loss: 0.8466 - accuracy: 0.5296 - val_loss: 1.1595 - val_accuracy: 0.4444
Epoch 5/1000
270/270 [==============================] - 0s 115us/step - loss: 0.8482 - accuracy: 0.5556 - val_loss: 1.1615 - val_accuracy: 0.4444
Epoch 6/1000
270/270 [==============================] - 0s 121us/step - loss: 0.8454 - accuracy: 0.5481 - val_loss: 1.1654 - val_accuracy: 0.4701
Epoch 7/1000
270/270 [==============================] - 0s 128us/step - loss: 

Epoch 112/1000
270/270 [==============================] - 0s 170us/step - loss: 0.8446 - accuracy: 0.5556 - val_loss: 1.1777 - val_accuracy: 0.4701
Epoch 113/1000
270/270 [==============================] - 0s 137us/step - loss: 0.8467 - accuracy: 0.5556 - val_loss: 1.1760 - val_accuracy: 0.4701
Epoch 114/1000
270/270 [==============================] - 0s 94us/step - loss: 0.8520 - accuracy: 0.5037 - val_loss: 1.1698 - val_accuracy: 0.4615
Epoch 115/1000
270/270 [==============================] - 0s 101us/step - loss: 0.8491 - accuracy: 0.5481 - val_loss: 1.1751 - val_accuracy: 0.4701
Epoch 116/1000
270/270 [==============================] - 0s 122us/step - loss: 0.8458 - accuracy: 0.5556 - val_loss: 1.1694 - val_accuracy: 0.4701
Epoch 117/1000
270/270 [==============================] - 0s 116us/step - loss: 0.8488 - accuracy: 0.5333 - val_loss: 1.1732 - val_accuracy: 0.4359
Epoch 118/1000
270/270 [==============================] - 0s 111us/step - loss: 0.8463 - accuracy: 0.5444 - val_l

270/270 [==============================] - 0s 137us/step - loss: 0.8433 - accuracy: 0.5519 - val_loss: 1.1815 - val_accuracy: 0.4530
Epoch 223/1000
270/270 [==============================] - 0s 98us/step - loss: 0.8442 - accuracy: 0.5556 - val_loss: 1.1830 - val_accuracy: 0.4530
Epoch 224/1000
270/270 [==============================] - 0s 102us/step - loss: 0.8464 - accuracy: 0.5370 - val_loss: 1.1827 - val_accuracy: 0.4444
Epoch 225/1000
270/270 [==============================] - 0s 195us/step - loss: 0.8462 - accuracy: 0.5556 - val_loss: 1.1876 - val_accuracy: 0.4530
Epoch 226/1000
270/270 [==============================] - 0s 99us/step - loss: 0.8466 - accuracy: 0.5556 - val_loss: 1.1928 - val_accuracy: 0.4530
Epoch 227/1000
270/270 [==============================] - 0s 106us/step - loss: 0.8481 - accuracy: 0.5556 - val_loss: 1.1934 - val_accuracy: 0.4530
Epoch 228/1000
270/270 [==============================] - 0s 104us/step - loss: 0.8462 - accuracy: 0.5556 - val_loss: 1.1882 - va

270/270 [==============================] - 0s 91us/step - loss: 0.8478 - accuracy: 0.5407 - val_loss: 1.1994 - val_accuracy: 0.4530
Epoch 333/1000
270/270 [==============================] - 0s 148us/step - loss: 0.8460 - accuracy: 0.5519 - val_loss: 1.2020 - val_accuracy: 0.4444
Epoch 334/1000
270/270 [==============================] - 0s 148us/step - loss: 0.8457 - accuracy: 0.5556 - val_loss: 1.2026 - val_accuracy: 0.4786
Epoch 335/1000
270/270 [==============================] - 0s 110us/step - loss: 0.8455 - accuracy: 0.5556 - val_loss: 1.1992 - val_accuracy: 0.4530
Epoch 336/1000
270/270 [==============================] - 0s 99us/step - loss: 0.8457 - accuracy: 0.5111 - val_loss: 1.1972 - val_accuracy: 0.4444
Epoch 337/1000
270/270 [==============================] - 0s 102us/step - loss: 0.8466 - accuracy: 0.5296 - val_loss: 1.2045 - val_accuracy: 0.4530
Epoch 338/1000
270/270 [==============================] - 0s 110us/step - loss: 0.8458 - accuracy: 0.5556 - val_loss: 1.2128 - va

270/270 [==============================] - 0s 127us/step - loss: 0.8482 - accuracy: 0.5593 - val_loss: 1.1978 - val_accuracy: 0.4701
Epoch 443/1000
270/270 [==============================] - 0s 100us/step - loss: 0.8473 - accuracy: 0.5407 - val_loss: 1.1910 - val_accuracy: 0.4444
Epoch 444/1000
270/270 [==============================] - 0s 122us/step - loss: 0.8451 - accuracy: 0.5481 - val_loss: 1.1969 - val_accuracy: 0.4530
Epoch 445/1000
270/270 [==============================] - 0s 178us/step - loss: 0.8450 - accuracy: 0.5556 - val_loss: 1.2041 - val_accuracy: 0.4786
Epoch 446/1000
270/270 [==============================] - 0s 212us/step - loss: 0.8445 - accuracy: 0.5556 - val_loss: 1.1931 - val_accuracy: 0.4701
Epoch 447/1000
270/270 [==============================] - 0s 147us/step - loss: 0.8449 - accuracy: 0.5630 - val_loss: 1.1884 - val_accuracy: 0.4530
Epoch 448/1000
270/270 [==============================] - 0s 188us/step - loss: 0.8463 - accuracy: 0.5222 - val_loss: 1.1852 - 

270/270 [==============================] - 0s 118us/step - loss: 0.8461 - accuracy: 0.5185 - val_loss: 1.2157 - val_accuracy: 0.4786
Epoch 553/1000
270/270 [==============================] - 0s 100us/step - loss: 0.8439 - accuracy: 0.5556 - val_loss: 1.2131 - val_accuracy: 0.4530
Epoch 554/1000
270/270 [==============================] - 0s 109us/step - loss: 0.8434 - accuracy: 0.5556 - val_loss: 1.2132 - val_accuracy: 0.4444
Epoch 555/1000
270/270 [==============================] - 0s 114us/step - loss: 0.8434 - accuracy: 0.5407 - val_loss: 1.2138 - val_accuracy: 0.4444
Epoch 556/1000
270/270 [==============================] - 0s 112us/step - loss: 0.8448 - accuracy: 0.5519 - val_loss: 1.2077 - val_accuracy: 0.4615
Epoch 557/1000
270/270 [==============================] - 0s 127us/step - loss: 0.8447 - accuracy: 0.5519 - val_loss: 1.2184 - val_accuracy: 0.4530
Epoch 558/1000
270/270 [==============================] - 0s 168us/step - loss: 0.8453 - accuracy: 0.5296 - val_loss: 1.2274 - 

270/270 [==============================] - 0s 188us/step - loss: 0.8466 - accuracy: 0.5519 - val_loss: 1.2028 - val_accuracy: 0.4786
Epoch 663/1000
270/270 [==============================] - 0s 152us/step - loss: 0.8442 - accuracy: 0.5556 - val_loss: 1.1995 - val_accuracy: 0.4786
Epoch 664/1000
270/270 [==============================] - 0s 143us/step - loss: 0.8426 - accuracy: 0.5593 - val_loss: 1.1917 - val_accuracy: 0.4530
Epoch 665/1000
270/270 [==============================] - 0s 162us/step - loss: 0.8488 - accuracy: 0.5259 - val_loss: 1.1940 - val_accuracy: 0.4701
Epoch 666/1000
270/270 [==============================] - 0s 162us/step - loss: 0.8441 - accuracy: 0.5444 - val_loss: 1.1892 - val_accuracy: 0.4786
Epoch 667/1000
270/270 [==============================] - 0s 155us/step - loss: 0.8450 - accuracy: 0.5556 - val_loss: 1.1903 - val_accuracy: 0.4786
Epoch 668/1000
270/270 [==============================] - 0s 209us/step - loss: 0.8430 - accuracy: 0.5556 - val_loss: 1.2003 - 

270/270 [==============================] - 0s 115us/step - loss: 0.8510 - accuracy: 0.5222 - val_loss: 1.1784 - val_accuracy: 0.4786
Epoch 773/1000
270/270 [==============================] - 0s 103us/step - loss: 0.8425 - accuracy: 0.5259 - val_loss: 1.1715 - val_accuracy: 0.4530
Epoch 774/1000
270/270 [==============================] - 0s 102us/step - loss: 0.8467 - accuracy: 0.5556 - val_loss: 1.1659 - val_accuracy: 0.4530
Epoch 775/1000
270/270 [==============================] - 0s 150us/step - loss: 0.8429 - accuracy: 0.5593 - val_loss: 1.1732 - val_accuracy: 0.4872
Epoch 776/1000
270/270 [==============================] - 0s 129us/step - loss: 0.8461 - accuracy: 0.5556 - val_loss: 1.1777 - val_accuracy: 0.4872
Epoch 777/1000
270/270 [==============================] - 0s 122us/step - loss: 0.8430 - accuracy: 0.5519 - val_loss: 1.1732 - val_accuracy: 0.4786
Epoch 778/1000
270/270 [==============================] - 0s 122us/step - loss: 0.8428 - accuracy: 0.5556 - val_loss: 1.1729 - 

270/270 [==============================] - 0s 66us/step - loss: 0.8451 - accuracy: 0.5556 - val_loss: 1.1672 - val_accuracy: 0.4530
Epoch 883/1000
270/270 [==============================] - 0s 83us/step - loss: 0.8450 - accuracy: 0.5481 - val_loss: 1.1686 - val_accuracy: 0.4786
Epoch 884/1000
270/270 [==============================] - 0s 68us/step - loss: 0.8486 - accuracy: 0.5519 - val_loss: 1.1632 - val_accuracy: 0.4530
Epoch 885/1000
270/270 [==============================] - 0s 81us/step - loss: 0.8466 - accuracy: 0.5519 - val_loss: 1.1653 - val_accuracy: 0.4786
Epoch 886/1000
270/270 [==============================] - 0s 70us/step - loss: 0.8458 - accuracy: 0.5519 - val_loss: 1.1735 - val_accuracy: 0.4872
Epoch 887/1000
270/270 [==============================] - 0s 81us/step - loss: 0.8442 - accuracy: 0.5519 - val_loss: 1.1682 - val_accuracy: 0.4786
Epoch 888/1000
270/270 [==============================] - 0s 72us/step - loss: 0.8426 - accuracy: 0.5556 - val_loss: 1.1670 - val_acc

Epoch 938/1000
270/270 [==============================] - 0s 85us/step - loss: 0.8441 - accuracy: 0.5556 - val_loss: 1.1802 - val_accuracy: 0.4786
Epoch 939/1000
270/270 [==============================] - 0s 72us/step - loss: 0.8425 - accuracy: 0.5556 - val_loss: 1.1788 - val_accuracy: 0.4786
Epoch 940/1000
270/270 [==============================] - 0s 89us/step - loss: 0.8438 - accuracy: 0.5370 - val_loss: 1.1831 - val_accuracy: 0.4444
Epoch 941/1000
270/270 [==============================] - 0s 75us/step - loss: 0.8438 - accuracy: 0.5185 - val_loss: 1.1834 - val_accuracy: 0.4530
Epoch 942/1000
270/270 [==============================] - 0s 73us/step - loss: 0.8434 - accuracy: 0.5370 - val_loss: 1.1839 - val_accuracy: 0.4444
Epoch 943/1000
270/270 [==============================] - 0s 76us/step - loss: 0.8444 - accuracy: 0.5481 - val_loss: 1.1828 - val_accuracy: 0.4872
Epoch 944/1000
270/270 [==============================] - 0s 80us/step - loss: 0.8435 - accuracy: 0.5556 - val_loss: 1

In [220]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist2_over2.history['accuracy'])*100))

over-sampling train accuracy: 54.77%


In [46]:
df_proba6 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_lasso_2.xlsx",
                        sheet_name=1,
                        index_col=None)

In [47]:
df_proba6

,phage,strain,phenotype,prediction,0,1,2
0,p0006kpresabs_qual,NRS249,2,1,1.888869e-01,5.108038e-01,3.003094e-01
1,p0006kpresabs_qual,NRS188,1,1,1.888869e-01,5.108038e-01,3.003094e-01
2,p0006kpresabs_qual,NRS232,2,2,4.222906e-01,7.029924e-02,5.074101e-01
3,p0006kpresabs_qual,NY439,2,2,3.558408e-04,2.976018e-04,9.993465e-01
4,p0006kpresabs_qual,GA27,2,1,3.940971e-01,4.184215e-01,1.874814e-01
...,...,...,...,...,...,...,...
984,p0017Skpresabs_qual,NRS252,0,0,7.239556e-01,2.760444e-01,1.176030e-09
985,p0017Skpresabs_qual,SR2852,1,1,1.052276e-07,9.999999e-01,1.101559e-28
986,p0017Skpresabs_qual,NRS108,1,1,1.540350e-17,1.000000e+00,9.011977e-16
987,p0017Skpresabs_qual,NRS202,0,0,6.888959e-01,3.111042e-01,2.228958e-09


In [48]:
y_prob6 = df_proba6[df_proba6['phage']=='p0006kpresabs_qual'].iloc[:,-3:]
y_prob6 = y_prob6.to_numpy()
y_prob6

array([[1.8888690e-01, 5.1080380e-01, 3.0030936e-01],
       [1.8888690e-01, 5.1080380e-01, 3.0030936e-01],
       [4.2229065e-01, 7.0299245e-02, 5.0741010e-01],
       [3.5584080e-04, 2.9760180e-04, 9.9934655e-01],
       [3.9409712e-01, 4.1842150e-01, 1.8748140e-01],
       [1.8888690e-01, 5.1080380e-01, 3.0030936e-01],
       [3.9409712e-01, 4.1842150e-01, 1.8748140e-01],
       [9.8233277e-01, 1.4282453e-02, 3.3847615e-03],
       [3.9409712e-01, 4.1842150e-01, 1.8748140e-01],
       [3.5584080e-04, 2.9760180e-04, 9.9934655e-01],
       [2.5331424e-04, 1.0664241e-02, 9.8908246e-01],
       [5.9236000e-04, 3.8518402e-01, 6.1422360e-01],
       [3.9409712e-01, 4.1842150e-01, 1.8748140e-01],
       [9.0621240e-01, 7.6262460e-02, 1.7525123e-02],
       [1.8888690e-01, 5.1080380e-01, 3.0030936e-01],
       [2.2520465e-04, 4.0490914e-07, 9.9977440e-01],
       [3.9409712e-01, 4.1842150e-01, 1.8748140e-01],
       [3.9409712e-01, 4.1842150e-01, 1.8748140e-01],
       [2.5331424e-04, 1.066

In [49]:
ovo6 = rocauc_ovo(y_sel_test_over, y_prob6, average="macro", multi_class="ovo")
ovo6

0.6386149463072539

In [50]:
ovr6 = rocauc_ovr(y_sel_test_over, y_prob6, average="macro", multi_class="ovr")
ovr6

0.6386149463072539

In [51]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_sel_train_over, X_sel_test_over, y_sel_train_over, y_sel_test_over = train_test_split(X_sel_over, y_sel_over,
                                                    test_size = 0.3,
                                                    random_state=789,
                                                    stratify=y_sel_over)

In [52]:
dat7 = pd.DataFrame(X_sel_test_over[:,-1])
dat7['test'] = y_sel_test_over

In [53]:
dat7

,0,test
0,NRS210,0
1,NRS205,2
2,312,2
3,GA15,2
4,SR4035,0
...,...,...
112,NRS265,2
113,CFBREBSa108,1
114,NY224,1
115,NRS386,2


In [54]:
X_sel_train_over = X_sel_train_over[:,:-1]
X_sel_test_over = X_sel_test_over[:,:-1]

In [109]:
model2_over3 = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [110]:
model2_over3.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [111]:
model2_over3.fit(X_sel_train_over, y_sel_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test_over, y_sel_test_over))

Train on 270 samples, validate on 117 samples
Epoch 1/1000
270/270 [==============================] - 0s 485us/step - loss: 1.2622 - accuracy: 0.3333 - val_loss: 1.1851 - val_accuracy: 0.3419
Epoch 2/1000
270/270 [==============================] - 0s 88us/step - loss: 1.1526 - accuracy: 0.3333 - val_loss: 1.1185 - val_accuracy: 0.3675
Epoch 3/1000
270/270 [==============================] - 0s 99us/step - loss: 1.1122 - accuracy: 0.3741 - val_loss: 1.1030 - val_accuracy: 0.4359
Epoch 4/1000
270/270 [==============================] - 0s 155us/step - loss: 1.1027 - accuracy: 0.3333 - val_loss: 1.0976 - val_accuracy: 0.3333
Epoch 5/1000
270/270 [==============================] - 0s 131us/step - loss: 1.0942 - accuracy: 0.3259 - val_loss: 1.0875 - val_accuracy: 0.3590
Epoch 6/1000
270/270 [==============================] - 0s 476us/step - loss: 1.0844 - accuracy: 0.3852 - val_loss: 1.0797 - val_accuracy: 0.3932
Epoch 7/1000
270/270 [==============================] - 0s 392us/step - loss: 1.

270/270 [==============================] - 0s 65us/step - loss: 0.9476 - accuracy: 0.4778 - val_loss: 0.9460 - val_accuracy: 0.5214
Epoch 112/1000
270/270 [==============================] - 0s 85us/step - loss: 0.9452 - accuracy: 0.4741 - val_loss: 0.9449 - val_accuracy: 0.5556
Epoch 113/1000
270/270 [==============================] - 0s 72us/step - loss: 0.9439 - accuracy: 0.5148 - val_loss: 0.9460 - val_accuracy: 0.5556
Epoch 114/1000
270/270 [==============================] - 0s 73us/step - loss: 0.9439 - accuracy: 0.5148 - val_loss: 0.9450 - val_accuracy: 0.5556
Epoch 115/1000
270/270 [==============================] - 0s 80us/step - loss: 0.9447 - accuracy: 0.5148 - val_loss: 0.9461 - val_accuracy: 0.5556
Epoch 116/1000
270/270 [==============================] - 0s 118us/step - loss: 0.9427 - accuracy: 0.5148 - val_loss: 0.9455 - val_accuracy: 0.5556
Epoch 117/1000
270/270 [==============================] - 0s 93us/step - loss: 0.9461 - accuracy: 0.5074 - val_loss: 0.9470 - val_ac

Epoch 222/1000
270/270 [==============================] - 0s 85us/step - loss: 0.9218 - accuracy: 0.5185 - val_loss: 0.9450 - val_accuracy: 0.5299
Epoch 223/1000
270/270 [==============================] - 0s 96us/step - loss: 0.9208 - accuracy: 0.5185 - val_loss: 0.9441 - val_accuracy: 0.5299
Epoch 224/1000
270/270 [==============================] - 0s 75us/step - loss: 0.9197 - accuracy: 0.5185 - val_loss: 0.9432 - val_accuracy: 0.5641
Epoch 225/1000
270/270 [==============================] - 0s 71us/step - loss: 0.9224 - accuracy: 0.5000 - val_loss: 0.9443 - val_accuracy: 0.5128
Epoch 226/1000
270/270 [==============================] - 0s 84us/step - loss: 0.9193 - accuracy: 0.5185 - val_loss: 0.9437 - val_accuracy: 0.5641
Epoch 227/1000
270/270 [==============================] - 0s 76us/step - loss: 0.9190 - accuracy: 0.5222 - val_loss: 0.9445 - val_accuracy: 0.5641
Epoch 228/1000
270/270 [==============================] - 0s 71us/step - loss: 0.9204 - accuracy: 0.5222 - val_loss: 0

270/270 [==============================] - 0s 118us/step - loss: 0.9063 - accuracy: 0.5259 - val_loss: 0.9428 - val_accuracy: 0.5641
Epoch 333/1000
270/270 [==============================] - 0s 104us/step - loss: 0.9065 - accuracy: 0.5259 - val_loss: 0.9423 - val_accuracy: 0.5641
Epoch 334/1000
270/270 [==============================] - 0s 95us/step - loss: 0.9057 - accuracy: 0.5148 - val_loss: 0.9438 - val_accuracy: 0.5128
Epoch 335/1000
270/270 [==============================] - 0s 113us/step - loss: 0.9061 - accuracy: 0.5148 - val_loss: 0.9426 - val_accuracy: 0.5641
Epoch 336/1000
270/270 [==============================] - 0s 119us/step - loss: 0.9054 - accuracy: 0.5259 - val_loss: 0.9432 - val_accuracy: 0.5641
Epoch 337/1000
270/270 [==============================] - 0s 103us/step - loss: 0.9054 - accuracy: 0.5148 - val_loss: 0.9465 - val_accuracy: 0.5128
Epoch 338/1000
270/270 [==============================] - 0s 118us/step - loss: 0.9082 - accuracy: 0.5000 - val_loss: 0.9450 - v

Epoch 388/1000
270/270 [==============================] - 0s 58us/step - loss: 0.9003 - accuracy: 0.5333 - val_loss: 0.9451 - val_accuracy: 0.5470
Epoch 389/1000
270/270 [==============================] - 0s 56us/step - loss: 0.9000 - accuracy: 0.5333 - val_loss: 0.9473 - val_accuracy: 0.5128
Epoch 390/1000
270/270 [==============================] - 0s 68us/step - loss: 0.9009 - accuracy: 0.5296 - val_loss: 0.9489 - val_accuracy: 0.5043
Epoch 391/1000
270/270 [==============================] - 0s 81us/step - loss: 0.9012 - accuracy: 0.5111 - val_loss: 0.9454 - val_accuracy: 0.5470
Epoch 392/1000
270/270 [==============================] - 0s 84us/step - loss: 0.9000 - accuracy: 0.5519 - val_loss: 0.9477 - val_accuracy: 0.5043
Epoch 393/1000
270/270 [==============================] - 0s 108us/step - loss: 0.8996 - accuracy: 0.5259 - val_loss: 0.9461 - val_accuracy: 0.5470
Epoch 394/1000
270/270 [==============================] - 0s 90us/step - loss: 0.8989 - accuracy: 0.5000 - val_loss: 

...

270/270 [==============================] - 0s 121us/step - loss: 0.8883 - accuracy: 0.5185 - val_loss: 0.9564 - val_accuracy: 0.5043
Epoch 610/1000
270/270 [==============================] - 0s 116us/step - loss: 0.8872 - accuracy: 0.5296 - val_loss: 0.9592 - val_accuracy: 0.5043
Epoch 611/1000
270/270 [==============================] - 0s 84us/step - loss: 0.8875 - accuracy: 0.5630 - val_loss: 0.9599 - val_accuracy: 0.5385
Epoch 612/1000
270/270 [==============================] - 0s 76us/step - loss: 0.8902 - accuracy: 0.5333 - val_loss: 0.9548 - val_accuracy: 0.5385
Epoch 613/1000
270/270 [==============================] - 0s 89us/step - loss: 0.8853 - accuracy: 0.5333 - val_loss: 0.9546 - val_accuracy: 0.5385
Epoch 614/1000
270/270 [==============================] - 0s 98us/step - loss: 0.8844 - accuracy: 0.5370 - val_loss: 0.9560 - val_accuracy: 0.5043
Epoch 615/1000
270/270 [==============================] - 0s 135us/step - loss: 0.8865 - accuracy: 0.5296 - val_loss: 0.9557 - val_

270/270 [==============================] - 0s 97us/step - loss: 0.8827 - accuracy: 0.5407 - val_loss: 0.9585 - val_accuracy: 0.5385
Epoch 720/1000
270/270 [==============================] - 0s 105us/step - loss: 0.8873 - accuracy: 0.5333 - val_loss: 0.9574 - val_accuracy: 0.5385
Epoch 721/1000
270/270 [==============================] - 0s 123us/step - loss: 0.8904 - accuracy: 0.5037 - val_loss: 0.9647 - val_accuracy: 0.5043
Epoch 722/1000
270/270 [==============================] - 0s 132us/step - loss: 0.8816 - accuracy: 0.5259 - val_loss: 0.9567 - val_accuracy: 0.5385
Epoch 723/1000
270/270 [==============================] - 0s 121us/step - loss: 0.8803 - accuracy: 0.5333 - val_loss: 0.9564 - val_accuracy: 0.5385
Epoch 724/1000
270/270 [==============================] - 0s 117us/step - loss: 0.8811 - accuracy: 0.5333 - val_loss: 0.9620 - val_accuracy: 0.5385
Epoch 725/1000
270/270 [==============================] - 0s 123us/step - loss: 0.8818 - accuracy: 0.5296 - val_loss: 0.9634 - v

270/270 [==============================] - 0s 111us/step - loss: 0.8770 - accuracy: 0.5370 - val_loss: 0.9633 - val_accuracy: 0.5470
Epoch 830/1000
270/270 [==============================] - 0s 104us/step - loss: 0.8773 - accuracy: 0.5370 - val_loss: 0.9611 - val_accuracy: 0.5470
Epoch 831/1000
270/270 [==============================] - 0s 81us/step - loss: 0.8755 - accuracy: 0.5222 - val_loss: 0.9641 - val_accuracy: 0.5128
Epoch 832/1000
270/270 [==============================] - 0s 87us/step - loss: 0.8741 - accuracy: 0.5333 - val_loss: 0.9631 - val_accuracy: 0.5470
Epoch 833/1000
270/270 [==============================] - 0s 79us/step - loss: 0.8764 - accuracy: 0.5370 - val_loss: 0.9620 - val_accuracy: 0.5470
Epoch 834/1000
270/270 [==============================] - 0s 83us/step - loss: 0.8759 - accuracy: 0.5074 - val_loss: 0.9617 - val_accuracy: 0.5128
Epoch 835/1000
270/270 [==============================] - 0s 90us/step - loss: 0.8764 - accuracy: 0.5333 - val_loss: 0.9628 - val_a

Epoch 885/1000
270/270 [==============================] - 0s 94us/step - loss: 0.8735 - accuracy: 0.5333 - val_loss: 0.9653 - val_accuracy: 0.5128
Epoch 886/1000
270/270 [==============================] - 0s 82us/step - loss: 0.8734 - accuracy: 0.5407 - val_loss: 0.9632 - val_accuracy: 0.5470
Epoch 887/1000
270/270 [==============================] - 0s 86us/step - loss: 0.8745 - accuracy: 0.5296 - val_loss: 0.9648 - val_accuracy: 0.5128
Epoch 888/1000
270/270 [==============================] - 0s 84us/step - loss: 0.8734 - accuracy: 0.5111 - val_loss: 0.9629 - val_accuracy: 0.5128
Epoch 889/1000
270/270 [==============================] - 0s 86us/step - loss: 0.8736 - accuracy: 0.5333 - val_loss: 0.9612 - val_accuracy: 0.5470
Epoch 890/1000
270/270 [==============================] - 0s 90us/step - loss: 0.8737 - accuracy: 0.5333 - val_loss: 0.9620 - val_accuracy: 0.5128
Epoch 891/1000
270/270 [==============================] - 0s 86us/step - loss: 0.8738 - accuracy: 0.5074 - val_loss: 0

270/270 [==============================] - 0s 87us/step - loss: 0.8720 - accuracy: 0.5185 - val_loss: 0.9698 - val_accuracy: 0.5128
Epoch 996/1000
270/270 [==============================] - 0s 83us/step - loss: 0.8743 - accuracy: 0.5148 - val_loss: 0.9691 - val_accuracy: 0.5470
Epoch 997/1000
270/270 [==============================] - 0s 88us/step - loss: 0.8763 - accuracy: 0.5074 - val_loss: 0.9744 - val_accuracy: 0.5128
Epoch 998/1000
270/270 [==============================] - 0s 83us/step - loss: 0.8723 - accuracy: 0.5222 - val_loss: 0.9691 - val_accuracy: 0.5470
Epoch 999/1000
270/270 [==============================] - 0s 89us/step - loss: 0.8705 - accuracy: 0.5370 - val_loss: 0.9710 - val_accuracy: 0.5470
Epoch 1000/1000
270/270 [==============================] - 0s 81us/step - loss: 0.8708 - accuracy: 0.5296 - val_loss: 0.9740 - val_accuracy: 0.5128


In [246]:
acc_test2_over3 = model2_over3.evaluate(X_sel_test_over, y_sel_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test2_over3*100))

117/117 [==============================] - 0s 65us/step
over-sampling test accuracy: 56.41%


In [112]:
pred7 = model2_over3.predict_classes(X_sel_test_over)
pred7

array([0, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 2, 1, 1, 1, 2, 1, 1, 2, 1, 1,
       1, 2, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 2, 1, 1, 1, 1, 1, 0, 1, 2, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 2, 1, 2, 1,
       2, 1, 1, 2, 1, 1, 2, 1, 2, 1, 1, 1, 0, 2, 2, 1, 1, 1, 1, 1, 1, 1,
       2, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1,
       0, 1, 2, 1, 1, 2, 2])

In [113]:
dat7['pred'] = pred7
dat7

,0,test,pred
0,NRS210,0,0
1,NRS205,2,2
2,312,2,1
3,GA15,2,1
4,SR4035,0,1
...,...,...,...
112,NRS265,2,2
113,CFBREBSa108,1,1
114,NY224,1,1
115,NRS386,2,2


In [114]:
proba7 = model2_over3.predict_proba(X_sel_test_over)
dat_proba7 = pd.DataFrame(proba7)

In [115]:
dat_proba7

,0,1,2
0,0.613208,2.812180e-01,0.105574
1,0.000199,6.834937e-07,0.999800
2,0.358946,3.982787e-01,0.242775
3,0.358946,3.982787e-01,0.242775
4,0.358946,3.982787e-01,0.242775
...,...,...,...
112,0.047203,1.739226e-03,0.951057
113,0.358946,3.982787e-01,0.242775
114,0.221757,4.551860e-01,0.323057
115,0.266153,1.839188e-01,0.549929


In [116]:
dat_proba7.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba7.csv", index = False,
         header=None)

In [117]:
dat7.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/7p006.csv", index = False,
         header=None)

In [250]:
hist2_over3 = model2_over3.fit(X_sel_train_over, y_sel_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test_over, y_sel_test_over))

Train on 270 samples, validate on 117 samples
Epoch 1/1000
270/270 [==============================] - 0s 108us/step - loss: 0.8684 - accuracy: 0.5370 - val_loss: 0.9881 - val_accuracy: 0.5641
Epoch 2/1000
270/270 [==============================] - 0s 106us/step - loss: 0.8686 - accuracy: 0.5407 - val_loss: 0.9894 - val_accuracy: 0.5299
Epoch 3/1000
270/270 [==============================] - 0s 140us/step - loss: 0.8683 - accuracy: 0.5333 - val_loss: 0.9900 - val_accuracy: 0.5299
Epoch 4/1000
270/270 [==============================] - 0s 168us/step - loss: 0.8681 - accuracy: 0.5148 - val_loss: 0.9910 - val_accuracy: 0.5641
Epoch 5/1000
270/270 [==============================] - 0s 124us/step - loss: 0.8708 - accuracy: 0.5333 - val_loss: 0.9943 - val_accuracy: 0.5641
Epoch 6/1000
270/270 [==============================] - 0s 135us/step - loss: 0.8703 - accuracy: 0.4926 - val_loss: 0.9944 - val_accuracy: 0.5641
Epoch 7/1000
270/270 [==============================] - 0s 142us/step - loss: 

Epoch 112/1000
270/270 [==============================] - 0s 95us/step - loss: 0.8684 - accuracy: 0.5037 - val_loss: 1.0169 - val_accuracy: 0.5299
Epoch 113/1000
270/270 [==============================] - 0s 102us/step - loss: 0.8681 - accuracy: 0.5148 - val_loss: 1.0141 - val_accuracy: 0.5641
Epoch 114/1000
270/270 [==============================] - 0s 88us/step - loss: 0.8651 - accuracy: 0.5370 - val_loss: 1.0185 - val_accuracy: 0.5299
Epoch 115/1000
270/270 [==============================] - 0s 90us/step - loss: 0.8671 - accuracy: 0.5296 - val_loss: 1.0170 - val_accuracy: 0.5299
Epoch 116/1000
270/270 [==============================] - 0s 90us/step - loss: 0.8659 - accuracy: 0.5185 - val_loss: 1.0128 - val_accuracy: 0.5641
Epoch 117/1000
270/270 [==============================] - 0s 87us/step - loss: 0.8674 - accuracy: 0.5370 - val_loss: 1.0149 - val_accuracy: 0.5641
Epoch 118/1000
270/270 [==============================] - 0s 84us/step - loss: 0.8665 - accuracy: 0.5407 - val_loss: 

270/270 [==============================] - 0s 91us/step - loss: 0.8644 - accuracy: 0.5370 - val_loss: 1.0188 - val_accuracy: 0.5641
Epoch 223/1000
270/270 [==============================] - 0s 96us/step - loss: 0.8666 - accuracy: 0.5370 - val_loss: 1.0222 - val_accuracy: 0.5299
Epoch 224/1000
270/270 [==============================] - 0s 97us/step - loss: 0.8655 - accuracy: 0.5259 - val_loss: 1.0221 - val_accuracy: 0.5641
Epoch 225/1000
270/270 [==============================] - 0s 87us/step - loss: 0.8640 - accuracy: 0.5370 - val_loss: 1.0230 - val_accuracy: 0.5641
Epoch 226/1000
270/270 [==============================] - 0s 103us/step - loss: 0.8656 - accuracy: 0.5370 - val_loss: 1.0221 - val_accuracy: 0.5641
Epoch 227/1000
270/270 [==============================] - 0s 92us/step - loss: 0.8655 - accuracy: 0.5370 - val_loss: 1.0244 - val_accuracy: 0.5299
Epoch 228/1000
270/270 [==============================] - 0s 92us/step - loss: 0.8643 - accuracy: 0.5259 - val_loss: 1.0234 - val_ac

270/270 [==============================] - 0s 98us/step - loss: 0.8651 - accuracy: 0.5333 - val_loss: 1.0233 - val_accuracy: 0.5556
Epoch 333/1000
270/270 [==============================] - 0s 92us/step - loss: 0.8647 - accuracy: 0.5333 - val_loss: 1.0223 - val_accuracy: 0.5556
Epoch 334/1000
270/270 [==============================] - 0s 128us/step - loss: 0.8619 - accuracy: 0.5370 - val_loss: 1.0242 - val_accuracy: 0.5556
Epoch 335/1000
270/270 [==============================] - 0s 126us/step - loss: 0.8665 - accuracy: 0.5370 - val_loss: 1.0235 - val_accuracy: 0.5556
Epoch 336/1000
270/270 [==============================] - 0s 115us/step - loss: 0.8652 - accuracy: 0.5185 - val_loss: 1.0225 - val_accuracy: 0.5214
Epoch 337/1000
270/270 [==============================] - 0s 224us/step - loss: 0.8648 - accuracy: 0.5037 - val_loss: 1.0262 - val_accuracy: 0.5214
Epoch 338/1000
270/270 [==============================] - 0s 220us/step - loss: 0.8623 - accuracy: 0.5556 - val_loss: 1.0260 - va

270/270 [==============================] - 0s 163us/step - loss: 0.8634 - accuracy: 0.5259 - val_loss: 1.0202 - val_accuracy: 0.5214
Epoch 443/1000
270/270 [==============================] - 0s 153us/step - loss: 0.8650 - accuracy: 0.5333 - val_loss: 1.0196 - val_accuracy: 0.5556
Epoch 444/1000
270/270 [==============================] - 0s 187us/step - loss: 0.8623 - accuracy: 0.5370 - val_loss: 1.0229 - val_accuracy: 0.5556
Epoch 445/1000
270/270 [==============================] - 0s 166us/step - loss: 0.8645 - accuracy: 0.4963 - val_loss: 1.0261 - val_accuracy: 0.5214
Epoch 446/1000
270/270 [==============================] - 0s 186us/step - loss: 0.8617 - accuracy: 0.5259 - val_loss: 1.0243 - val_accuracy: 0.5556
Epoch 447/1000
270/270 [==============================] - 0s 124us/step - loss: 0.8633 - accuracy: 0.5333 - val_loss: 1.0284 - val_accuracy: 0.5556
Epoch 448/1000
270/270 [==============================] - 0s 156us/step - loss: 0.8625 - accuracy: 0.5111 - val_loss: 1.0316 - 

...

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.185678). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


Epoch 669/1000
270/270 [==============================] - 0s 130us/step - loss: 0.8614 - accuracy: 0.5407 - val_loss: 1.0553 - val_accuracy: 0.5214
Epoch 670/1000
270/270 [==============================] - 0s 156us/step - loss: 0.8603 - accuracy: 0.5333 - val_loss: 1.0574 - val_accuracy: 0.5214
Epoch 671/1000
270/270 [==============================] - 0s 520us/step - loss: 0.8621 - accuracy: 0.5148 - val_loss: 1.0550 - val_accuracy: 0.5556
Epoch 672/1000
270/270 [==============================] - 0s 313us/step - loss: 0.8616 - accuracy: 0.5370 - val_loss: 1.0516 - val_accuracy: 0.5556
Epoch 673/1000
270/270 [==============================] - 0s 186us/step - loss: 0.8600 - accuracy: 0.5370 - val_loss: 1.0544 - val_accuracy: 0.5556
Epoch 674/1000
270/270 [==============================] - 0s 221us/step - loss: 0.8603 - accuracy: 0.5185 - val_loss: 1.0582 - val_accuracy: 0.5214
Epoch 675/1000
270/270 [==============================] - 0s 418us/step - loss: 0.8613 - accuracy: 0.5333 - val_

270/270 [==============================] - 0s 391us/step - loss: 0.8621 - accuracy: 0.5333 - val_loss: 1.0600 - val_accuracy: 0.5214
Epoch 725/1000
270/270 [==============================] - 0s 161us/step - loss: 0.8649 - accuracy: 0.5333 - val_loss: 1.0657 - val_accuracy: 0.5214
Epoch 726/1000
270/270 [==============================] - 0s 159us/step - loss: 0.8649 - accuracy: 0.5000 - val_loss: 1.0585 - val_accuracy: 0.5556
Epoch 727/1000
270/270 [==============================] - 0s 163us/step - loss: 0.8616 - accuracy: 0.5370 - val_loss: 1.0605 - val_accuracy: 0.5556
Epoch 728/1000
270/270 [==============================] - ETA: 0s - loss: 0.7845 - accuracy: 0.62 - 0s 199us/step - loss: 0.8607 - accuracy: 0.5370 - val_loss: 1.0599 - val_accuracy: 0.5214
Epoch 729/1000
270/270 [==============================] - 0s 873us/step - loss: 0.8617 - accuracy: 0.5333 - val_loss: 1.0595 - val_accuracy: 0.5214
Epoch 730/1000
270/270 [==============================] - 0s 158us/step - loss: 0.863

Epoch 835/1000
270/270 [==============================] - 0s 83us/step - loss: 0.8617 - accuracy: 0.5296 - val_loss: 1.0663 - val_accuracy: 0.5128
Epoch 836/1000
270/270 [==============================] - 0s 77us/step - loss: 0.8604 - accuracy: 0.5333 - val_loss: 1.0623 - val_accuracy: 0.5556
Epoch 837/1000
270/270 [==============================] - 0s 73us/step - loss: 0.8601 - accuracy: 0.5370 - val_loss: 1.0601 - val_accuracy: 0.5556
Epoch 838/1000
270/270 [==============================] - 0s 76us/step - loss: 0.8604 - accuracy: 0.5370 - val_loss: 1.0599 - val_accuracy: 0.5556
Epoch 839/1000
270/270 [==============================] - 0s 73us/step - loss: 0.8596 - accuracy: 0.5370 - val_loss: 1.0617 - val_accuracy: 0.5556
Epoch 840/1000
270/270 [==============================] - 0s 70us/step - loss: 0.8623 - accuracy: 0.5037 - val_loss: 1.0669 - val_accuracy: 0.5214
Epoch 841/1000
270/270 [==============================] - 0s 131us/step - loss: 0.8592 - accuracy: 0.5333 - val_loss: 

270/270 [==============================] - 0s 110us/step - loss: 0.8586 - accuracy: 0.5370 - val_loss: 1.0743 - val_accuracy: 0.5556
Epoch 946/1000
270/270 [==============================] - 0s 113us/step - loss: 0.8607 - accuracy: 0.5370 - val_loss: 1.0719 - val_accuracy: 0.5556
Epoch 947/1000
270/270 [==============================] - 0s 120us/step - loss: 0.8606 - accuracy: 0.5370 - val_loss: 1.0758 - val_accuracy: 0.5556
Epoch 948/1000
270/270 [==============================] - 0s 122us/step - loss: 0.8592 - accuracy: 0.5333 - val_loss: 1.0749 - val_accuracy: 0.5556
Epoch 949/1000
270/270 [==============================] - 0s 157us/step - loss: 0.8577 - accuracy: 0.5407 - val_loss: 1.0747 - val_accuracy: 0.5214
Epoch 950/1000
270/270 [==============================] - 0s 184us/step - loss: 0.8591 - accuracy: 0.5296 - val_loss: 1.0802 - val_accuracy: 0.5556
Epoch 951/1000
270/270 [==============================] - 0s 118us/step - loss: 0.8754 - accuracy: 0.4889 - val_loss: 1.0754 - 

In [251]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist2_over3.history['accuracy'])*100))

over-sampling train accuracy: 52.94%


In [55]:
df_proba7 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_lasso_2.xlsx",
                        sheet_name=2,
                        index_col=None)

In [56]:
df_proba7

,phage,strain,phenotype,prediction,0,1,2
0,p0006kpresabs_qual,NRS210,0,0,6.132076e-01,2.812180e-01,1.055744e-01
1,p0006kpresabs_qual,NRS205,2,2,1.993202e-04,6.834937e-07,9.998000e-01
2,p0006kpresabs_qual,312,2,1,3.589463e-01,3.982787e-01,2.427750e-01
3,p0006kpresabs_qual,GA15,2,1,3.589463e-01,3.982787e-01,2.427750e-01
4,p0006kpresabs_qual,SR4035,0,1,3.589463e-01,3.982787e-01,2.427750e-01
...,...,...,...,...,...,...,...
984,p0017Skpresabs_qual,NRS383,1,0,5.477194e-01,4.522807e-01,1.761374e-08
985,p0017Skpresabs_qual,NRS218,1,1,6.953657e-05,9.999305e-01,3.132419e-10
986,p0017Skpresabs_qual,NRS209,2,2,2.713214e-09,6.656316e-09,1.000000e+00
987,p0017Skpresabs_qual,SR2852,1,1,9.956684e-12,1.000000e+00,7.441288e-26


In [57]:
y_prob7 = df_proba7[df_proba7['phage']=='p0006kpresabs_qual'].iloc[:,-3:]
y_prob7 = y_prob7.to_numpy()
y_prob7

array([[6.13207640e-01, 2.81218000e-01, 1.05574414e-01],
       [1.99320160e-04, 6.83493700e-07, 9.99800000e-01],
       [3.58946260e-01, 3.98278740e-01, 2.42775010e-01],
       [3.58946260e-01, 3.98278740e-01, 2.42775010e-01],
       [3.58946260e-01, 3.98278740e-01, 2.42775010e-01],
       [3.58946260e-01, 3.98278740e-01, 2.42775010e-01],
       [3.58946260e-01, 3.98278740e-01, 2.42775010e-01],
       [2.21757250e-01, 4.55185980e-01, 3.23056800e-01],
       [3.58946260e-01, 3.98278740e-01, 2.42775010e-01],
       [3.58946260e-01, 3.98278740e-01, 2.42775010e-01],
       [3.58946260e-01, 3.98278740e-01, 2.42775010e-01],
       [6.13207640e-01, 2.81218000e-01, 1.05574414e-01],
       [2.73889710e-02, 3.94654400e-01, 5.77956600e-01],
       [3.58946260e-01, 3.98278740e-01, 2.42775010e-01],
       [3.58946260e-01, 3.98278740e-01, 2.42775010e-01],
       [3.58946260e-01, 3.98278740e-01, 2.42775010e-01],
       [2.73889710e-02, 3.94654400e-01, 5.77956600e-01],
       [3.58946260e-01, 3.98278

In [58]:
ovo7 = rocauc_ovo(y_sel_test_over, y_prob7, average="macro", multi_class="ovo")
ovo7

0.7072101687486304

In [59]:
ovr7 = rocauc_ovr(y_sel_test_over, y_prob7, average="macro", multi_class="ovr")
ovr7

0.7072101687486304

In [60]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_sel_train_over, X_sel_test_over, y_sel_train_over, y_sel_test_over = train_test_split(X_sel_over, y_sel_over,
                                                    test_size = 0.3,
                                                    random_state=890,
                                                    stratify=y_sel_over)

In [61]:
dat8 = pd.DataFrame(X_sel_test_over[:,-1])
dat8['test'] = y_sel_test_over

In [62]:
dat8

,0,test
0,NRS236,1
1,NRS113,2
2,CFBRSa23,0
3,NRS249,2
4,107,1
...,...,...
112,NY439,2
113,NRS106,0
114,221,0
115,NRS386,2


In [63]:
X_sel_train_over = X_sel_train_over[:,:-1]
X_sel_test_over = X_sel_test_over[:,:-1]

In [64]:
model2_over4 = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [123]:
model2_over4.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [124]:
model2_over4.fit(X_sel_train_over, y_sel_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test_over, y_sel_test_over))

Train on 270 samples, validate on 117 samples
Epoch 1/1000
270/270 [==============================] - 0s 617us/step - loss: 1.0973 - accuracy: 0.2963 - val_loss: 1.0694 - val_accuracy: 0.3932
Epoch 2/1000
270/270 [==============================] - 0s 156us/step - loss: 1.0744 - accuracy: 0.3815 - val_loss: 1.0552 - val_accuracy: 0.4701
Epoch 3/1000
270/270 [==============================] - 0s 178us/step - loss: 1.0619 - accuracy: 0.4222 - val_loss: 1.0481 - val_accuracy: 0.4188
Epoch 4/1000
270/270 [==============================] - 0s 130us/step - loss: 1.0544 - accuracy: 0.5000 - val_loss: 1.0420 - val_accuracy: 0.4786
Epoch 5/1000
270/270 [==============================] - 0s 138us/step - loss: 1.0479 - accuracy: 0.4704 - val_loss: 1.0376 - val_accuracy: 0.4786
Epoch 6/1000
270/270 [==============================] - 0s 118us/step - loss: 1.0428 - accuracy: 0.4741 - val_loss: 1.0322 - val_accuracy: 0.4786
Epoch 7/1000
270/270 [==============================] - 0s 119us/step - loss: 

Epoch 112/1000
270/270 [==============================] - 0s 94us/step - loss: 0.9191 - accuracy: 0.5481 - val_loss: 1.0080 - val_accuracy: 0.4786
Epoch 113/1000
270/270 [==============================] - 0s 84us/step - loss: 0.9191 - accuracy: 0.5481 - val_loss: 1.0070 - val_accuracy: 0.4786
Epoch 114/1000
270/270 [==============================] - 0s 138us/step - loss: 0.9202 - accuracy: 0.5519 - val_loss: 1.0086 - val_accuracy: 0.4872
Epoch 115/1000
270/270 [==============================] - 0s 98us/step - loss: 0.9192 - accuracy: 0.5519 - val_loss: 1.0054 - val_accuracy: 0.4786
Epoch 116/1000
270/270 [==============================] - 0s 123us/step - loss: 0.9178 - accuracy: 0.5481 - val_loss: 1.0065 - val_accuracy: 0.4786
Epoch 117/1000
270/270 [==============================] - 0s 107us/step - loss: 0.9212 - accuracy: 0.5481 - val_loss: 1.0089 - val_accuracy: 0.4786
Epoch 118/1000
270/270 [==============================] - 0s 139us/step - loss: 0.9189 - accuracy: 0.5481 - val_los

Epoch 223/1000
270/270 [==============================] - 0s 97us/step - loss: 0.9049 - accuracy: 0.5556 - val_loss: 1.0155 - val_accuracy: 0.4872
Epoch 224/1000
270/270 [==============================] - 0s 70us/step - loss: 0.9066 - accuracy: 0.5556 - val_loss: 1.0160 - val_accuracy: 0.4872
Epoch 225/1000
270/270 [==============================] - 0s 93us/step - loss: 0.9044 - accuracy: 0.5556 - val_loss: 1.0121 - val_accuracy: 0.4872
Epoch 226/1000
270/270 [==============================] - 0s 128us/step - loss: 0.9046 - accuracy: 0.5556 - val_loss: 1.0134 - val_accuracy: 0.4872
Epoch 227/1000
270/270 [==============================] - 0s 108us/step - loss: 0.9049 - accuracy: 0.5556 - val_loss: 1.0139 - val_accuracy: 0.4872
Epoch 228/1000
270/270 [==============================] - 0s 168us/step - loss: 0.9064 - accuracy: 0.5556 - val_loss: 1.0125 - val_accuracy: 0.4872
Epoch 229/1000
270/270 [==============================] - 0s 93us/step - loss: 0.9043 - accuracy: 0.5556 - val_loss

Epoch 334/1000
270/270 [==============================] - 0s 95us/step - loss: 0.9014 - accuracy: 0.5556 - val_loss: 1.0234 - val_accuracy: 0.4872
Epoch 335/1000
270/270 [==============================] - 0s 95us/step - loss: 0.8980 - accuracy: 0.5556 - val_loss: 1.0177 - val_accuracy: 0.4872
Epoch 336/1000
270/270 [==============================] - 0s 70us/step - loss: 0.8968 - accuracy: 0.5556 - val_loss: 1.0207 - val_accuracy: 0.4872
Epoch 337/1000
270/270 [==============================] - 0s 84us/step - loss: 0.8947 - accuracy: 0.5556 - val_loss: 1.0184 - val_accuracy: 0.4872
Epoch 338/1000
270/270 [==============================] - 0s 79us/step - loss: 0.8981 - accuracy: 0.5556 - val_loss: 1.0194 - val_accuracy: 0.4872
Epoch 339/1000
270/270 [==============================] - 0s 86us/step - loss: 0.8960 - accuracy: 0.5556 - val_loss: 1.0200 - val_accuracy: 0.4872
Epoch 340/1000
270/270 [==============================] - 0s 93us/step - loss: 0.8959 - accuracy: 0.5556 - val_loss: 1

Epoch 390/1000
270/270 [==============================] - 0s 115us/step - loss: 0.8959 - accuracy: 0.5556 - val_loss: 1.0260 - val_accuracy: 0.4872
Epoch 391/1000
270/270 [==============================] - 0s 84us/step - loss: 0.8936 - accuracy: 0.5556 - val_loss: 1.0222 - val_accuracy: 0.4872
Epoch 392/1000
270/270 [==============================] - 0s 74us/step - loss: 0.8922 - accuracy: 0.5556 - val_loss: 1.0222 - val_accuracy: 0.4872
Epoch 393/1000
270/270 [==============================] - 0s 97us/step - loss: 0.8932 - accuracy: 0.5556 - val_loss: 1.0231 - val_accuracy: 0.4872
Epoch 394/1000
270/270 [==============================] - 0s 92us/step - loss: 0.8941 - accuracy: 0.5556 - val_loss: 1.0202 - val_accuracy: 0.4872
Epoch 395/1000
270/270 [==============================] - 0s 131us/step - loss: 0.8935 - accuracy: 0.5556 - val_loss: 1.0198 - val_accuracy: 0.4872
Epoch 396/1000
270/270 [==============================] - 0s 96us/step - loss: 0.8947 - accuracy: 0.5556 - val_loss:

...

270/270 [==============================] - 0s 111us/step - loss: 0.8858 - accuracy: 0.5556 - val_loss: 1.0436 - val_accuracy: 0.4872
Epoch 611/1000
270/270 [==============================] - 0s 93us/step - loss: 0.8830 - accuracy: 0.5556 - val_loss: 1.0388 - val_accuracy: 0.4957
Epoch 612/1000
270/270 [==============================] - 0s 87us/step - loss: 0.8842 - accuracy: 0.5593 - val_loss: 1.0393 - val_accuracy: 0.4957
Epoch 613/1000
270/270 [==============================] - 0s 90us/step - loss: 0.8844 - accuracy: 0.5593 - val_loss: 1.0352 - val_accuracy: 0.4957
Epoch 614/1000
270/270 [==============================] - 0s 82us/step - loss: 0.8869 - accuracy: 0.5296 - val_loss: 1.0369 - val_accuracy: 0.4957
Epoch 615/1000
270/270 [==============================] - 0s 86us/step - loss: 0.8834 - accuracy: 0.5593 - val_loss: 1.0346 - val_accuracy: 0.4957
Epoch 616/1000
270/270 [==============================] - 0s 83us/step - loss: 0.8867 - accuracy: 0.5593 - val_loss: 1.0367 - val_ac

Epoch 666/1000
270/270 [==============================] - 0s 98us/step - loss: 0.8819 - accuracy: 0.5593 - val_loss: 1.0412 - val_accuracy: 0.4957
Epoch 667/1000
270/270 [==============================] - 0s 89us/step - loss: 0.8814 - accuracy: 0.5593 - val_loss: 1.0409 - val_accuracy: 0.5043
Epoch 668/1000
270/270 [==============================] - 0s 96us/step - loss: 0.8819 - accuracy: 0.5593 - val_loss: 1.0397 - val_accuracy: 0.5043
Epoch 669/1000
270/270 [==============================] - 0s 114us/step - loss: 0.8789 - accuracy: 0.5630 - val_loss: 1.0403 - val_accuracy: 0.5043
Epoch 670/1000
270/270 [==============================] - 0s 114us/step - loss: 0.8826 - accuracy: 0.5630 - val_loss: 1.0414 - val_accuracy: 0.5043
Epoch 671/1000
270/270 [==============================] - 0s 123us/step - loss: 0.8800 - accuracy: 0.5630 - val_loss: 1.0387 - val_accuracy: 0.5043
Epoch 672/1000
270/270 [==============================] - 0s 120us/step - loss: 0.8819 - accuracy: 0.5630 - val_los

270/270 [==============================] - 0s 117us/step - loss: 0.8769 - accuracy: 0.5630 - val_loss: 1.0501 - val_accuracy: 0.5043
Epoch 777/1000
270/270 [==============================] - 0s 119us/step - loss: 0.8763 - accuracy: 0.5630 - val_loss: 1.0482 - val_accuracy: 0.5043
Epoch 778/1000
270/270 [==============================] - 0s 157us/step - loss: 0.8775 - accuracy: 0.5630 - val_loss: 1.0470 - val_accuracy: 0.5043
Epoch 779/1000
270/270 [==============================] - 0s 128us/step - loss: 0.8798 - accuracy: 0.5630 - val_loss: 1.0500 - val_accuracy: 0.5043
Epoch 780/1000
270/270 [==============================] - 0s 141us/step - loss: 0.8769 - accuracy: 0.5630 - val_loss: 1.0546 - val_accuracy: 0.5043
Epoch 781/1000
270/270 [==============================] - 0s 107us/step - loss: 0.8770 - accuracy: 0.5630 - val_loss: 1.0543 - val_accuracy: 0.5043
Epoch 782/1000
270/270 [==============================] - 0s 96us/step - loss: 0.8794 - accuracy: 0.5630 - val_loss: 1.0489 - v

270/270 [==============================] - 0s 78us/step - loss: 0.8729 - accuracy: 0.5667 - val_loss: 1.0598 - val_accuracy: 0.5043
Epoch 887/1000
270/270 [==============================] - 0s 89us/step - loss: 0.8730 - accuracy: 0.5630 - val_loss: 1.0574 - val_accuracy: 0.5043
Epoch 888/1000
270/270 [==============================] - 0s 78us/step - loss: 0.8737 - accuracy: 0.5667 - val_loss: 1.0576 - val_accuracy: 0.5043
Epoch 889/1000
270/270 [==============================] - 0s 84us/step - loss: 0.8735 - accuracy: 0.5667 - val_loss: 1.0626 - val_accuracy: 0.5043
Epoch 890/1000
270/270 [==============================] - 0s 98us/step - loss: 0.8732 - accuracy: 0.5667 - val_loss: 1.0599 - val_accuracy: 0.5043
Epoch 891/1000
270/270 [==============================] - 0s 85us/step - loss: 0.8747 - accuracy: 0.5667 - val_loss: 1.0578 - val_accuracy: 0.5043
Epoch 892/1000
270/270 [==============================] - 0s 86us/step - loss: 0.8741 - accuracy: 0.5667 - val_loss: 1.0662 - val_acc

Epoch 942/1000
270/270 [==============================] - 0s 98us/step - loss: 0.8720 - accuracy: 0.5667 - val_loss: 1.0638 - val_accuracy: 0.5043
Epoch 943/1000
270/270 [==============================] - 0s 116us/step - loss: 0.8713 - accuracy: 0.5667 - val_loss: 1.0604 - val_accuracy: 0.5043
Epoch 944/1000
270/270 [==============================] - 0s 95us/step - loss: 0.8716 - accuracy: 0.5667 - val_loss: 1.0583 - val_accuracy: 0.5043
Epoch 945/1000
270/270 [==============================] - 0s 101us/step - loss: 0.8710 - accuracy: 0.5667 - val_loss: 1.0570 - val_accuracy: 0.5043
Epoch 946/1000
270/270 [==============================] - 0s 94us/step - loss: 0.8744 - accuracy: 0.5667 - val_loss: 1.0558 - val_accuracy: 0.5043
Epoch 947/1000
270/270 [==============================] - 0s 94us/step - loss: 0.8715 - accuracy: 0.5667 - val_loss: 1.0621 - val_accuracy: 0.5043
Epoch 948/1000
270/270 [==============================] - 0s 103us/step - loss: 0.8703 - accuracy: 0.5667 - val_loss

In [279]:
acc_test2_over4 = model2_over4.evaluate(X_sel_test_over, y_sel_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test2_over4*100))

117/117 [==============================] - 0s 90us/step
over-sampling test accuracy: 48.72%


In [125]:
pred8 = model2_over4.predict_classes(X_sel_test_over)
pred8

array([2, 2, 0, 1, 0, 2, 0, 0, 0, 0, 0, 2, 0, 2, 0, 2, 1, 0, 2, 2, 1, 0,
       0, 0, 0, 0, 2, 0, 0, 0, 0, 1, 0, 0, 1, 2, 0, 2, 0, 2, 0, 2, 1, 2,
       2, 1, 0, 0, 0, 1, 2, 0, 1, 2, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 2,
       0, 0, 1, 1, 0, 2, 2, 0, 0, 0, 2, 2, 2, 0, 0, 2, 0, 0, 0, 0, 1, 2,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 1, 2, 2, 2, 0, 0, 2, 0, 0, 0,
       1, 0, 2, 0, 0, 2, 0])

In [126]:
dat8['pred'] = pred8
dat8

,0,test,pred
0,NRS236,1,2
1,NRS113,2,2
2,CFBRSa23,0,0
3,NRS249,2,1
4,107,1,0
...,...,...,...
112,NY439,2,2
113,NRS106,0,0
114,221,0,0
115,NRS386,2,2


In [127]:
proba8 = model2_over4.predict_proba(X_sel_test_over)
dat_proba8 = pd.DataFrame(proba8)

In [128]:
dat_proba8

,0,1,2
0,0.013220,0.244626,0.742154
1,0.034782,0.280668,0.684549
2,0.409025,0.340501,0.250474
3,0.198791,0.533104,0.268105
4,0.409025,0.340501,0.250474
...,...,...,...
112,0.000021,0.000006,0.999972
113,0.409025,0.340501,0.250474
114,0.409025,0.340501,0.250474
115,0.138160,0.309219,0.552621


In [129]:
dat_proba8.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba8.csv", index = False,
         header=None)

In [130]:
dat8.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/8p006.csv", index = False,
         header=None)

In [283]:
hist2_over4 = model2_over4.fit(X_sel_train_over, y_sel_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test_over, y_sel_test_over))

Train on 270 samples, validate on 117 samples
Epoch 1/1000
270/270 [==============================] - 0s 98us/step - loss: 0.8734 - accuracy: 0.5667 - val_loss: 1.0262 - val_accuracy: 0.4872
Epoch 2/1000
270/270 [==============================] - 0s 104us/step - loss: 0.8734 - accuracy: 0.5667 - val_loss: 1.0278 - val_accuracy: 0.4872
Epoch 3/1000
270/270 [==============================] - 0s 112us/step - loss: 0.8710 - accuracy: 0.5667 - val_loss: 1.0307 - val_accuracy: 0.4872
Epoch 4/1000
270/270 [==============================] - 0s 123us/step - loss: 0.8737 - accuracy: 0.5667 - val_loss: 1.0280 - val_accuracy: 0.4872
Epoch 5/1000
270/270 [==============================] - 0s 140us/step - loss: 0.8734 - accuracy: 0.5667 - val_loss: 1.0285 - val_accuracy: 0.4872
Epoch 6/1000
270/270 [==============================] - 0s 118us/step - loss: 0.8812 - accuracy: 0.5667 - val_loss: 1.0328 - val_accuracy: 0.4872
Epoch 7/1000
270/270 [==============================] - 0s 114us/step - loss: 0

Epoch 112/1000
270/270 [==============================] - 0s 103us/step - loss: 0.8688 - accuracy: 0.5704 - val_loss: 1.0377 - val_accuracy: 0.4872
Epoch 113/1000
270/270 [==============================] - 0s 109us/step - loss: 0.8687 - accuracy: 0.5667 - val_loss: 1.0367 - val_accuracy: 0.4872
Epoch 114/1000
270/270 [==============================] - 0s 332us/step - loss: 0.8715 - accuracy: 0.5667 - val_loss: 1.0371 - val_accuracy: 0.4872
Epoch 115/1000
270/270 [==============================] - 0s 128us/step - loss: 0.8701 - accuracy: 0.5704 - val_loss: 1.0384 - val_accuracy: 0.4872
Epoch 116/1000
270/270 [==============================] - 0s 164us/step - loss: 0.8692 - accuracy: 0.5704 - val_loss: 1.0368 - val_accuracy: 0.4872
Epoch 117/1000
270/270 [==============================] - 0s 751us/step - loss: 0.8727 - accuracy: 0.5704 - val_loss: 1.0320 - val_accuracy: 0.4872
Epoch 118/1000
270/270 [==============================] - 0s 161us/step - loss: 0.8703 - accuracy: 0.5704 - val_

270/270 [==============================] - 0s 85us/step - loss: 0.8684 - accuracy: 0.5704 - val_loss: 1.0466 - val_accuracy: 0.4872
Epoch 223/1000
270/270 [==============================] - 0s 125us/step - loss: 0.8720 - accuracy: 0.5704 - val_loss: 1.0438 - val_accuracy: 0.4872
Epoch 224/1000
270/270 [==============================] - 0s 110us/step - loss: 0.8705 - accuracy: 0.5704 - val_loss: 1.0469 - val_accuracy: 0.4872
Epoch 225/1000
270/270 [==============================] - 0s 92us/step - loss: 0.8685 - accuracy: 0.5704 - val_loss: 1.0423 - val_accuracy: 0.4872
Epoch 226/1000
270/270 [==============================] - 0s 125us/step - loss: 0.8692 - accuracy: 0.5704 - val_loss: 1.0467 - val_accuracy: 0.4872
Epoch 227/1000
270/270 [==============================] - 0s 118us/step - loss: 0.8696 - accuracy: 0.5704 - val_loss: 1.0413 - val_accuracy: 0.4872
Epoch 228/1000
270/270 [==============================] - 0s 92us/step - loss: 0.8684 - accuracy: 0.5704 - val_loss: 1.0461 - val

270/270 [==============================] - 0s 148us/step - loss: 0.8650 - accuracy: 0.5704 - val_loss: 1.0467 - val_accuracy: 0.4872
Epoch 333/1000
270/270 [==============================] - 0s 116us/step - loss: 0.8676 - accuracy: 0.5667 - val_loss: 1.0453 - val_accuracy: 0.4786
Epoch 334/1000
270/270 [==============================] - 0s 167us/step - loss: 0.8677 - accuracy: 0.5667 - val_loss: 1.0508 - val_accuracy: 0.4872
Epoch 335/1000
270/270 [==============================] - 0s 140us/step - loss: 0.8676 - accuracy: 0.5704 - val_loss: 1.0533 - val_accuracy: 0.4872
Epoch 336/1000
270/270 [==============================] - 0s 131us/step - loss: 0.8663 - accuracy: 0.5704 - val_loss: 1.0525 - val_accuracy: 0.4872
Epoch 337/1000
270/270 [==============================] - 0s 116us/step - loss: 0.8661 - accuracy: 0.5704 - val_loss: 1.0515 - val_accuracy: 0.4872
Epoch 338/1000
270/270 [==============================] - 0s 132us/step - loss: 0.8664 - accuracy: 0.5704 - val_loss: 1.0510 - 

270/270 [==============================] - 0s 111us/step - loss: 0.8665 - accuracy: 0.5704 - val_loss: 1.0518 - val_accuracy: 0.4872
Epoch 443/1000
270/270 [==============================] - 0s 107us/step - loss: 0.8686 - accuracy: 0.5704 - val_loss: 1.0526 - val_accuracy: 0.4872
Epoch 444/1000
270/270 [==============================] - 0s 157us/step - loss: 0.8641 - accuracy: 0.5704 - val_loss: 1.0591 - val_accuracy: 0.4872
Epoch 445/1000
270/270 [==============================] - 0s 160us/step - loss: 0.8645 - accuracy: 0.5704 - val_loss: 1.0575 - val_accuracy: 0.4872
Epoch 446/1000
270/270 [==============================] - 0s 187us/step - loss: 0.8646 - accuracy: 0.5704 - val_loss: 1.0573 - val_accuracy: 0.4872
Epoch 447/1000
270/270 [==============================] - 0s 292us/step - loss: 0.8644 - accuracy: 0.5704 - val_loss: 1.0587 - val_accuracy: 0.4872
Epoch 448/1000
270/270 [==============================] - 0s 162us/step - loss: 0.8632 - accuracy: 0.5704 - val_loss: 1.0607 - 

270/270 [==============================] - 0s 97us/step - loss: 0.8637 - accuracy: 0.5704 - val_loss: 1.0642 - val_accuracy: 0.4872
Epoch 553/1000
270/270 [==============================] - 0s 88us/step - loss: 0.8686 - accuracy: 0.5704 - val_loss: 1.0650 - val_accuracy: 0.4872
Epoch 554/1000
270/270 [==============================] - 0s 78us/step - loss: 0.8647 - accuracy: 0.5704 - val_loss: 1.0580 - val_accuracy: 0.4872
Epoch 555/1000
270/270 [==============================] - 0s 79us/step - loss: 0.8635 - accuracy: 0.5704 - val_loss: 1.0571 - val_accuracy: 0.4872
Epoch 556/1000
270/270 [==============================] - 0s 95us/step - loss: 0.8627 - accuracy: 0.5704 - val_loss: 1.0642 - val_accuracy: 0.4872
Epoch 557/1000
270/270 [==============================] - 0s 95us/step - loss: 0.8611 - accuracy: 0.5704 - val_loss: 1.0598 - val_accuracy: 0.4872
Epoch 558/1000
270/270 [==============================] - 0s 75us/step - loss: 0.8640 - accuracy: 0.5704 - val_loss: 1.0614 - val_acc

Epoch 663/1000
270/270 [==============================] - 0s 106us/step - loss: 0.8616 - accuracy: 0.5704 - val_loss: 1.0703 - val_accuracy: 0.4872
Epoch 664/1000
270/270 [==============================] - 0s 115us/step - loss: 0.8626 - accuracy: 0.5704 - val_loss: 1.0645 - val_accuracy: 0.4872
Epoch 665/1000
270/270 [==============================] - 0s 125us/step - loss: 0.8616 - accuracy: 0.5704 - val_loss: 1.0639 - val_accuracy: 0.4872
Epoch 666/1000
270/270 [==============================] - 0s 98us/step - loss: 0.8611 - accuracy: 0.5667 - val_loss: 1.0650 - val_accuracy: 0.4872
Epoch 667/1000
270/270 [==============================] - 0s 111us/step - loss: 0.8677 - accuracy: 0.5704 - val_loss: 1.0752 - val_accuracy: 0.4872
Epoch 668/1000
270/270 [==============================] - 0s 87us/step - loss: 0.8613 - accuracy: 0.5704 - val_loss: 1.0637 - val_accuracy: 0.4872
Epoch 669/1000
270/270 [==============================] - 0s 85us/step - loss: 0.8620 - accuracy: 0.5704 - val_los

Epoch 774/1000
270/270 [==============================] - 0s 166us/step - loss: 0.8617 - accuracy: 0.5704 - val_loss: 1.0727 - val_accuracy: 0.4872
Epoch 775/1000
270/270 [==============================] - 0s 144us/step - loss: 0.8667 - accuracy: 0.5222 - val_loss: 1.0634 - val_accuracy: 0.5385
Epoch 776/1000
270/270 [==============================] - 0s 150us/step - loss: 0.8667 - accuracy: 0.5593 - val_loss: 1.0745 - val_accuracy: 0.4872
Epoch 777/1000
270/270 [==============================] - 0s 146us/step - loss: 0.8591 - accuracy: 0.5704 - val_loss: 1.0735 - val_accuracy: 0.4872
Epoch 778/1000
270/270 [==============================] - 0s 129us/step - loss: 0.8638 - accuracy: 0.5296 - val_loss: 1.0648 - val_accuracy: 0.4872
Epoch 779/1000
270/270 [==============================] - 0s 84us/step - loss: 0.8598 - accuracy: 0.5704 - val_loss: 1.0630 - val_accuracy: 0.4872
Epoch 780/1000
270/270 [==============================] - 0s 134us/step - loss: 0.8614 - accuracy: 0.5704 - val_l

Epoch 884/1000
270/270 [==============================] - 0s 103us/step - loss: 0.8579 - accuracy: 0.5704 - val_loss: 1.0766 - val_accuracy: 0.4786
Epoch 885/1000
270/270 [==============================] - 0s 132us/step - loss: 0.8632 - accuracy: 0.5704 - val_loss: 1.0770 - val_accuracy: 0.4872
Epoch 886/1000
270/270 [==============================] - 0s 123us/step - loss: 0.8579 - accuracy: 0.5667 - val_loss: 1.0691 - val_accuracy: 0.4872
Epoch 887/1000
270/270 [==============================] - 0s 150us/step - loss: 0.8597 - accuracy: 0.5667 - val_loss: 1.0708 - val_accuracy: 0.4872
Epoch 888/1000
270/270 [==============================] - 0s 144us/step - loss: 0.8588 - accuracy: 0.5704 - val_loss: 1.0798 - val_accuracy: 0.4872
Epoch 889/1000
270/270 [==============================] - 0s 112us/step - loss: 0.8606 - accuracy: 0.5704 - val_loss: 1.0814 - val_accuracy: 0.4872
Epoch 890/1000
270/270 [==============================] - 0s 200us/step - loss: 0.8601 - accuracy: 0.5704 - val_

270/270 [==============================] - 0s 113us/step - loss: 0.8622 - accuracy: 0.5704 - val_loss: 1.0786 - val_accuracy: 0.4872
Epoch 995/1000
270/270 [==============================] - 0s 123us/step - loss: 0.8604 - accuracy: 0.5667 - val_loss: 1.0753 - val_accuracy: 0.4786
Epoch 996/1000
270/270 [==============================] - 0s 108us/step - loss: 0.8584 - accuracy: 0.5704 - val_loss: 1.0807 - val_accuracy: 0.4872
Epoch 997/1000
270/270 [==============================] - 0s 96us/step - loss: 0.8582 - accuracy: 0.5704 - val_loss: 1.0836 - val_accuracy: 0.4872
Epoch 998/1000
270/270 [==============================] - 0s 81us/step - loss: 0.8615 - accuracy: 0.5704 - val_loss: 1.0834 - val_accuracy: 0.4872
Epoch 999/1000
270/270 [==============================] - 0s 91us/step - loss: 0.8640 - accuracy: 0.5185 - val_loss: 1.0801 - val_accuracy: 0.5385
Epoch 1000/1000
270/270 [==============================] - 0s 77us/step - loss: 0.8627 - accuracy: 0.5519 - val_loss: 1.0880 - val

In [284]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist2_over4.history['accuracy'])*100))

over-sampling train accuracy: 56.78%


In [65]:
df_proba8 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_lasso_2.xlsx",
                        sheet_name=3,
                        index_col=None)

In [66]:
df_proba8

,phage,strain,phenotype,prediction,0,1,2
0,p0006kpresabs_qual,NRS236,1,2,1.321970e-02,2.446264e-01,7.421539e-01
1,p0006kpresabs_qual,NRS113,2,2,3.478230e-02,2.806685e-01,6.845492e-01
2,p0006kpresabs_qual,CFBRSa23,0,0,4.090251e-01,3.405008e-01,2.504741e-01
3,p0006kpresabs_qual,NRS249,2,1,1.987907e-01,5.331044e-01,2.681049e-01
4,p0006kpresabs_qual,107,1,0,4.090251e-01,3.405008e-01,2.504741e-01
...,...,...,...,...,...,...,...
984,p0017Skpresabs_qual,CFBRSa30,0,0,7.207667e-01,2.792331e-01,2.571588e-07
985,p0017Skpresabs_qual,NRS383,1,0,6.129044e-01,3.870795e-01,1.601290e-05
986,p0017Skpresabs_qual,NRS110,2,2,3.260306e-07,7.910664e-07,9.999989e-01
987,p0017Skpresabs_qual,NRS209,2,2,3.604249e-12,2.698129e-07,9.999998e-01


In [67]:
y_prob8 = df_proba8[df_proba8['phage']=='p0006kpresabs_qual'].iloc[:,-3:]
y_prob8 = y_prob8.to_numpy()
y_prob8

array([[1.3219704e-02, 2.4462637e-01, 7.4215394e-01],
       [3.4782300e-02, 2.8066847e-01, 6.8454915e-01],
       [4.0902510e-01, 3.4050083e-01, 2.5047413e-01],
       [1.9879068e-01, 5.3310440e-01, 2.6810485e-01],
       [4.0902510e-01, 3.4050083e-01, 2.5047413e-01],
       [1.3815957e-01, 3.0921945e-01, 5.5262100e-01],
       [4.0902510e-01, 3.4050083e-01, 2.5047413e-01],
       [6.3482640e-01, 2.2028415e-01, 1.4488940e-01],
       [4.0902510e-01, 3.4050083e-01, 2.5047413e-01],
       [6.8404920e-01, 1.0945539e-01, 2.0649539e-01],
       [4.0902510e-01, 3.4050083e-01, 2.5047413e-01],
       [2.1217002e-02, 1.9669692e-01, 7.8208610e-01],
       [4.0902510e-01, 3.4050083e-01, 2.5047413e-01],
       [1.0336818e-02, 1.0571358e-02, 9.7909180e-01],
       [4.0902510e-01, 3.4050083e-01, 2.5047413e-01],
       [1.5825848e-05, 1.8219705e-06, 9.9998236e-01],
       [1.9879068e-01, 5.3310440e-01, 2.6810485e-01],
       [4.0902510e-01, 3.4050083e-01, 2.5047413e-01],
       [1.3219704e-02, 2.446

In [68]:
ovo8 = rocauc_ovo(y_sel_test_over, y_prob8, average="macro", multi_class="ovo")
ovo8

0.7145518299364454

In [69]:
ovr8 = rocauc_ovr(y_sel_test_over, y_prob8, average="macro", multi_class="ovr")
ovr8

0.7145518299364454

In [70]:
ovos2 = [ovo5, ovo6, ovo7, ovo8]
np.mean(ovos2)

0.6888286215209293

In [71]:
np.std(ovos2)

0.029825730529423274

In [72]:
ovrs2 = [ovr5, ovr6, ovr7, ovr8]
np.mean(ovrs2)

0.6888286215209293

In [73]:
np.std(ovrs2)

0.029825730529423274

In [285]:
accs_l_over = [acc_test2_over, acc_test2_over2, acc_test2_over3, acc_test2_over4]

In [286]:
mean_l_over = np.mean(accs_l_over)
print('over-sampling test accuracy mean after lasso: %.2f%%' % (mean_l_over*100))

over-sampling test accuracy mean after lasso: 51.07%


In [287]:
std_l_over = np.std(accs_l_over)
print('over-sampling test accuracy standard deviation after lasso:', std_l_over)

over-sampling test accuracy standard deviation after lasso: 0.03594574584774779


In [288]:
accs_train_l_over = [np.mean(hist2_over.history['accuracy']), np.mean(hist2_over2.history['accuracy']), np.mean(hist2_over3.history['accuracy']),
             np.mean(hist2_over4.history['accuracy'])]

In [289]:
mean_train_l_over = np.mean(accs_train_l_over)
print('over-sampling train accuracy mean after lasso: %.2f%%' % (mean_train_l_over*100))

over-sampling train accuracy mean after lasso: 54.87%


In [290]:
std_train_l_over = np.std(accs_train_l_over)
print('over-sampling train accuracy standard deviation after lasso:', std_train_l_over)

over-sampling train accuracy standard deviation after lasso: 0.013600758
